# 1. Libs

In [1]:
if 'SKIP' not in globals() or not SKIP:   # torch
    print('libs torch v4.2')

    # 4.2: cleanup of unused and add bits=16
    # 4.1: pick_audio_segment returns 7s interval instead of offset
    # 4.0: h5 dataset with custom filter
    # 3.6: aws changes in train
    # 3.5: transform_to_spec with parameter on number of bits for audio
    # 3.4: changed get_checkpoint_path to pick last epoch in case of tie
    # 3.3: fixed bug in get_checkpoint_path
    # 3.2: Separate config
    # 3.1: fix load_state
    # 3.0: initial

    # ----------------------------------------------------------------------------------------------------
    # 0.0 imports
    # ----------------------------------------------------------------------------------------------------

    import warnings
    warnings.filterwarnings('ignore')

    import pyarrow.parquet as pq

    import copy
    from contextlib import contextmanager, nullcontext
    
    import os
    import gc
    import time
    import pickle
    import psutil
    import ast
    import copy
    import math
    import random
    import pdb
    import json
    import glob
    import shutil

    # from tqdm import tqdm
    from tqdm.autonotebook import tqdm
    import wandb
    from collections import deque
    import numba as nb

    import numpy as np
    import numpy.matlib
    import pandas as pd
    # import datatable as dt
    # import polars

    pd.set_option('display.max_columns', 300)
    pd.set_option('display.max_rows', 100)
    pd.set_option("display.width", 150)
    np.set_printoptions(linewidth=140)

    from multiprocessing.pool import ThreadPool

    # torch
    import torch
    # from torch import cuda
    from torch import nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, IterableDataset, DataLoader
    from torchinfo import summary
    import pytorch_lightning as L
    from pytorch_lightning.utilities.model_summary import ModelSummary
    from pytorch_lightning.callbacks import (LearningRateMonitor, ModelCheckpoint, TQDMProgressBar, StochasticWeightAveraging)
    from pytorch_lightning.loggers import WandbLogger
    # from pytorch_lightning.tuner.tuning import Tuner
    import torchaudio
    import torchaudio.transforms as T
    # from torchvision import transforms
    import torchvision
    from torchvision.transforms import v2
    from torch.distributions import Beta
    import cv2
    from PIL import Image, ImageOps

    try:
        from lion_pytorch import Lion
    except:
        pass

    from kaggle_secrets import UserSecretsClient
    import os
    #os.environ["TOKENIZERS_PARALLELISM"] = "false"
    # os.environ['CUDA_VISIBLE_DEVICES'] = '1'

    # visual models
    import timm
    from timm.models.layers import drop_path

    import albumentations as A

    # metrics
    import sklearn.metrics

    # multiprocessing
    # import joblib
    from joblib import Parallel, delayed
    from joblib.externals.loky.backend.context import get_context
    import concurrent.futures

    # onnx
    try:
        import onnx
        import onnxruntime as ort
    except:
        pass

    try:
        import openvino.runtime as ovrt
    except:
        pass

    # ----------------------------------------------------------------------------------------------------
    # 0.0 constants
    # ----------------------------------------------------------------------------------------------------

    STRICT = False

    SR = 32_000
    
    BIRDS = [
        '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116',
        '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010',
        '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513',
        '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082',
        '67252', '714022', '715170', '787625', '81930', '868458', '963335', 
        'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 
        'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1',
        'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1',
        'colcha1', 'compau', 'compot1', 'cotfly1', 'crbtan1', 'crcwoo1', 'crebob1', 'cregua1', 'creoro1', 'eardov1', 'fotfly', 'gohman1',
        'grasal4', 'grbhaw1', 'greani1', 'greegr', 'greibi1', 'grekis', 'grepot1', 'gretin1', 'grnkin', 'grysee1', 'gybmar', 'gycwor1', 
        'labter1', 'laufal1', 'leagre', 'linwoo1', 'littin1', 'mastit1', 'neocor', 'norscr1', 'olipic1', 'orcpar', 'palhor2', 'paltan1',
        'pavpig2', 'piepuf1', 'pirfly1', 'piwtyr1', 'plbwoo1', 'plctan1', 'plukit1', 'purgal2', 'ragmac1', 'rebbla1', 'recwoo1', 'rinkin1',
        'roahaw', 'rosspo1', 'royfly1', 'rtlhum', 'rubsee1', 'rufmot1', 'rugdov', 'rumfly1', 'ruther1', 'rutjac1', 'rutpuf1', 'saffin',
        'sahpar1', 'savhaw1', 'secfly1', 'shghum1', 'shtfly1', 'smbani', 'snoegr', 'sobtyr1', 'socfly1', 'solsan', 'soulap1', 'spbwoo1',
        'speowl1', 'spepar1', 'srwswa1', 'stbwoo2', 'strcuc1', 'strfly1', 'strher', 'strowl1', 'tbsfin1', 'thbeup1', 'thlsch3', 'trokin',
        'tropar', 'trsowl', 'turvul', 'verfly', 'watjac1', 'wbwwre1', 'whbant1', 'whbman1', 'whfant1', 'whmtyr1', 'whtdov', 'whttro1',
        'whwswa1', 'woosto', 'y00678', 'yebela1', 'yebfly1', 'yebsee1', 'yecspi2', 'yectyr1', 'yehbla2', 'yehcar1', 'yelori1', 'yeofly1',
        'yercac1', 'ywcpar', 
    ]

    N_BIRDS = len(BIRDS)
    RANGE_BIRDS = range(N_BIRDS)
    
    label_to_num = dict(zip(BIRDS, RANGE_BIRDS))

    RARE_BIRDS = [ # 39 birds
        '1139490', '1192948', '1194042', '126247', '1346504', '134933', '1462711', '1462737', '1564122',
        '21038', '21116', '24272', '24292', '41778', '42087', '42113', '46010', '47067', '476537',
        '476538', '523060', '528041', '548639', '555142', '64862', '65336', '65419', '65547', '66016',
        '66531', '66578', '66893', '67082', '714022', '787625', '81930', '868458', '963335', 'plctan1',
    ]
    
    DIFFICULT_BIRDS = [
        
    ]

    # ----------------------------------------------------------------------------------------------------
    # 0.1 utils
    # ----------------------------------------------------------------------------------------------------

    class dotdict(dict):
        def __getattr__(self, name):
            return self.get(name, None)

        def __setattr__(self, name, val):
            self[name] = val

        def __set__(self, name, val):
            self.__setattr__(name, val)
            
        def to_dict(self):
            return eval(str(self))

        # the following are required by save_obj
        def __getstate__(self):
            return self.__dict__

        def __setstate__(self, d):
            self.__dict__.update(d)


    class Log:
        def __init__(self, log_path, time_key=True):
            self.path = log_path
            if time_key:
                self.path = self.path.replace('.','{}.'.format(time.strftime('_%Y%m%d%H%M%S', time.localtime(time.time()))))
            print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), file=open(self.path,'a+'))
            print('log path:', self.path)
            print('------------ begin ------------', file=open(self.path, 'a+'))

        def __call__(self, *content):
            t1 = time.strftime('%H:%M:%S', time.localtime(time.time()))
            print(*content)
            print(t1, *content, file=open(self.path,'a+'))

        def clean(self):
            print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), file=open(self.path,'w'))
            print('------------ begin -------------', file=open(self.path,'a+'))


    def is_interactive():
        return os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'


    class Config:
        def __getattr__(self, name):
            """ retun None if attribute doesn't exist """
            return None

        def __repr__(self):
            out = dict()
            names = dir(self)
            for key in names:
                if not key.startswith('__') and key != 'to_dict':
                    out[key] = getattr(self, key)
            return str(out)

        def to_dict(self):
            config = dotdict(eval(str(self)))
            if config.model:
                config.model = dotdict(config.model)
            if config.aug:
                config.aug = dotdict(config.aug)
            return config


    def save_config():
        name = f'config {config.name}.json'
        folder = f'./{config.name}' if os.path.exists(config.name) else '.'
            
        with open(f'{folder}/{name}', 'w') as f:
            json.dump(config.to_dict(), f)


    def load_config(name, folder=None):
        name = f'{folder}/config {name}.json' if folder is not None else f'config {name}.json'
        with open(name, 'r') as f:
            config = dotdict(json.load(f))
        return config


    class Timer:
        def __init__(self, margin=1.1):
            self.margin = margin

        def start(self, duration):
            self.laps = 1
            self.start_time = time.time()
            self.duration = duration

        def lap(self):
            elapsed = time.time() - self.start_time
            mean_lap = elapsed / self.laps
            if elapsed + self.margin * mean_lap > self.duration:
                return False

            self.laps += 1
            return True


    def fillna_df(df, values=0):
        if not hasattr(values, '__len__'):
            values = [values] * df.shape[-1]
        numeric_columns = [c for c in df.columns if df[c].dtype not in ['object', 'bool']]
        for i, col in enumerate(numeric_columns):
            a = df[col].values
            if np.isnan(a.sum()):
                df[col] = np.where(np.isnan(a), values[i], a)


    def wait():
        for i in range(10000):
            print('.', end='')
            time.sleep(60)


    def save_obj(obj, name, protocol=4): # pickle.HIGHEST_PROTOCOL):
        with open('./'+ name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, protocol)


    def load_obj(name, folder=''):
        name = name.replace('.pkl', '')
        with open(folder + name + '.pkl', 'rb') as f:
            return pickle.load(f)


    @contextmanager
    def timer(name=''):
        s = time.time()
        mem_before = psutil.virtual_memory().used/1024/1024/1024
        yield
        elapsed = time.time() - s
        mem_after = psutil.virtual_memory().used/1024/1024/1024
        print(f'[{name}] {elapsed: .3f} s   {mem_before: .2f} GB -> {mem_after: .2f} GB')


    def seed_everything(seed_value=42):
        os.environ['PYTHONHASHSEED'] = str(seed_value)
        np.random.seed(seed_value)
        random.seed(seed_value)
        try:
            tf.random.set_seed(seed_value)
        except:
            pass

        try:
            torch.manual_seed(seed_value)
            torch.cuda.manual_seed(seed_value)
            torch.cuda.manual_seed_all(seed_value)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        except:
            pass


    def get_gpu_ram():
        """ nvidia GPU memory utilization in MB """
        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        info = nvmlDeviceGetMemoryInfo(handle)
        return info.used/1024/1024


    def reduce_mem_usage(df, do_categoricals=False, do_float=False):
        """ iterate through all the columns of a dataframe and modify the data type
            to reduce memory usage.
        """
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

        for col in df.columns:
            col_type = df[col].dtype

            if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                    #print(col, c_min, c_max, df[col].dtype)
                elif do_float:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                if do_categoricals==True:
                    df[col] = df[col].astype('category')

        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        return df


    def memory_cleanup():
        """
        Cleans up GPU memory. Call after a fold is trained.
        https://github.com/huggingface/transformers/issues/1742
        """
        for obj in gc.get_objects():
            if torch.is_tensor(obj):
                del obj
        gc.collect()
        torch.cuda.empty_cache()


    def clean_config(models_config, models=None):
        out, out_dict = [], dict()
        for row in models_config:
            if len(row) == 5:
                name, folds, file, folder, config = row
                folder = f'{ROOT}/{folder}'
            elif len(row) == 4:
                name, folds, folder, config = row
                file = name
                folder = f'{ROOT}/{folder}'
            else:
                name, folds, config = row
                file = name
                folder = f'{ROOT}/b5-models'
                
            if models is not None and not name in models:
                continue
            config = dotdict(config)
            if config.aug:
                config.aug = dotdict(config.aug)
            out.append((name, folds, file, folder, config))
            out_dict[name] = config
        return out, out_dict
        
    
    def print_num_parameters(model, note=None):
        n = sum(p.numel() for p in model.parameters())
        if note:
            print(f'{note:<30}: {n:,.0f}')
        return n
    
    
    def print_model_size(pl_model):
        """ print size of lightning model. Adjust example_input_array based on model input """
        size = ModelSummary(pl_model, max_depth=0).trainable_parameters
        model_name = str(pl_model.model.__class__).split('.')[-1][:-2]
        print(f'model {model_name}: {size:,.0f}')
        return size


    def print_weights(model):
        """ prints model weights and returns them as a dataframe """
        items = []
        for i, (name, module) in enumerate(model.named_modules()):
            weight = module.weight.shape if hasattr(module, 'weight') else ''
            items.append((name, f'{module.__class__.__module__}.{module.__class__.__name__}', weight))

        res = pd.DataFrame(items, columns=['name', 'class', 'weight'])
        res.to_csv('x3d_l.csv')
        pd.set_option('display.max_rows', None)
        display(res)
        return res


    def are_weights_equal(state_dict1, state_dict2):
        if len(state_dict1) != len(state_dict2):
            return False
        for (key1, value1), (key2, value2) in zip(state_dict1.items(), state_dict2.items()):
            if key1 != key2 or not torch.equal(value1, value2):
                return False
        return True

    # ----------------------------------------------------------------------------------------------------
    # 0.2 Layers
    # ----------------------------------------------------------------------------------------------------

    def glorot_uniform(parameter):
        nn.init.xavier_uniform_(parameter.data, gain=1.0)
            

    class Sinusoidal_positional_embedding(nn.Module):
        """ performs fourier positional embedding converting from (B, T) to (B, T, E),
            batch, time (sequence), embedding.
            Used by P2 in iceCube to embed continuous variables (categorical used 
            regular embedding. Unlike the positional encoding in the attention is all
            you need paper, which provides a different embedding for each embedding position 
            and position in the sequence, this function only provides a different embedding
            for each embedding position (all positions in the sequence are embded in the same way).
        """
        
        def __init__(self, e_dim=16, M=10_000):
            super().__init__()
            self.e_dim = e_dim  # embedding e_dimension
            self.M = M

        def forward(self, x):
            device = x.device
            half_e_dim = self.e_dim // 2
            emb = math.log(self.M) / half_e_dim
            emb = torch.exp(torch.arange(half_e_dim, device=device) * (-emb))
            emb = x[..., None] * emb[None, ...]
            emb = torch.cat((emb.sin(), emb.cos()), e_dim=-1)
            return emb


    class DropPath(nn.Module):
        """ drop entire all features for random instances of dim[-2] (regular drop drops
            random features)
        """
    
        def __init__(self, drop_prob=None):
            super(DropPath, self).__init__()
            self.drop_prob = drop_prob

        def forward(self, x):
            return drop_path(x, self.drop_prob, self.training)

        def extra_repr(self) -> str:
            return "p={}".format(self.drop_prob)
            

    class GaussianNoiseRandom(object):
        """ add random noise with random std.
            Noise is randomized across last two dimensions (h,w)
        """
        def __init__(self, mean=0., std_range=(0.05, 0.2)):
            self.mean = mean
            self.std_range = std_range

        def __call__(self, tensor):
            std = random.uniform(*self.std_range)  # Randomly choose a std deviation each time
            # Generate noise only for one channel
            single_channel_noise = torch.randn(tensor[0, :, :].size()) * std + self.mean
            # Expand noise to all channels
            noise = single_channel_noise.repeat(tensor.size(0), 1, 1)
            return tensor + noise

        def __repr__(self):
            return f'{self.__class__.__name__}(mean={self.mean}, std_range={self.std_range})'


    class GaussianNoiseLearned(nn.Module):
        """ add random noise with random std with learnable std. 
            Noise is randomized across all dimensions.

        Args:
            sigma (float, optional): relative standard deviation used to generate the
                noise. Relative means that it will be multiplied by the magnitude of
                the value your are adding the noise to. This means that sigma can be
                the same regardless of the scale of the vector.
            is_relative_detach (bool, optional): whether to detach the variable before
                computing the scale of the noise. If `False` then the scale of the noise
                won't be seen as a constant but something to optimize: this will bias the
                network to generate vectors with smaller values.
        """

        def __init__(self, sigma=0.1, is_relative_detach=True):
            super().__init__()
            self.sigma = sigma
            self.is_relative_detach = is_relative_detach
            self.noise = torch.tensor(0).to(config.device)

        def forward(self, x):
            if self.training and self.sigma != 0:
                scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
                sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
                x = x + sampled_noise
            return x
            

    class Model_head(nn.Module):
        def __init__(self, input_dim, output_dim=1, dropout_head=None, criterion=None):
            super(Model_head, self).__init__()
            self.criterion = criterion
            self.dropout_head = dropout_head
            if criterion is not None:
                if dropout_head is None:
                    self.dropout1 = nn.Dropout(0.1)
                    self.dropout2 = nn.Dropout(0.2)
                    self.dropout3 = nn.Dropout(0.3)
                    self.dropout4 = nn.Dropout(0.4)
                    self.dropout5 = nn.Dropout(0.5)
                else:
                    self.dropout = nn.Dropout(dropout_head)
                
            self.regressor = nn.Linear(input_dim, output_dim)
            glorot_uniform(self.regressor.weight)
            #self._init_weights(self.regressor)
            
        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)
            
        def forward(self, x, target=None):
            # x is B x S x C
            if self.criterion is not None and target is not None:
                if self.dropout_head is None:
                    logits1 = self.regressor(self.dropout1(x))
                    logits2 = self.regressor(self.dropout2(x))
                    logits3 = self.regressor(self.dropout3(x))
                    logits4 = self.regressor(self.dropout4(x))
                    logits5 = self.regressor(self.dropout5(x))
                    logits = ((logits1 + logits2 + logits3 + logits4 + logits5) / 5)

                    loss1 = self.criterion(logits1, target)
                    loss2 = self.criterion(logits2, target)
                    loss3 = self.criterion(logits3, target)
                    loss4 = self.criterion(logits4, target)
                    loss5 = self.criterion(logits5, target)
                    loss = (loss1 + loss2 + loss3  + loss4 + loss5) / 5
                else:
                    logits = self.regressor(self.dropout(x))
                    loss = self.criterion(logits, target)

                #print(loss.shape, attention_mask.shape)
                loss = loss.mean()
            else:
                logits = self.regressor(x)
                loss = 0

            logits = logits.squeeze()
            return loss, logits


    class GeM(nn.Module):
        """ average_pool_2D with power parameter """
        def __init__(self, p=1.5, eps=1e-6):
            super().__init__()
            self.p = nn.Parameter(torch.ones(1) * p)
            self.eps = eps

        def forward(self, x):
            # return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))).pow(1.0 / self.p)
            x = x.clamp(min=self.eps).pow(self.p)
            return x.mean([2,3]).pow(1.0 / self.p)

        def __repr__(self):
            return (self.__class__.__name__
                + f"(p={self.p.data.tolist()[0]:.4f}, eps={self.eps})")

    
    def init_layer(layer):
        nn.init.xavier_uniform_(layer.weight)
        if hasattr(layer, "bias"):
            if layer.bias is not None:
                layer.bias.data.fill_(0.0)


    def init_bn(bn):
        bn.bias.data.fill_(0.0)
        bn.weight.data.fill_(1.0)


    class AttBlockV2(nn.Module):
        def __init__(self, in_features, out_features=N_BIRDS, activation="sigmoid"):
            super().__init__()
            self.activation = activation
            self.att = nn.Conv1d(
                in_channels=in_features, out_channels=out_features,
                kernel_size=1, stride=1, padding=0, bias=True)
            self.cla = nn.Conv1d(in_channels=in_features, out_channels=out_features,
                kernel_size=1, stride=1, padding=0, bias=True)
            self._init_weights()

        def _init_weights(self):
            init_layer(self.att)
            init_layer(self.cla)

        def forward(self, x):
            # x: (n_samples, n_in, n_time)
            norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
            cla = self.nonlinear_transform(self.cla(x))
            x = torch.sum(norm_att * cla, dim=2)
            return x, norm_att, cla

        def nonlinear_transform(self, x):
            if self.activation == "linear":
                return x
            elif self.activation == "sigmoid":
                return torch.sigmoid(x)

    # ----------------------------------------------------------------------------------------------------
    # 0.3 Models
    # ----------------------------------------------------------------------------------------------------

    ENCODER_SIZE = { # dim_channels, channels first
        'efficientnet_b0': (1280, True),
        'efficientnet_b1': (1280, True),
        'efficientnet_b2': (1408, True),
        'efficientnet_b3': (1536, True),
        'efficientnet_b4': (1792, True),
        'efficientnet_b5': (2048, True),
        'efficientnet_b6': (2304, True),
        'efficientnet_b7': (2560, True),
        
        'swinv2_tiny_window16_256': (768, False),           # (3, 256, 256) -> (8, 8, 768)    h,w = 256; bs=8, lr=1e-4
        'maxvit_base_tf_512.in21k_ft_in1k': (768, True),    # (3, 512, 512) -> (768, 16, 16)  h,w = 512; bs=1 crashes
        'tiny_vit_21m_512': (576, True),                    # (3, 512, 512) -> (576, 16, 16)  h,w = multiple of 16; bs=2, lr=1e-3
        'tiny_vit_21m_224': (576, True),                    # (3, 224, 224) -> (576, 14, 14)  h,w = multiple of 16; bs=8, lr=1e-3
        'efficientvit_b2': (None, None),
    }

    class SpecNetImg(nn.Module):
        """ models that reads a 2D image directly from dataset """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True, resize=None, add_position=False,
                     pretrained=True, submit=False, in_chans=3, **kwargs):
            super().__init__()
            self.submit = submit
            self.in_chans = in_chans
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            if add_position:
                rows, cols = config.img_dim
                row_pos = torch.arange(rows).unsqueeze(1).expand(rows, cols)
                col_pos = torch.arange(cols).unsqueeze(0).expand(rows, cols)
                self.pos = torch.stack((row_pos, col_pos), dim=0).unsqueeze(0)
            else:
                self.pos = None
            
            self.resize = v2.Resize(resize) if resize else None
            
            if isinstance(pretrained, str):
                self.model = timm.create_model(encoder, pretrained=False, in_chans=in_chans)
                load_encoder_state(self.model, pretrained)
            else:
                self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=in_chans)
            n_channels = self.model.num_features
            #if config.transformer:
            #    self.pool = lambda x: x.mean([1])
            #else:
            self.pool = nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
            self.drop = nn.Dropout(p=head_dropout)
            self.fc = nn.Linear(n_channels, out_features=num_classes, bias=True)

        def forward(self, x):
            B = x.size()[0]
            if self.resize is not None:
                x = self.resize(x)
            if self.pos is not None:
                pos = self.pos.expand(B,-1,-1,-1).to(x.device)
                x = torch.concat((x, pos), dim=1)
            elif self.in_chans != x.shape[1]:
                x = x.expand(-1, 3, -1, -1)     # C3HW
            x = self.model.forward_features(x)  # BHWC
            x = self.pool(x)                    # BHWC -> BC or BC11
            x = x.view(B, -1)  # flatten
            x = self.drop(x)
            x = self.fc(x)
            return x

        # def _load_encoder_state(self, pretrained):
        #     chkp = torch.load(pretrained)
        #     encoder_state = {k.replace('backbone.0', 'stem').replace('backbone.1', 'stages')
        #                       .replace('backbone.2', 'final_conv').replace('global_pool.', ''): v 
        #                          for k, v in chkp.items()}
        #     self.model.load_state_dict(encoder_state, strict=False)
            

    class Cola(nn.Module):
        """ model for cola unsupervised training. Takes a batch of pairs of images (anchors and positives)
            and returns y_hat (projection(anchors) @ projection(positives). Must be compared with a sparse
            label consisting of a range (F.cross_entropy(y_hat, torch.arange(x1.size(0)))
        """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True,
                     pretrained=True, submit=False, in_chans=3, **kwargs):
            super().__init__()
            self.submit = submit
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            dim = 512
            
            self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=in_chans)
            n_channels = self.model.num_features
            self.pool = nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
            self.g = torch.nn.Linear(n_channels, dim)
            self.ln = torch.nn.LayerNorm(normalized_shape=dim)
            self.drop = nn.Dropout(p=head_dropout)
            self.linear = torch.nn.Linear(dim, dim, bias=False)

        def forward(self, x1, x2):
            B = x1.size()[0]
            x1 = x1.expand(-1, 3, -1, -1)     # C3HW
            x2 = x1.expand(-1, 3, -1, -1)     # C3HW

            x1 = self.model.forward_features(x1)
            x1 = self.pool(x1)   # BHWC -> BC or BC11
            x1 = x1.view(B, -1)  # flatten
            x1 = self.g(x1)
            x1 = self.drop(torch.tanh(self.ln(x1)))
            x2 = self.model.forward_features(x2)
            x2 = self.pool(x2)   # BHWC -> BC or BC11
            x2 = x2.view(B, -1)  # flatten
            x2 = self.g(x2)
            x2 = self.drop(torch.tanh(self.ln(x2)))
            x1 = self.linear(x1)
            y_hat = torch.mm(x1, x2.t())
            return y_hat


    class SpecNetImgGem(nn.Module):
        """ models that reads a 2D image directly from dataset. Takes last channels from encoder and 
            consolidates with GeM neck
        """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True, resize=None,
                     pretrained=True, submit=False, out_indices=2, in_chans=3, **kwargs):
            super().__init__()
            self.submit = submit
            self.in_chans = in_chans
            self.out_indices = out_indices
            encoder = config.encoder or 'efficientnet_b0'
            p = config.gem_p or 2.
            head_dropout = config.head_dropout or 0.
            self.resize = v2.Resize(resize) if resize else None
            
            if isinstance(pretrained, str):
                self.model = timm.create_model(encoder, pretrained=False, in_chans=in_chans, features_only=True)
                load_encoder_state(self.model, pretrained)
            else:
                self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=in_chans, features_only=True)
            self.pools = torch.nn.ModuleList([GeM(p=p) for i in range(out_indices)])
            mid_features = self.model.feature_info.channels()[-out_indices:]
            self.n_mid_features = sum(mid_features)
            self.drop = nn.Dropout(p=head_dropout) if head_dropout > 0 else None
            self.bn = torch.nn.BatchNorm1d(self.n_mid_features)
            self.fc = nn.Linear(self.n_mid_features, out_features=num_classes, bias=True)

        def forward(self, x):
            B = x.size()[0]
            if self.resize is not None:
                x = self.resize(x)
            elif self.in_chans != x.shape[1]:
                x = x.expand(-1, 3, -1, -1)     # C3HW
            x = self.model(x)[-self.out_indices:]   # n*BHWC
            if self.out_indices > 1:
                h = torch.cat([pool(m) for m, pool in zip(x, self.pools)], dim=1)  # n*BHWC -> BC or BC11
                x = h.view(B, -1)  # flatten
            else:
                x = self.pools[0](x[0])
            if self.drop is not None:
                x = self.drop(x)
            x = self.bn(x)
            x = self.fc(x)
            return x


    class SpecNetImgSed(nn.Module):
        """ models that reads a 2D image directly from dataset. Takes last channels from encoder and 
            consolidates with GEM.
        """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True, resize=None, add_position=False,
                     pretrained=True, submit=False, out_indices=2, **kwargs):
            super().__init__()
            self.submit = submit
            self.out_indices = out_indices
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            self.model = timm.create_model(
                encoder, pretrained=pretrained, in_chans=3, features_only=True, drop_path_rate=0.2, drop_rate=0.5)
            self.pools = torch.nn.ModuleList([nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
                    for i in range(out_indices)])
            mid_features = self.model.feature_info.channels()[-out_indices:]
            self.n_mid_features = sum(mid_features)
            self.drop = nn.Dropout(p=head_dropout) if head_dropout > 0 else None
            self.fc1 = nn.Linear(in_features, in_features, bias=True)
            self.att_block = AttBlockV2(in_features, self.num_classes, activation="sigmoid")


            self.bn = torch.nn.BatchNorm1d(self.n_mid_features)
            self.fc = nn.Linear(self.n_mid_features, out_features=num_classes, bias=True)

        def forward(self, x):
            B = x.size()[0]
            x = x.expand(-1, 3, -1, -1)         # C3HW
            x = self.model(x)[-self.out_indices:]   # n*BHWC
            h = torch.cat([pool(m) for m, pool in zip(x, self.pools)], dim=1)  # n*BHWC -> BC or BC11
            x = h.view(B, -1)  # flatten
            if self.drop is not None:
                x = self.drop(x)
            x = self.bn(x)
            x = self.fc(x)
            return x


    class RawSignalNet(nn.Module):
        def __init__(self, config, num_classes=N_BIRDS, resize=None, channels_first=True, pretrained=True):
            super(RawSignalNet, self).__init__()
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            self.img_dim = config.img_dim
            self.resize = v2.Resize(resize) if resize is not None else None
            self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=3)
            n_channels = self.model.num_features
            self.pool = nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
            self.drop = nn.Dropout(p=head_dropout)
            self.fc = nn.Linear(n_channels, out_features=num_classes, bias=True)

        def forward(self, batch):
            # x = (bs, 1, 32_000*img_duration)
            x = batch['x']
            bs = x.size(0)
            x = x.view(bs, 1, self.img_dim[0], -1).transpose(3,2)
            x = x.expand(-1, 3, -1, -1)         # C3HW
            bs = x.size(0)
            if self.resize:
                x = self.resize(x)
            x = self.model.forward_features(x)
            x = self.pool(x)
            x = x.view(bs, -1)  # flatten
            x = self.drop(x)
            x = self.fc(x)
            return x


    class SpecNetRaw(nn.Module):
        """ models that reads a 2D image directly from dataset """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True, resize=None, add_position=False,
                     pretrained=True):
            super().__init__()
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            if add_position:
                rows, cols = config.img_dim
                row_pos = torch.arange(rows).unsqueeze(1).expand(rows, cols)
                col_pos = torch.arange(cols).unsqueeze(0).expand(rows, cols)
                self.pos = torch.stack((row_pos, col_pos), dim=0).unsqueeze(0)
            else:
                self.pos = None
            
            self.resize = v2.Resize(resize) if resize else None
            self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=3)
            n_channels = self.model.num_features
            self.pool = nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
            self.drop = nn.Dropout(p=head_dropout)
            self.fc = nn.Linear(n_channels, out_features=num_classes, bias=True)

        def forward(self, batch):
            # do preprocess
            x = batch['x']
            B = x.size()[0]
            # x = torch.unsqueeze(x, dim=1)  # image to dataset adds channel dimension
            if self.resize:
                x = self.resize(x)
            if self.pos is not None:
                pos = self.pos.expand(B,-1,-1,-1).to(x.device)
                x = torch.concat((x, pos), dim=1)
            else:
                x = x.expand(-1, 3, -1, -1)     # C3HW
            x = self.model.forward_features(x)  # BHWC
            x = self.pool(x)                    # BHWC -> BC or BC11
            x = x.view(B, -1)  # flatten
            x = self.drop(x)
            x = self.fc(x)
            return x


    class CWT(nn.Module):
        # - wavelet_width=7, fs=200, lower_freq=0.5, upper_freq=40, n_scales=40, border_crop=1, stride=16

        def __init__(self, wavelet_width=7, fs=200, lower_freq=.5, upper_freq=40., n_scales=40,
                     size_factor=1.0, border_crop=1, stride=16):
            super().__init__()

            self.initial_wavelet_width = wavelet_width
            self.fs = fs
            self.lower_freq = lower_freq
            self.upper_freq = upper_freq
            self.size_factor = size_factor
            self.n_scales = n_scales
            self.wavelet_width = wavelet_width
            self.border_crop = border_crop
            self.stride = stride
            wavelet_bank_real, wavelet_bank_imag = self._build_wavelet_kernel()
            self.wavelet_bank_real = nn.Parameter(wavelet_bank_real, requires_grad=False)
            self.wavelet_bank_imag = nn.Parameter(wavelet_bank_imag, requires_grad=False)

            self.kernel_size = self.wavelet_bank_real.size(3)

        def _build_wavelet_kernel(self):
            s_0 = 1 / self.upper_freq
            s_n = 1 / self.lower_freq

            base = np.power(s_n / s_0, 1 / (self.n_scales - 1))
            scales = s_0 * np.power(base, np.arange(self.n_scales))

            frequencies = 1 / scales
            truncation_size = scales.max() * np.sqrt(4.5 * self.initial_wavelet_width) * self.fs
            one_side = int(self.size_factor * truncation_size)
            kernel_size = 2 * one_side + 1

            k_array = np.arange(kernel_size, dtype=np.float32) - one_side
            t_array = k_array / self.fs

            wavelet_bank_real = []
            wavelet_bank_imag = []

            for scale in scales:
                norm_constant = np.sqrt(np.pi * self.wavelet_width) * scale * self.fs / 2.0
                scaled_t = t_array / scale
                exp_term = np.exp(-(scaled_t ** 2) / self.wavelet_width)
                kernel_base = exp_term / norm_constant
                kernel_real = kernel_base * np.cos(2 * np.pi * scaled_t)
                kernel_imag = kernel_base * np.sin(2 * np.pi * scaled_t)
                wavelet_bank_real.append(kernel_real)
                wavelet_bank_imag.append(kernel_imag)

            wavelet_bank_real = np.stack(wavelet_bank_real, axis=0)
            wavelet_bank_imag = np.stack(wavelet_bank_imag, axis=0)

            wavelet_bank_real = torch.from_numpy(wavelet_bank_real).unsqueeze(1).unsqueeze(2)
            wavelet_bank_imag = torch.from_numpy(wavelet_bank_imag).unsqueeze(1).unsqueeze(2)
            return wavelet_bank_real, wavelet_bank_imag

        def forward(self, x):
            border_crop = self.border_crop // self.stride
            start = border_crop
            end = (-border_crop) if border_crop > 0 else None

            # x [n_batch, n_channels, time_len]
            out_reals = []
            out_imags = []

            n_signals = x.size(1)
            in_width = x.size(2)
            out_width = int(np.ceil(in_width / self.stride))
            pad_along_width = np.max((out_width - 1) * self.stride + self.kernel_size - in_width, 0)
            padding = pad_along_width // 2 + 1

            for i in range(n_signals):
                # [n_batch, 1, 1, time_len]
                x_ = x[:, i, :].unsqueeze(1).unsqueeze(2)
                out_real = F.conv2d(x_, self.wavelet_bank_real, stride=(1, self.stride), padding=(0, padding))
                out_imag = F.conv2d(x_, self.wavelet_bank_imag, stride=(1, self.stride), padding=(0, padding))
                out_real = out_real.transpose(2, 1)
                out_imag = out_imag.transpose(2, 1)
                out_reals.append(out_real)
                out_imags.append(out_imag)

            out_real = torch.cat(out_reals, axis=1)
            out_imag = torch.cat(out_imags, axis=1)

            out_real = out_real[:, :, :, start:end]
            out_imag = out_imag[:, :, :, start:end]

            scalograms = torch.sqrt(out_real ** 2 + out_imag ** 2)
            return scalograms


    class SpecNetCwt2D(nn.Module):
        def __init__(self, config, num_classes=N_BIRDS, resize=None, channels_first=True, pretrained=True):
            super().__init__()
            encoder = config.encoder or 'maxvit_base_tf_512.in21k_ft_in1k'
            head_dropout = config.head_dropout or 0.
            n_scales = config.img_dim[0]
            stride = math.ceil(32_000 * config.img_duration / config.img_dim[1])
            self.preprocess = CWT(
                wavelet_width=7, fs=32_000, lower_freq=50., upper_freq=16_000., n_scales=n_scales,
                size_factor=1.0, border_crop=1, stride=stride)
            self.resize = v2.Resize(resize) if resize is not None else None
            self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=3)
                                         # drop_rate=FLAGS.drop_rate, drop_path_rate=FLAGS.drop_path_rate
            n_channels = self.model.num_features
            self.pool = nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
            self.drop = nn.Dropout(p=head_dropout)
            self.fc = nn.Linear(n_channels, out_features=num_classes, bias=True)

        def forward(self, batch):
            # do preprocess
            x = batch['x']
            x = self.preprocess(x)
            n, c, f, t = x.size()   # N1HW
            if self.resize:
                x = self.resize(x)
            x = x.expand(-1, 3, -1, -1)         # C3HW
            x = self.model.forward_features(x)  # BHWC
            x = self.pool(x)                    # BHWC -> BC or BC11
            x = x.view(n, -1)  # flatten
            x = self.drop(x)
            x = self.fc(x)
            return x

    # p4 models

    class AttModelP4(nn.Module):
        """ simplified version of p4. used to load compiled state (and save it as regular state) """
        def __init__(self, backbone="resnet34", num_class=397, in_chans=1):
            super().__init__()
            base_model = timm.create_model(
                backbone, features_only=False, pretrained=False, in_chans=in_chans)

            layers = list(base_model.children())[:-2]
            self.backbone = nn.Sequential(*layers)
            if "efficientnet" in backbone:
                dense_input = base_model.num_features
            elif "swin" in backbone:
                dense_input = base_model.num_features
            elif hasattr(base_model, "fc"):
                dense_input = base_model.fc.in_features
            else:
                dense_input = base_model.feature_info[-1]["num_chs"]

            self.global_pool = GeM()
            self.dropouts = nn.ModuleList([nn.Dropout(p) for p in[.1,.2,.3,.4,.5]])
            self.head = nn.Linear(dense_input, num_class)

        def forward(self, batch):
            x = batch['x']
            x = self.backbone(x)
            x = self.global_pool(x)
            logit = sum([self.head(dropout(x)) for dropout in self.dropouts]) / 5
            return logit


    class SpecNetImgP4(nn.Module):
        """ models that reads a 2D image directly from dataset """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True,
                     pretrained=True, submit=False, **kwargs):
            super().__init__()
            self.submit = submit
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            
            if isinstance(pretrained, str):
                self.model = timm.create_model(encoder, pretrained=False, in_chans=1)
                load_encoder_state(self.model, pretrained)
                # self._load_encoder_state(pretrained)
            else:
                self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=1)
            n_channels = self.model.num_features
            self.pool = nn.AdaptiveAvgPool2d(1) if channels_first else lambda x: x.mean([1,2])
            self.drop = nn.Dropout(p=head_dropout)
            self.fc = nn.Linear(n_channels, out_features=num_classes, bias=True)

        def forward(self, x):
            B = x.size()[0]
            x = self.model.forward_features(x)  # BHWC
            x = self.pool(x)                    # BHWC -> BC or BC11
            x = x.view(B, -1)  # flatten
            x = self.drop(x)
            x = self.fc(x)
            return x


    class SpecNetImgGemP4(nn.Module):
        """ models that reads a 2D image directly from dataset """
        def __init__(self, config, num_classes=N_BIRDS, channels_first=True,
                     pretrained=True, submit=False, **kwargs):
            super().__init__()
            self.submit = submit
            encoder = config.encoder or 'efficientnet_b0'
            head_dropout = config.head_dropout or 0.
            p = config.gem_p if config.gem_p is not None else 1.8
            self.pool = GeM(p=p)  # placed before encoder to enable state load
            if isinstance(pretrained, str):
                self.model = timm.create_model(encoder, pretrained=False, in_chans=1)
                self._load_encoder_state(pretrained)
            else:
                self.model = timm.create_model(encoder, pretrained=pretrained, in_chans=1)
            self.drop = nn.ModuleList([nn.Dropout(p) for p in[.1,.2,.3,.4,.5]])
            n_channels = self.model.num_features
            self.fc = nn.Linear(n_channels, out_features=num_classes, bias=True)

        def forward(self, x):
            B = x.size()[0]
            x = self.model.forward_features(x)  # BHWC
            x = self.pool(x)                    # BHWC -> BC or BC11
            x = x.view(B, -1)  # flatten
            x = sum([self.fc(drop(x)) for drop in self.drop]) / 5
            return x
        
        def _load_encoder_state(self, pretrained):
            encoder_state = load_encoder_state(self.model, pretrained)
            if 'p' in encoder_state:
                self.pool.load_state_dict(encoder_state, strict=STRICT)
            print(f"Loaded pool.p={self.pool}")


    # ----------------------------------------------------------------------------------------------------
    # 0.4 augment
    # ----------------------------------------------------------------------------------------------------
    
    def augment_img_train():
        """ train augmentation to run on dataset """
        aug = config.aug
        pipeline = []
        if aug.Flip > 0:
            pipeline.append(A.HorizontalFlip(p=aug.Flip))
        
        if aug.CoarseDropout is not None:  # (max_height=.375, max_width=.375, max_holes=1, p=.7)
            max_height = int(config.img_dim[0] * aug.CoarseDropout[0]) # 0.375
            max_width = int(config.img_dim[1] * aug.CoarseDropout[1]) # 0.375
            pipeline.append(A.CoarseDropout(max_height=max_height, max_width=max_width,
                            max_holes=aug.CoarseDropout[2], p=aug.CoarseDropout[3]))

        tranform_fn = A.Compose(pipeline)
        return tranform_fn

    try:
        import audiomentations as AA

        def augment_audio_train():
            """ train augmentation on audio, which add background noise """
            tranform_fn = AA.Compose([
                AA.OneOf([
                    AA.Gain(min_gain_db=-15, max_gain_db=15, p=1.0),
                    AA.GainTransition(
                        min_gain_db=-24.0,
                        max_gain_db=6.0,
                        min_duration=0.2,
                        max_duration=6.0,
                        p=1.0
                    )
                ], p=0.5,),
                AA.OneOf([
                    AA.AddGaussianNoise(p=1.0),
                    AA.AddGaussianSNR(p=1.0),
                ], p=0.3,),
                AA.OneOf([
                    AA.AddShortNoises(
                        sounds_path="/kaggle/input/birdclef2021-background-noise/ff1010bird_nocall/nocall",
                        min_snr_db=0,
                        max_snr_db=3,
                        p=1.0,
                        lru_cache_size=10,
                        min_time_between_sounds=4.0,
                        max_time_between_sounds=16.0,
                    ),
                    #AA.AddShortNoises(
                    #    sounds_path="/kaggle/input/birdclef2021-background-noise/esc50/use_label",
                    #    min_snr_db=0,
                    #    max_snr_db=3,
                    #    p=1.0,
                    #    lru_cache_size=10,
                    #    min_time_between_sounds=4.0,
                    #    max_time_between_sounds=16.0,
                    #),
                ], p=0.5,),
                AA.OneOf([
                    AA.AddBackgroundNoise(
                        sounds_path="/kaggle/input/birdclef2021-background-noise/train_soundscapes/nocall",
                        min_snr_db=0,
                        max_snr_db=3,
                        p=1.0,
                        lru_cache_size=3,),
                    AA.AddBackgroundNoise(
                        sounds_path="/kaggle/input/birdclef2021-background-noise/aicrowd2020_noise_30sec/noise_30sec",
                        min_snr_db=0,
                        max_snr_db=3,
                        p=1.0,
                        lru_cache_size=20,
                        #lru_cache_size=450,
                       ),
                ], p=0.5,),
                AA.LowPassFilter(p=0.5),
            ])
            return tranform_fn
        
    except:
        def augment_audio_train():
            return lambda x: x

    BETA = Beta(1., 1.)
    
    def mixup(batch):
        """ mixup augmentation to run during training step """
        x, y = batch['x'], batch['y']
        perm = torch.randperm(x.size(0))

        # lam = torch.FloatTensor([np.random.beta(alpha, alpha)])
        lam = BETA.rsample(x.shape[:1]).to(x.device)
        lam_x = lam.view(-1, 1, 1, 1)
        lam_y = lam.view(-1, 1)
        x = x * lam_x + x[perm] * (1 - lam_x)
        y = y * lam_y + y[perm] * (1 - lam_y)

        batch['x'], batch['y'] = x, y

    # ----------------------------------------------------------------------------------------------------
    # 0.5 Datasets
    # ----------------------------------------------------------------------------------------------------

    class SnipetAmplitudeToDB(torch.nn.Module):
        """Turn a tensor from the power/amplitude scale to the decibel scale.

        .. devices:: CPU CUDA

        .. properties:: Autograd TorchScript

        This output operates per image, hence it returns the same values for an audio clip 
        split into snippets vs. a full clip.

        Args:
            stype (str, optional): scale of input tensor (``"power"`` or ``"magnitude"``). The
                power being the elementwise square of the magnitude. (Default: ``"power"``)
            top_db (float or None, optional): minimum negative cut-off in decibels.  A reasonable
                number is 80. (Default: ``None``)

        Example
            >>> waveform, sample_rate = torchaudio.load("test.wav", normalize=True)
            >>> spec = melspec_128_256_5(waveform)
            >>> transform = StableAmplitudeToDB(stype="power", top_db=80)
            >>> spec_db = transform(spec)
        """
        __constants__ = ["multiplier", "amin", "ref_value", "db_multiplier"]

        def __init__(self, stype='power', top_db=None):
            super().__init__()
            self.stype = stype
            if top_db is not None and top_db < 0:
                raise ValueError("top_db must be positive value")
            self.top_db = top_db
            self.multiplier = 10.0 if stype == "power" else 20.0
            self.amin = 1e-10
            self.ref_value = 1.0
            self.db_multiplier = math.log10(max(self.amin, self.ref_value))

        def forward(self, x):
            """Numerically stable implementation from Librosa.

            https://librosa.org/doc/latest/generated/librosa.amplitude_to_db.html

            Args:
                x (Tensor): Input tensor before being converted to decibel scale.

            Returns:
                Tensor: Output tensor in decibel scale.
            """
            x_db = self.multiplier * torch.log10(torch.clamp(x, min=self.amin))
            x_db -= self.multiplier * self.db_multiplier
            if self.top_db is not None:
                x_db = torch.max(x_db, (x_db.amax(dim=(-2, -1), keepdims=True) - self.top_db))
            return x_db


    hop_length_512_5 = 32_000 * 5 // (512 - 1)
    hop_length_256_5 = 32_000 * 5 // (256 - 1)
    hop_length_128_5 = 32_000 * 5 // (128 - 1)
    hop_length_224_10 = 32_000 * 10 // (224 - 1)

    melspec_128_512_5 = T.MelSpectrogram(
        n_fft=2048, hop_length=hop_length_512_5, f_min=50, f_max=16_000, sample_rate=32_000,
        n_mels=128, norm='slaney', mel_scale='slaney', pad_mode='constant')
    melspec_128_256_5 = T.MelSpectrogram(
        n_fft=2048, hop_length=hop_length_256_5, f_min=50, f_max=16_000, sample_rate=32_000,
        n_mels=128, norm='slaney', mel_scale='slaney', pad_mode='constant')
    melspec_224_224b_10 = T.MelSpectrogram(
        n_fft=2048, hop_length=hop_length_224_10, f_min=400, f_max=10_000, sample_rate=32_000,
        n_mels=224, norm='slaney', mel_scale='slaney', pad_mode='constant')
    melspec_p4_1 = T.MelSpectrogram(
        n_fft=2048, hop_length=512, f_min=20, f_max=16_000, sample_rate=32_000,
        n_mels=128, normalized=True)
    melspec_p4_1b = T.MelSpectrogram(
        n_fft=2048, hop_length=hop_length_256_5, f_min=20, f_max=16_000, sample_rate=32_000,
        n_mels=128, normalized=True)
    melspec_p4_4 = T.MelSpectrogram(
        n_fft=1024, hop_length=hop_length_512_5, f_min=50, f_max=14_000, sample_rate=32_000,
        n_mels=64, normalized=True)
    melspec_p4_4b = T.MelSpectrogram(
        n_fft=1024, hop_length=hop_length_512_5, f_min=50, f_max=14_000, sample_rate=32_000,
        n_mels=64, norm='slaney', mel_scale='slaney', pad_mode='constant')
    melspec_256_128_5 = T.MelSpectrogram(
        n_fft=2048, hop_length=hop_length_128_5, f_min=50, f_max=16_000, sample_rate=32_000,
        n_mels=256, norm='slaney', mel_scale='slaney', pad_mode='constant')
    melspec_256_256_5 = T.MelSpectrogram(
        n_fft=2048, hop_length=hop_length_256_5, f_min=50, f_max=16_000, sample_rate=32_000,
        n_mels=256, norm='slaney', mel_scale='slaney', pad_mode='constant')
    
    db_transform = T.AmplitudeToDB(stype='power', top_db=80)
    db_transform_snipet = SnipetAmplitudeToDB(stype='power', top_db=80)
        
    len_audio_5s = 5 * 32_000
    len_audio_10s = 10 * 32_000
    len_audio_1m = 1 * 60 * 32_000
    
    # Spectrogram types
    SPEC_128_512_5 = 1
    SPEC_224_224B_10 = 2


    SPEC_DIMS = {
        'p4_1': (128, 313),
        'p4_4': (64, 512),
        'p4_4b': (64, 512),
        'f256': (256, 128),
        'f256x2': (256, 256),
    }
        

    def transform_to_spec(dim=(128, 256), norm=False, duration=5, snipet=True, spec=None, bits=8):
        """ converts an audio array to spectrograms. Audio is a 2D array with a single segment to check 
            against train or multiple segments for submit prediction. Transformation is performed per
            audio snipet, which means that submitting an audio by itself (1, 32_000*5) or as part of a
            batch (6, 32_000*5), produces the same result for the common audio snipet.
        """
        if spec == 'p4_1':
            melspec_fn = melspec_p4_1
        elif spec == 'p4_1b':
            melspec_fn = melspec_p4_1b
        elif spec == 'p4_4':
            melspec_fn = melspec_p4_4
        elif spec == 'p4_4b':
            melspec_fn = melspec_p4_4b
        elif spec == 'f256':
            melspec_fn = melspec_256_128_5
        elif spec == 'f256x2':
            melspec_fn = melspec_256_256_5
        elif dim[-1] == 512:
            if duration == 5:
                melspec_fn = melspec_128_512_5
        elif dim[-1] == 256:
            if duration == 5:
                melspec_fn = melspec_128_256_5
        elif dim[-1] == 224:
            if duration == 10:
                melspec_fn = melspec_224_224b_10
                
        db_transform_ = db_transform_snipet if snipet else db_transform
        eps = 1e-6
        
        def _process(audio):
            spec = melspec_fn(audio)
            spec = db_transform(spec)
            min_ = torch.amin(spec)
            max_ = torch.amax(spec)
            spec = (spec - min_) / (max_ - min_)
            if bits == 8:
                spec = (spec * 255).to(torch.uint8) / 255
            return spec

        def _process_norm(audio):
            spec = melspec_fn(audio)
            spec = db_transform(spec)
            mean_, std_ = spec.mean(), spec.std()
            spec = (spec - mean_) / (std_ + eps)
            min_ = torch.amin(spec)
            max_ = torch.amax(spec)
            spec = (spec - min_) / (max_ - min_)
            if bits == 8:
                spec = (spec * 255).to(torch.uint8) / 255
            elif bits == 16:
                spec = (spec * 65535).to(torch.uint32) / 65535
            return spec

        def _process_snipet(audio):
            spec = melspec_fn(audio)
            spec = db_transform(spec)
            min_ = torch.amin(spec, dim=(-2,-1), keepdims=True)
            max_ = torch.amax(spec, dim=(-2,-1), keepdims=True)
            spec = (spec - min_) / (max_ - min_)
            if bits == 8:
                spec = (spec * 255).to(torch.uint8) / 255
            elif bits == 16:
                spec = (spec * 65535).to(torch.uint32) / 65535
            return spec

        def _process_norm_snipet(audio):
            spec = melspec_fn(audio)
            spec = db_transform(spec)
            mean_, std_ = spec.mean((-2,-1), keepdims=True), spec.std((-2,-1), keepdims=True)
            spec = (spec - mean_) / (std_ + eps)
            min_ = torch.amin(spec, dim=(-2,-1), keepdims=True)
            max_ = torch.amax(spec, dim=(-2,-1), keepdims=True)
            spec = (spec - min_) / (max_ - min_)
            if bits == 8:
                spec = (spec * 255).to(torch.uint8) / 255
            return spec

        if snipet:
            return _process_norm_snipet if norm else _process_snipet
        else:
            return _process_norm if norm else _process


    def prepare_data(files, num_workers, verbose=1, norm=False, dim=(128,256), duration=5,
                     snipet=False, spec=None, bits=8):
        """ generate specs for submission in parallel """
        transform_to_spec_ = transform_to_spec(
            dim=dim, norm=norm, duration=duration, snipet=snipet, spec=spec, bits=bits)
        len_duration = duration * 32_000
        
        def _process_chunk(chunk):
            out = []
            for file in tqdm(chunk):
                audio, sr = torchaudio.load(file)
                audio = audio[:, :len_audio_1m].view(-1, len_duration)
                out.append(transform_to_spec_(audio))
            out = torch.cat(out, axis=0)
            out = out.unsqueeze(1)
            return out

        t1 = time.time()
        chunk_size = math.ceil(len(files) / num_workers)
        chunks = [files[i : i+chunk_size] for i in range(0, len(files), chunk_size)]

        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            out = list(executor.map(_process_chunk, chunks))
        out = torch.cat(out, axis=0)
        duration = time.time() - t1
        if verbose:
            print(f'specs: {out.shape}')
            print(f'time: {duration:,.0f} s for {len(files):,.0f} files  -  all: {duration * 700 / len(files) / 60:,.0f} m')
        return out


    def prepare_data_ragged(files, num_workers, verbose=1, norm=False, dim=(128,256),
                            duration=5, max_duration=32_000*4*60, snipet=False, spec=None, bits=8):
        """ generate specs for prediction in parallel. Audio is limited to 4m and broken into 5s batches.
            The last batch is padded with 0s.
            Returns specs and size (number of specs per file, i.e., 5s slots). 
        """
        transform_to_spec_ = transform_to_spec(
            dim=dim, norm=norm, duration=duration, snipet=snipet, spec=spec, bits=bits)
        
        def _process_chunk(chunk):
            out, size = [], []
            range_ = tqdm(chunk) if verbose >= 2 else chunk
            for file in range_:
                audio, sr = torchaudio.load(file)
                if max_duration is not None:
                    audio = audio[:, :max_duration]
                rest = audio.shape[-1] % len_audio_5s
                if rest > 0:
                    audio = F.pad(audio, pad=[0, len_audio_5s - rest], mode='constant', value=0)
                audio = audio[:, :len_audio_1m].view(-1, len_audio_5s)
                out.append(transform_to_spec_(audio))
                size.append(audio.shape[0])
            out = torch.cat(out, axis=0)
            out = out.unsqueeze(1)
            return out, size

        t1 = time.time()
        chunk_size = math.ceil(len(files) / num_workers)
        chunks = [files[i : i+chunk_size] for i in range(0, len(files), chunk_size)]

        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            out_size = list(executor.map(_process_chunk, chunks))
        out, size = zip(*out_size)
        out = torch.cat(out, axis=0)
        size = np.concatenate(size, axis=0)
        duration = time.time() - t1
        if verbose:
            print(f'specs: {out.shape}')
            print(f'time: {duration:,.0f} s for {len(files):,.0f} files  -  all: {duration * 700 / len(files) / 60:,.0f} m')
        return out, size


    class Ba_data_module_montage_raw(L.LightningDataModule):
        def __init__(self, files, y, birds, train_idx, val_idx, dataset_val, fold, train_mode='random', ws=None, files_unlabeled=None):
            super().__init__()
            self.files = files
            self.y = y
            self.birds = birds
            self.ws = ws
            self.train_idx = train_idx
            self.val_idx = val_idx
            self.dataset_val = dataset_val
            self.fold = fold
            self.train_mode = train_mode
            self.files_unlabeled = files_unlabeled
            self.prefetch_factor = config.prefetch_factor
      
        def prepare_data(self):
            """ processing to be done only in one GPU """
            train_idx = self.train_idx
            val_idx = self.val_idx
            self.train_dataset = Ba_dataset_montage_raw(
                self.files[train_idx], self.y[train_idx], self.birds[train_idx], 
                shuffle=True, mode=self.train_mode, cache=False, augment=True,
                montage_cache_size=config.montage_cache_size,
                files_unlabeled=self.files_unlabeled, ws=self.ws)
            folder = f'{ROOT}/b5-data-{self.dataset_val}'
            if self.val_idx is not None:
                self.val_dataset = Ba_dataset_val(
                    f'{folder}/x_f{self.fold}', f'{folder}/y_f{self.fold}')
                size = self.val_idx.sum()
                if size != self.val_dataset.x.shape[0]:
                    self.val_dataset.x = self.val_dataset.x[:size]
                    self.val_dataset.y = self.val_dataset.y[:size]
      
        def setup(self, stage=None):
            """ processing to be done in all GPUs """
            pass
      
        def train_dataloader(self):
            return DataLoader(
                self.train_dataset, batch_size=config.train_batch_size, shuffle=False, 
                num_workers=config.num_workers, prefetch_factor=self.prefetch_factor,
                collate_fn=None, drop_last=True)

        def val_dataloader(self):
            if self.val_idx is None:
                return None
            return DataLoader(
                self.val_dataset, batch_size=config.val_batch_size, shuffle=False, 
                num_workers=config.num_workers, prefetch_factor=self.prefetch_factor,
                collate_fn=None, drop_last=False)
      
        def test_dataloader(self):
            return None


    class Cache_audio_files:
        """ keeps a cache of a random `duration` segment of unlabeled audio files. 
            Refreshes segment every `max_usage` times it is used. 
        """
        
        def __init__(self, files, duration=5, max_usage=None, size=1000):
            self.files = files
            random.shuffle(self.files)
            self.n_loaded = 0
            self.duration = duration * 32_000
            self.max_usage = max_usage or 6
            self.size = size or 1000
            self.cache = dict()
            self.count_used = np.zeros(size)
            
        def get_random(self):
            idx = random.randint(0, self.size - 1)
            cache = self.cache
            if not idx in cache or self.count_used[idx] > self.max_usage:
                file = self.files[self.n_loaded]
                self.n_loaded += 1
                if self.n_loaded == len(files):
                    random.shuffle(self.files)
                    self.n_loaded = 0
                audio, sr = torchaudio.load(file)
                gap = self.duration - audio.shape[-1]
                if gap > 0:
                    pad_before = random.randint(0, gap - 1)
                    pad_after = gap - pad_before
                    x = F.pad(x, pad=(pad_before, pad_after), mode='constant', value=0)
                elif gap < 0:
                    start = random.randint(0, -gap - 1)
                    audio = audio[:, start : start + self.duration]

                self.cache[idx] = audio
                self.count_used[idx] = 1
                return audio
            else:
                self.count_used[idx] += 1
                audio = self.cache[idx]
                return audio


    class Ba_dataset_montage_raw(Dataset):
        """ montage of raw audio files and unlabeled (if `files_unlabeled` is not None). 
            Montage includes additional audios if only one bird in initial audio. 
            If files_unlabeled is defined, the montage includes a random unlabeled audio with probability
            `config.unlabeled_prob`. If an unlabeled audio is included, the mask includes only the birds in
            the labeled audios. Otherwise, it includes all labels.
            Used only for train.
        """
        def __init__(self, files, y, birds, shuffle=True, ws=None, mode='random', augment=None,
                     cache=False, montage_cache_size=10, files_unlabeled=None, **kwargs):
            super().__init__()
            self.files = files
            self.y = torch.tensor(y, dtype=torch.float32)
            self.birds = birds
            self.shuffle = shuffle
            self.ws = ws
            self.mode = mode
            self.augment = augment_img_train() if augment else None
            self.augment_audio = augment_audio_train() if (augment and config.aug is not None and config.aug.audio) else None
            if config.unlabeled_prob and files_unlabeled is not None:
                self.unlabeled_cache = Cache_audio_files(
                    files_unlabeled, config.img_duration, config.unlabeled_max_usage, config.unlabeled_cache_size)
                self.unlabeled_prob = config.unlabeled_prob
            else:
                self.unlabeled_prob = 0
            self.aug_volume = (math.log(config.aug.volume[0]), math.log(config.aug.volume[1])) \
                if config.aug.volume is not None else (math.log(.5), math.log(2.))
            self.unlabeled_reference = config.unlabeled_reference
            self.norm_audio = config.norm_audio or False

            self.label_smoothing = config.label_smoothing
            self.montage = config.montage            
            self.apply_montage = y.sum(-1) == 1
            self.montage_cache_size = montage_cache_size
            self.montage_cache = {bird: [] for bird in RANGE_BIRDS}
            self.snipet = config.snipet if config.snipet is not None else False
            self.width = 32_000 * config.img_duration
            self.idxs = np.arange(len(files))
            bits = config.bits or 8
            self.transform_to_spec_ = transform_to_spec(
                dim=config.img_dim, norm=config.spec_norm, duration=config.img_duration,
                snipet=self.snipet, spec=config.spec, bits=bits)
            self.on_epoch_end()
            self.all_true = torch.ones(len(BIRDS), dtype=torch.float32)
            
            
        def __len__(self):
            return len(self.files)


        def __getitem__(self, index):
            if self.ws is not None:
                idx = np.random.choice(self.idxs, size=1, p=self.ws)[0]
            else:
                idx = self.idxs[index]
            
            x = self._get_audio(idx)
            if self.y is not None:
                y = self.y[idx]
            else:
                y = ws = None

            x, y = self._get_montage(x, y, idx)
            if random.random() < self.unlabeled_prob:
                unlabeled_audio = self.unlabeled_cache.get_random()
                if self.unlabeled_reference:
                    max_x = x.abs().max()
                    max_unlabeled = unlabeled_audio.abs().max()
                    x = x / max_x * max_unlabeled + unlabeled_audio
                else:
                    x = x + unlabeled_audio * math.exp(random.uniform(*self.aug_volume))
                mask = (y > 0).to(torch.float32)
            else:
                mask = self.all_true
                
            y = torch.clamp(y, 0., 1.)
            if self.label_smoothing:
                tp = y.sum(-1)
                y = y * (1 - self.label_smoothing) + self.label_smoothing * tp / y.shape[-1]

            if self.augment_audio is not None:
                dtype = x.dtype
                x = self.augment_audio(x[0].numpy(), sample_rate=32_000)
                x = torch.tensor(x, dtype=dtype).unsqueeze(0)

            if self.norm_audio:
                x /= torch.abs(x).max()

            x = self.transform_to_spec_(x)

            if self.augment:
                dtype = x.dtype
                x = self.augment(image=x.permute(1,2,0).numpy())['image'].astype(np.float32)
                x = torch.tensor(x, dtype=dtype).permute(2,0,1)

            return dict(idx=idx, x=x, y=y, mask=mask)


        def on_epoch_end(self):
            if self.shuffle and self.ws is not None:
                n = len(self.files)
                self.idxs = np.random.choice(np.arange(n), size=n, replace=True, p=self.ws)


        def _get_audio(self, idx):
            file = self.files[idx]
            if file[-1] == 'e' and random.random() < .5:
                file = file[:-1] + 's'
            audio, sr = torchaudio.load(file)
            width = self.width
            width_x = audio.shape[-1]
            if width_x > width:
                offset = random.randint(0, width_x - width)
                offset_montage = random.randint(0, width_x - width)
            else:
                offset = offset_montage = 0

            self._add_montage_cache(audio, offset_montage, idx)
            x = audio[..., offset : offset + width]
            gap = width - x.shape[-1]
            if gap > 0:
                pad_before = random.randint(0, gap - 1)
                pad_after = gap - pad_before
                x = F.pad(x, pad=(pad_before, pad_after), mode='constant', value=0)
            return x

        def _get_montage(self, x, y, idx):
            """ build montage. Applies a random volume change to all signals, including the original one """
            aug_volume = self.aug_volume
            x *= math.exp(random.uniform(*aug_volume))
            if self.apply_montage[idx]:
                bird = self.birds[idx]
                if self.mode == 'random':
                    n = random.randint(*self.montage)
                    sample = random.sample(RANGE_BIRDS, n)
                    sample = [x for x in sample if x != bird]
                else:
                    sample = self.montage_samples[idx]
                # print('n', n, bird, sample)
                if len(sample) > 0:
                    x = x.clone()
                    y = y.clone()
                    for i in sample:
                        mx, my = self._get_montage_cache(i)
                        if mx is not None:
                            x += mx * math.exp(random.uniform(*aug_volume))
                            y += my
            return x, y

        def _add_montage_cache(self, audio, offset, idx):
            """ add bird instance to cache """
            if not self.apply_montage[idx]:
                return
            x = audio[..., offset : offset + self.width]
            gap = self.width - audio.shape[-1]
            if gap > 0:
                pad_before = random.randint(0, gap - 1)
                pad_after = gap - pad_before
                x = F.pad(x, pad=(pad_before, pad_after), mode='constant', value=0)
            
            bird = self.birds[idx]
            data = (x, self.y[idx])
            if len(self.montage_cache[bird]) <= self.montage_cache_size:
                self.montage_cache[bird].append(data)
            else:
                idx_cache = random.randint(0, self.montage_cache_size - 1)
                self.montage_cache[bird][idx_cache] = data

        def _get_montage_cache(self, bird):
            """ get random instance of bird from those cached """
            options = self.montage_cache[bird]
            if len(options) == 0:
                return None, None
            else:
                idx_cache = random.randint(0, len(options) - 1)
                return self.montage_cache[bird][idx_cache]


    class Ba_dataset_val(Dataset):
        """ Uses prebuilt specs and labels
        """
        def __init__(self, x_file, y_file, **kwargs):
            super().__init__()
            self.x_file = x_file
            self.y_file = y_file
            self.x = load_obj(x_file)
            self.y = load_obj(y_file)
            #self.ws = torch.ones(x.shape[0], dtype=torch.float32) / x.shape[0]
            self._tensorize_and_pin()

        def __len__(self):
            return len(self.y)

        def __getitem__(self, index):
            x = self.x[index]
            y = self.y[index]
            return dict(idx=index, x=x, y=y)

        def _tensorize_and_pin(self):
            if torch.cuda.is_available():
                self.x = torch.tensor(self.x).pin_memory()
                self.y = torch.tensor(self.y).pin_memory()
                # self.ws = torch.tensor(self.ws).pin_memory()
            else:
                self.x = torch.tensor(self.x)
                self.y = torch.tensor(self.y)


    class Ba_data_module_montage_h5(L.LightningDataModule):
        def __init__(self, files, y, birds, train_idx, val_idx, dataset_val, fold, train_mode='random',
                     ws=None, files_unlabeled=None, custom_filter=None):
            super().__init__()
            self.files = files
            self.y = y
            self.birds = birds
            self.ws = ws
            self.train_idx = train_idx
            self.val_idx = val_idx
            self.dataset_val = dataset_val
            self.fold = fold
            self.train_mode = train_mode
            self.files_unlabeled = files_unlabeled
            self.custom_filter = custom_filter
            self.prefetch_factor = config.prefetch_factor
      
        def prepare_data(self):
            """ processing to be done only in one GPU """
            train_idx = self.train_idx
            val_idx = self.val_idx
            self.train_dataset = Ba_dataset_montage_h5(
                self.files[train_idx], self.y[train_idx], self.birds[train_idx], 
                shuffle=True, mode=self.train_mode, cache=False, augment=True,
                montage_cache_size=config.montage_cache_size, files_unlabeled=self.files_unlabeled,
                ws=self.ws, custom_filter=self.custom_filter)
            folder = f'{ROOT}/b5-data-{self.dataset_val}'
            if self.val_idx is not None:
                self.val_dataset = Ba_dataset_val(
                    f'{folder}/x_f{self.fold}', f'{folder}/y_f{self.fold}')
                size = self.val_idx.sum()
                if size != self.val_dataset.x.shape[0]:
                    self.val_dataset.x = self.val_dataset.x[:size]
                    self.val_dataset.y = self.val_dataset.y[:size]
      
        def setup(self, stage=None):
            """ processing to be done in all GPUs """
            pass
      
        def train_dataloader(self):
            return DataLoader(
                self.train_dataset, batch_size=config.train_batch_size, shuffle=False, 
                num_workers=config.num_workers, prefetch_factor=self.prefetch_factor,
                collate_fn=None, drop_last=True)

        def val_dataloader(self):
            if self.val_idx is None:
                return None
            return DataLoader(
                self.val_dataset, batch_size=config.val_batch_size, shuffle=False, 
                num_workers=config.num_workers, prefetch_factor=self.prefetch_factor,
                collate_fn=None, drop_last=False)
      
        def test_dataloader(self):
            return None


    def pick_audio_segment(custom_filter, mode, width):
        """ pick a segment of duration `width` based on the sections configured in `custom_filter`
            using the following rules based on `mode`:
            - '2x7s': either first of last segment with a random offset <= 2
            - '7s': first segment with a random offset <=2
            - 'random': random segment
        """
        sections = custom_filter.sections
        if len(sections) == 1:
            section = sections[0]
            section_start, section_end = section
            if mode == '2x7s':
                if random.random() < .5:  # first 7s
                    segment_start = section_start
                    segment_end = min(section_start + width + 2, section_end)
                else:   # last 7s
                    segment_start = max(section_start, section_end - width - 2)
                    segment_end = section_end
            elif mode == '7s':
                    segment_start = section_start
                    segment_end = min(section_start + width + 2, section_end)
            elif mode == 'random':
                segment_start = section_start
                segment_end = section_end
        else:
            if mode == '2x7s':
                if random.random() < .5:  # first 7s
                    section = sections[0]
                    section_start, section_end = section
                    segment_start = section_start
                    segment_end = min(section_start + width + 2, section_end)
                else:   # last 7s
                    section = sections[-1]
                    section_start, section_end = section
                    segment_start = max(section_start, section_end - width - 2)
                    segment_end = section_end
            elif mode == '7s':
                section = sections[0]
                section_start, section_end = section
                segment_start = section_start
                segment_end = min(section_start + width + 2, section_end)
            elif mode == 'random':
                sections_prob = custom_filter.sections_prob
                idx_section = random.choices(range(len(sections)), weights=sections_prob, k=1)[0]
                section = sections[idx_section]
                section_start, section_end = section
                segment_start = section_start
                segment_end = section_end

        return segment_start, segment_end


    def test_pick_audio_segment():
        custom_filter = dotdict(sections=[(5, 8), (10, 20), (30, 60)], sections_prob=[3, 10, 30])
        pick_audio_segment(custom_filter, '2x7s', 5)


    class Ba_dataset_montage_h5(Dataset):
        """ montage of raw audio files and unlabeled (if `files_unlabeled` is not None). 
            Montage includes additional audios if only one bird in initial audio. 
            If files_unlabeled is defined, the montage includes a random unlabeled audio with probability
            `config.unlabeled_prob`. If an unlabeled audio is included, the mask includes only the birds in
            the labeled audios. Otherwise, it includes all labels.
            Audio files are h5 and we use a custom filter to pick the audio sections.
            
            Parameters:
                - mode:
                    - '2x7s' pick a random segment from first or last 7s.
                    - '7s' pick a random segment from first 7s.
                    - 'random' pick a segment at random.
                - false_prob: int:
                    - >0: include segments with no vocalization with probability `false_prob`
        """
        def __init__(self, files, y, birds, shuffle=True, ws=None, mode='2x7s', augment=None,
                     cache=False, montage_cache_size=10, files_unlabeled=None, custom_filter=None,
                     **kwargs):
            super().__init__()
            self.files = files
            self.y = torch.tensor(y, dtype=torch.float32)
            self.birds = birds
            self.shuffle = shuffle
            self.ws = ws
            self.mode = mode
            self.custom_filter = custom_filter
            self.false_prob = config.false_prob or 0
            
            self.augment = augment_img_train() if augment else None
            self.augment_audio = augment_audio_train() if (augment and config.aug is not None and config.aug.audio) else None
            if config.unlabeled_prob and files_unlabeled is not None:
                self.unlabeled_cache = Cache_audio_files(
                    files_unlabeled, config.img_duration, config.unlabeled_max_usage, config.unlabeled_cache_size)
                self.unlabeled_prob = config.unlabeled_prob
            else:
                self.unlabeled_prob = 0
            self.aug_volume = (math.log(config.aug.volume[0]), math.log(config.aug.volume[1])) \
                if config.aug.volume is not None else (math.log(.5), math.log(2.))
            self.unlabeled_reference = config.unlabeled_reference
            self.norm_audio = config.norm_audio or False

            self.label_smoothing = config.label_smoothing
            self.montage = config.montage if config.montage is not None else (0, 0)
            self.apply_montage = y.sum(-1) == 1
            self.montage_cache_size = montage_cache_size
            self.montage_cache = {bird: [] for bird in RANGE_BIRDS}
            self.snipet = config.snipet if config.snipet is not None else False
            self.width = config.img_duration
            self.idxs = np.arange(len(files))
            bits = config.bits or 8
            self.transform_to_spec_ = transform_to_spec(
                dim=config.img_dim, norm=config.spec_norm, duration=config.img_duration,
                snipet=self.snipet, spec=config.spec, bits=bits)
            self.on_epoch_end()
            self.all_true = torch.ones(len(BIRDS), dtype=torch.float32)
            
            
        def __len__(self):
            return len(self.files)


        def __getitem__(self, index):
            # pick primary audio
            idx = self.idxs[index]  # idxs is already randomized before each epoch
            # if self.ws is not None:
            #     idx = np.random.choice(self.idxs, size=1, p=self.ws)[0]
            # else:
            #     idx = self.idxs[index]
            
            x = self._get_audio(idx)
            if self.y is not None:
                y = self.y[idx]
            else:
                y = ws = None

            x, y = self._get_montage(x, y, idx)
            if random.random() < self.unlabeled_prob:
                unlabeled_audio = self.unlabeled_cache.get_random()
                if self.unlabeled_reference:
                    max_x = x.abs().max()
                    max_unlabeled = unlabeled_audio.abs().max()
                    x = x / max_x * max_unlabeled + unlabeled_audio
                else:
                    x = x + unlabeled_audio * math.exp(random.uniform(*self.aug_volume))
                mask = (y > 0).to(torch.float32)
            else:
                mask = self.all_true
                
            y = torch.clamp(y, 0., 1.)
            if self.label_smoothing:
                tp = y.sum(-1)
                y = y * (1 - self.label_smoothing) + self.label_smoothing * tp / y.shape[-1]

            if self.augment_audio is not None:
                dtype = x.dtype
                x = self.augment_audio(x[0].numpy(), sample_rate=32_000)
                x = torch.tensor(x, dtype=dtype).unsqueeze(0)

            if self.norm_audio:
                x /= torch.abs(x).max()

            x = self.transform_to_spec_(x)

            if self.augment:
                dtype = x.dtype
                x = self.augment(image=x.permute(1,2,0).numpy())['image'].astype(np.float32)
                x = torch.tensor(x, dtype=dtype).permute(2,0,1)

            return dict(idx=idx, x=x, y=y, mask=mask)


        def on_epoch_end(self):
            if self.shuffle and self.ws is not None:
                n = len(self.files)
                self.idxs = np.random.choice(np.arange(n), size=n, replace=True, p=self.ws)


        def _get_audio(self, idx):
            file = self.files[idx]
            width = self.width

            # pick segment start and read it from file
            segment_start, segment_end = pick_audio_segment(self.custom_filter[idx], self.mode, width)
            max_offset = max(segment_end - segment_start - width, 0)
            offset = random.uniform(0, max_offset)
            # pick montage offset to be within 2s of montage to  minimize size of audio file to load
            if self.montage[1] > 0:
                montage_min_offset = max(0, offset - 2)
                montage_max_offset = min(offset + 2, max_offset)
                offset_montage = random.uniform(montage_min_offset, montage_max_offset)
            else:
                offset_montage = offset
            
            min_offset = min(offset, offset_montage)
            max_offset = max(offset, offset_montage)
            audio_start = int((segment_start + min_offset) * SR)
            audio_end = int((segment_start + width + max_offset) * SR)
            with h5py.File(file, 'r') as f:
                audio = f['raw'][audio_start : audio_end]
            audio = torch.as_tensor(audio)
            
            # add montage
            start_montage = int((offset_montage - min_offset) * SR)
            if audio.shape[-1] < 5 * SR and start_montage > 0:
                print('######', idx, file, self.custom_filter[idx])
                print(segment_start, segment_end, audio_start/SR, audio_end/SR, audio.shape, audio.shape[-1]/SR)
                print(audio_start, audio_end, offset, offset_montage, start_montage/SR)
                assert 1 == 0

            self._add_montage_cache(audio.unsqueeze(0), start_montage, idx)
            start = int((offset - min_offset) * SR)
            width_sr = width * SR
            x = audio[start : start + width_sr]
            gap = width_sr - x.shape[-1]
            if gap > 0:
                pad_before = random.randint(0, gap - 1)
                pad_after = gap - pad_before
                x = F.pad(x, pad=(pad_before, pad_after), mode='constant', value=0)
            return x.unsqueeze(0)


        def _get_montage(self, x, y, idx):
            """ build montage. Applies a random volume change to all signals, including the original one """
            aug_volume = self.aug_volume
            x *= math.exp(random.uniform(*aug_volume))
            if self.apply_montage[idx]:
                bird = self.birds[idx]
                n = random.randint(*self.montage)
                sample = random.sample(RANGE_BIRDS, n)
                sample = [x for x in sample if x != bird]
                # print('n', n, bird, sample)
                if len(sample) > 0:
                    x = x.clone()
                    y = y.clone()
                    for i in sample:
                        mx, my = self._get_montage_cache(i)
                        if mx is not None:
                            x += mx * math.exp(random.uniform(*aug_volume))
                            y += my
            return x, y


        def _add_montage_cache(self, audio, offset, idx):
            """ add bird instance to cache """
            if not self.apply_montage[idx]:
                return
            width_sr = self.width * SR
            x = audio[..., offset : offset + width_sr]
            gap = width_sr - x.shape[-1]
            if gap > 0:
                pad_before = random.randint(0, gap - 1)
                pad_after = gap - pad_before
                x = F.pad(x, pad=(pad_before, pad_after), mode='constant', value=0)
            
            bird = self.birds[idx]
            data = (x, self.y[idx])
            if len(self.montage_cache[bird]) <= self.montage_cache_size:
                self.montage_cache[bird].append(data)
            else:
                idx_cache = random.randint(0, self.montage_cache_size - 1)
                self.montage_cache[bird][idx_cache] = data


        def _get_montage_cache(self, bird):
            """ get random instance of bird from those cached """
            options = self.montage_cache[bird]
            if len(options) == 0:
                return None, None
            else:
                idx_cache = random.randint(0, len(options) - 1)
                return self.montage_cache[bird][idx_cache]


    class Ba_data_module_montage_raw_l2(L.LightningDataModule):
        def __init__(self, files, meta, y, labeled_birds, train_idx, val_idx,
                     size_train=None, size_val=None, ws=None):
            super().__init__()
            self.files = file if isinstance(files, np.ndarray) else np.array(files)
            self.y = y
            self.labeled_birds = labeled_birds
            self.train_idx = train_idx
            self.val_idx = val_idx
            self.size_train = size_train
            self.size_val = size_val
            self.ws = ws
            self.prefetch_factor = config.prefetch_factor
            self.min_count = min_count
            self.min_duration = min_duration
            self.min_total_duration = min_total_duration
      
        def prepare_data(self):
            """ processing to be done only in one GPU """
            train_idx, val_idx = self.train_idx, self.val_idx
            # train
            meta_train = meta[self.train_idx]
            bird_catalog = build_bird_catalog(meta_train, self.min_duration)
            ws = 1 / np.sqrt([len(bird_catalog[idx]) for idx in self.labeled_birds])
            ws = ws / ws.sum()
            self.train_dataset = Ba_dataset_montage_raw_l2(
                self.files[train_idx], bird_catalog, self.y[val_idx], labeled_birds,
                size=self.size_train, shuffle=True, augment=True, ws=self.ws)
            # val
            meta_val = meta[self.val_idx]
            bird_catalog = build_bird_catalog(meta_val, self.min_duration)
            ws = 1 / np.sqrt([len(bird_catalog[idx]) for idx in labeled_birds])
            ws = ws / ws.sum()
            self.val_dataset = Ba_dataset_montage_raw_l2(
                self.files, bird_catalog, self.y, labeled_birds, 
                size=self.size_val, shuffle=True, augment=True, ws=self.ws)

  
        def setup(self, stage=None):
            """ processing to be done in all GPUs """
            pass
      
        def train_dataloader(self):
            return DataLoader(
                self.train_dataset, batch_size=config.train_batch_size, shuffle=False, 
                num_workers=config.num_workers, prefetch_factor=self.prefetch_factor,
                collate_fn=None, drop_last=True)

        def val_dataloader(self):
            if self.val_idx is None:
                return None
            return DataLoader(
                self.val_dataset, batch_size=config.val_batch_size, shuffle=False, 
                num_workers=config.num_workers, prefetch_factor=self.prefetch_factor,
                collate_fn=None, drop_last=False)
      
        def test_dataloader(self):
            return None


    class Ba_dataset_montage_raw_l2(Dataset):
        """ montage of raw audio files. Montage includes additional audios. No unlabeled is added to avoid 
            introducing noise in labels.
            bird_catalog: dict with list of audios files, size and label(s) per bird
            y: array of labels per file in meta using index in meta (meta.i). Dimension is len(labeled_birds)
        """
        def __init__(self, files, bird_catalog, y, labeled_birds, size=None,
                    shuffle=True, ws=None, augment=None, **kwargs):
            super().__init__()
            self.files = files
            self.bird_catalog = bird_catalog
            self.y = y
            self.montage_birds = np.array(bird_catalog.keys())
            self.labeled_birds = labeled_birds
            self.size = size or sum(len(x) for x in bird_catalog.values()) # total number of audios
            self.shuffle = shuffle
            self.ws = ws
            self.mode = mode
            self.augment = augment_img_train() if augment else None
            self.cache_data = dict()
            self.augment_audio = augment_audio_train() if (augment and config.aug is not None and config.aug.audio) else None
            self.aug_volume = (math.log(config.aug.volume[0]), math.log(config.aug.volume[1])) \
                if config.aug.volume is not None else (math.log(.5), math.log(2.))
            self.norm_audio = config.norm_audio or False
            self.label_smoothing = config.label_smoothing
            self.montage = config.montage            
            self.snipet = config.snipet if config.snipet is not None else False
            self.width = 32_000 * config.img_duration
            bits = config.bits or 8
            self.transform_to_spec_ = transform_to_spec(
                dim=config.img_dim, norm=config.spec_norm, duration=config.img_duration, snipet=self.snipet,
                spec=config.spec, bits=bits)
            self.cache_audio = dict()
            self.all_true = torch.ones(len(BIRDS), dtype=torch.float32)
            
            
        def __len__(self):
            return self.size


        def __getitem__(self, index):
            if not self.shuffle and index in self.cache_data:
                return self.cache_data[index]  # for performance and repeatability of val sample
                
            if self.ws is not None:
                idx = np.random.choice(self.labeled_birds, size=1, p=self.ws)[0]
            else:
                idx = self.labeled_birds[index % len(self.labeled_birds)]
            
            x, y = self._get_segment_1m(idx)
            x, y = self._get_montage(x, y, idx)
            mask = self.all_true
                
            y = torch.clamp(y, 0., 1.)
            if self.label_smoothing:
                tp = y.sum(-1)
                y = y * (1 - self.label_smoothing) + self.label_smoothing * tp / y.shape[-1]

            if self.augment_audio is not None:
                dtype = x.dtype
                x = self.augment_audio(x[0].numpy(), sample_rate=32_000)
                x = torch.tensor(x, dtype=dtype).unsqueeze(0)

            x = x.view(-1, len_audio_5s)
            x = self.transform_to_spec_(x)

            if self.augment:
                dtype = x.dtype
                x = self.augment(image=x.permute(1,2,0).numpy())['image'].astype(np.float32)
                x = torch.tensor(x, dtype=dtype).permute(2,0,1)

            out = dict(idx=idx, x=x, y=y, mask=mask)
            if not self.shuffle:
                self.cache_data[index] = out
                
            return out


        def _get_segment_1m(self, idx):
            """ buids an audio segment of 4m for bird `idx` by concatenating audios at random.
                Returns audio and sum of labels for those segments (may include secondary birds).
            """
            total_size = 0
            files_idx = []
            y = torch.zeros(len(self.labeled_birds))
            done = False
            options = self.bird_catalog[idx]
            options_idxs = np.arange(len(options))
            while not done:
                perm = np.random.permutation(options_idxs)
                for i in perm:
                    file, size, file_idx = options[i]
                    total_size += size
                    files_idx.append(file_idx)
                    y += self.y[file_idx]
                    if total_size >= len_audio_1m:
                        done = True
                        break
            
            audio = [self.get_audio(file_idx) for file_idx in files_idx]
            audio = torch.cat(audio, axis=1)[:, :len_audio_1m]
            y = torch.clamp(y, 0., 1.)
            return audio, y


        def _get_montage(self, x, y, idx):
            """ build montage. Applies a random volume change to all signals, including the original one """
            aug_volume = self.aug_volume
            x *= math.exp(random.uniform(*aug_volume))
            bird = idx
            n = random.randint(*self.montage)
            # sample = random.sample(self.montage_birds, n)
            sample = np.random.choice(self.montage_birds, n, replace=False)
            sample = [x for x in sample if x != bird]
            # print('n', n, bird, sample)
            if len(sample) > 0:
                x = x.clone()
                y = y.clone()
                for i in sample:
                    mx, my = self._get_segment_1m(i)
                    if mx is not None:
                        x += mx * math.exp(random.uniform(*aug_volume))
                        y += my
            return x, y


        def get_audio(self, idx):
            """ reads audio faile and normalizes by the maximum amplitude to minimize volume changes when
                concatenating audios together (considering the in most cases the higest amplitud appears to
                corresponds to the singing.
            """
            if idx not in self.cache_audio:
                audio, sr = torchaudio.load(self.files[idx])
                audio = audio[:1]  # in case we use stereo audios (previous contains some)
                audio /= torch.abs(audio).max()
                self.cache_audio[idx] = audio
            return self.cache_audio[idx]

        def add_montage_cache(self, audio, offset, idx):
            """ add bird instance to cache """
            if not self.apply_montage[idx]:
                return
            x = audio[..., offset : offset + self.width]
            gap = self.width - audio.shape[-1]
            if gap > 0:
                pad_before = random.randint(0, gap - 1)
                pad_after = gap - pad_before
                x = F.pad(x, pad=(pad_before, pad_after), mode='constant', value=0)
            
            bird = self.birds[idx]
            data = (x, self.y[idx])
            if len(self.montage_cache[bird]) <= self.montage_cache_size:
                self.montage_cache[bird].append(data)
            else:
                idx_cache = random.randint(0, self.montage_cache_size - 1)
                self.montage_cache[bird][idx_cache] = data

        def get_montage_cache(self, bird):
            """ get random instance of bird from those cached """
            options = self.montage_cache[bird]
            if len(options) == 0:
                return None, None
            else:
                idx_cache = random.randint(0, len(options) - 1)
                return self.montage_cache[bird][idx_cache]


    # ----------------------------------------------------------------------------------------------------
    # 0.6 Loss & metrics
    # ----------------------------------------------------------------------------------------------------

    def sigmoid(x):
        return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))


    def rank_based_accuracy(preds, labels):
        """ score that indicates the average ranking of the correct correct label:
            1 indicates the true value is always the top prediction, whereas 0
            indicates it's the last. labels must be an array of single labels.
        """
        B = preds.size(0)
        probabilities, sorted_indices = preds.sort(dim=1, descending=True)
        correct_indices = labels.view(-1, 1).expand_as(sorted_indices)
        ranks = (sorted_indices == correct_indices).nonzero(as_tuple=True)[1]
        scores = 1.0 / (ranks.float() + 1.0)
        return scores.sum().item() / B
        
    RANK = torch.range(1, N_BIRDS).reshape(1, -1).expand(32, -1)
    
    def rank_based_accuracy_multi(preds, labels):
        """ computes a score that mimics the average ranking of the true labels.
            Supports multiple true labels. Preds can either be logits or probs.
        """
        B = preds.size(0)
        _, sorted_indices = preds.sort(dim=1, descending=True)
        mask = torch.take_along_dim(labels, sorted_indices, dim=1)
        rank_ = RANK.to(mask.device)
        rank = rank_ if B == 32 else rank_[:B]
        scores = (1 / rank * mask).sum() / mask.sum()
        return scores.item() 
    
    
    RANK_NP = np.arange(N_BIRDS).reshape((1, -1))

    def rank_based_accuracy_multi_np(preds, labels):
        """ computes a score that mimics the average ranking of the true labels.
            Supports multiple true labels. Preds can either be logits or probs.
        """
        B = preds.shape[0]
        sorted_indices = np.argsort(preds, axis=1)
        sorted_indices = sorted_indices[::-1]
        mask = np.take_along_axis(labels, sorted_indices, axis=1)
        print(sorted_indices)
        rank_ = RANK_NP
        rank = rank_ if B == 32 else rank_[:B]
        print(rank)
        scores = (1 / rank * mask).sum() / mask.sum()
        return scores

    
    def auc_multi(preds, labels):
        """ Version of macro-averaged ROC-AUC score that ignores all classes that have no true positive labels. """
        scored_cols = labels.sum(axis=0) > 0
        return sklearn.metrics.roc_auc_score(
            labels[:, scored_cols], preds[:, scored_cols], average='macro')

    def get_loss_fn(name):
        reduction = 'none' if (config.use_mask) else 'mean'
        if name == 'bce':
            return nn.BCEWithLogitsLoss(reduction=reduction)
        if name == 'bces':
            return BCEWithLabelSmoothingLoss(smoothing=config.label_smoothing or 0, reduction=reduction)
        if name == 'focal':
            return BCEFocalLoss(alpha=config.loss_alpha or .25, gamma=config.gamma or 2., reduction=reduction)
        if name == 'focal_paper':
            return BCEFocalLossPaper(alpha=config.loss_alpha or .25, gamma=config.gamma or 2., reduction=reduction)
        if name == 'focal_bce':
            return FocalLossBCE(reduction=reduction)
        if name == 'cce':
            return nn.CrossEntropyLoss(label_smoothing=config.label_smoothing or 0, reduction=reduction)

    def test_get_loss_fn():
        t = torch.rand(32, N_BIRDS)
        p = torch.rand(32, N_BIRDS)
        print(get_loss_fn('bce')(p, t))
        print(get_loss_fn('bces')(p, t))
        print(get_loss_fn('focal')(p, t))
        print(get_loss_fn('cce')(p, t))


    class BCEWithLabelSmoothingLoss(torch.nn.Module):
        def __init__(self, smoothing=0.05, reduction='mean'):
            super(BCEWithLabelSmoothingLoss, self).__init__()
            self.smoothing = smoothing
            self.reduction = reduction

        def forward(self, pred, target):
            target_smoothed = target * (1.0 - self.smoothing) + self.smoothing / target.shape[-1]
            loss = F.binary_cross_entropy_with_logits(pred, target_smoothed, reduction=self.reduction)
            return loss


    class BCEFocalLoss(nn.Module):
        """ BCE focalized on positives (alpha applied only to positive) """
        def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
            super().__init__()
            self.alpha = alpha
            self.gamma = gamma
            self.loss_fn = nn.BCEWithLogitsLoss(reduction='none')
            self.reduction = reduction

        def forward(self, preds, targets):
            bce_loss = self.loss_fn(preds, targets)
            proba = torch.sigmoid(preds)
            loss = (targets * self.alpha * (1.0 - proba) ** self.gamma * bce_loss +
                    (1.0 - targets) * proba**self.gamma * bce_loss)
            if self.reduction != 'none':
                loss = loss.mean()
            return loss


    class BCEFocalLossPaper(nn.Module):
        """ BCE focalized on positives (as in paper) """
        def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
            super().__init__()
            self.alpha = alpha
            self.gamma = gamma
            self.loss_fn = nn.BCEWithLogitsLoss(reduction='none')
            self.reduction = reduction

        def forward(self, preds, targets):
            bce_loss = self.loss_fn(preds, targets)
            proba = torch.sigmoid(preds)
            #loss = (targets * self.alpha * (1.0 - proba) ** self.gamma * bce_loss +
            #        (1.0 - targets) * (1 - self.alpha) * proba**self.gamma * bce_loss)
            loss = (targets * self.alpha * (1.0 - proba) ** self.gamma +
                    (1.0 - targets) * (1 - self.alpha) * proba ** self.gamma)
            loss *= bce_loss
            if self.reduction != 'none':
                loss = loss.mean()
            return loss


    class FocalLossBCE(torch.nn.Module):
        """ focal + BCE. other 0.66 2024 using default parameters.
            lr_max = 1e-5
            lr_min = 1e-7
            weight_decay = 1e-6
        """
        def __init__(self, alpha=0.25, gamma=2, reduction="mean", bce_weight=1.0, focal_weight=1.0):
            super().__init__()
            self.alpha = alpha
            self.gamma = gamma
            self.reduction = reduction
            self.bce = torch.nn.BCEWithLogitsLoss(reduction=reduction)
            self.bce_weight = bce_weight
            self.focal_weight = focal_weight

        def forward(self, logits, targets):
            focall_loss = torchvision.ops.focal_loss.sigmoid_focal_loss(
                inputs=logits,
                targets=targets,
                alpha=self.alpha,
                gamma=self.gamma,
                reduction=self.reduction,
            )
            bce_loss = self.bce(logits, targets)
            return self.bce_weight * bce_loss + self.focal_weight * focall_loss
        

    class BCEKDLoss(nn.Module):  ## UNFINISHED
        """ weighted evarage of cce and kl_div with teacher labels.
            p1 2023 with default parameters. """
        def __init__(self, weights=[0.1, 0.9], class_weights=None):
            super().__init__()

            self.weights = weights
            self.T = 20

        def forward(self, output):
            input_ = output["logit"]
            target = output["target"].float()
            rating = output["rating"]
            teacher_preds = output["teacher_preds"]

            rating = rating.unsqueeze(1).repeat(1, CFG.num_classes)
            loss = nn.BCEWithLogitsLoss(
                weight=rating,
                reduction='mean',
            )(input_, target)

            KD_loss = nn.KLDivLoss()(
                F.log_softmax(input_ / self.T, dim=1),
                F.softmax(teacher_preds / self.T, dim=1)
                ) * (self.weights[1] * self.T * self.T)

            return self.weights[0] * loss + KD_loss
        

    class Multi_label_loss:
        """ Unlike cross entropy loss, which only takes into account one class, this loss also considers the neighboring
            classes by applying a weight that corresponds to normal distribution centered on the true value, which assigns 
            a larger weight to the true value, but also includes the neighboring values with a weight that decreases as they
            get farther away from the truth.
            For azimuth a 1Dcnn with circular padding is used. For zenith that wouldn't work because the vaues are the two 
            edges are not continous. Hence, we extend the edges with fictitious classes with a value that mirrors the real
            neighboring classes:
            - zenith:  [.2, .1, | 0, .1, .2, ..., 2.9, 3.0, 3.1, | 3.0, 2.9]
                                 <-------- real classes -------->
        """
        def __init__(self, num_bins, kernel_length, device):
            self.num_bins = num_bins
            # azimuth weights            
            local_weights = np.float32(gaussian_kernel(l=kernel_length, sig=kernel_length/5))

            kernel = torch.nn.Parameter(torch.tensor(local_weights)[None, None, :].to(device), requires_grad=False)
            self.az_conv = torch.nn.Conv1d(1, 1, kernel_size=kernel_length, bias=False,
                                            padding_mode='circular', padding='same', device=device)
            self.az_conv.weight = kernel
            
            # zenith weights
            self.ze_conv = torch.nn.Conv1d(1, 1, kernel_size=kernel_length,  bias=False,
                                           padding_mode='zeros', padding='same', device=device)
            self.ze_conv.weight = kernel

        def nll(self, pred, target):
            pred = pred.log_softmax(dim=1)
            loss = torch.mean(torch.sum(-target * pred, dim=1))
            return loss

        def __call__(self, pred, target):
            paz, pze = pred

            with torch.no_grad():
                az_class = target[:, 0].type(torch.int64)
                ze_class = target[:, 1].type(torch.int64)
                az_onehot = F.one_hot(az_class, self.num_bins).unsqueeze(1).type(torch.float32)
                ze_onehot = F.one_hot(ze_class, self.num_bins).unsqueeze(1).type(torch.float32)
                az_smoothed = self.az_conv(az_onehot).squeeze(1)
                ze_smoothed = self.ze_conv(ze_onehot).squeeze(1)

            loss = self.nll(paz, az_smoothed) + self.nll(pze, ze_smoothed)
            return loss


    def criterion_bce(logits, labels):
        loss_fct = nn.BCEWithLogitsLoss(reduction='none')
        truth = labels.view(-1, 1)
        loss = loss_fct(logits.view(-1, 1), truth)
        loss = torch.masked_select(loss, truth != -1).mean()
        return loss


    def criterion_mse(logits, labels):
        loss_fct = nn.MSELoss(reduction='none')
        truth = labels.view(-1, 1)
        loss = loss_fct(logits.view(-1, 1), truth)
        loss = torch.masked_select(loss, truth != -1).mean()
        return loss

    # ----------------------------------------------------------------------------------------------------
    # 0.7 Optimizers & schedulers
    # ----------------------------------------------------------------------------------------------------

    class AdamW(torch.optim.Optimizer):
        """
        Implements Adam algorithm with weight decay fix as introduced in
        `Decoupled Weight Decay Regularization <https://arxiv.org/abs/1711.05101>`__.

        Parameters:
            params (:obj:`Iterable[torch.nn.parameter.Parameter]`):
                Iterable of parameters to optimize or dictionaries defining parameter groups.
            lr (:obj:`float`, `optional`, defaults to 1e-3):
                The learning rate to use.
            betas (:obj:`Tuple[float,float]`, `optional`, defaults to (0.9, 0.999)):
                Adam's betas parameters (b1, b2).
            eps (:obj:`float`, `optional`, defaults to 1e-6):
                Adam's epsilon for numerical stability.
            weight_decay (:obj:`float`, `optional`, defaults to 0):
                Decoupled weight decay to apply.
            correct_bias (:obj:`bool`, `optional`, defaults to `True`):
                Whether ot not to correct bias in Adam (for instance, in Bert TF repository they use :obj:`False`).
        """

        def __init__(
            self,
            params,
            lr = 1e-3,
            betas = (0.9, 0.999),
            eps= 1e-6,
            weight_decay = 0.0,
            correct_bias = True,
        ):
            if lr < 0.0:
                raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
            if not 0.0 <= betas[0] < 1.0:
                raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
            if not 0.0 <= betas[1] < 1.0:
                raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
            if not 0.0 <= eps:
                raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
            defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias)
            super().__init__(params, defaults)

        def step(self, closure = None):
            """
            Performs a single optimization step.

            Arguments:
                closure (:obj:`Callable`, `optional`): A closure that reevaluates the model and returns the loss.
            """
            loss = None
            if closure is not None:
                loss = closure()

            for group in self.param_groups:
                for p in group["params"]:
                    if p.grad is None:
                        continue
                    grad = p.grad.data
                    if grad.is_sparse:
                        raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                    state = self.state[p]

                    # State initialization
                    if len(state) == 0:
                        state["step"] = 0
                        # Exponential moving average of gradient values
                        state["exp_avg"] = torch.zeros_like(p.data)
                        # Exponential moving average of squared gradient values
                        state["exp_avg_sq"] = torch.zeros_like(p.data)

                    exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                    beta1, beta2 = group["betas"]

                    state["step"] += 1

                    # Decay the first and second moment running average coefficient
                    # In-place operations to update the averages at the same time
                    exp_avg.mul_(beta1).add_(grad, alpha=1.0 - beta1)
                    exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1.0 - beta2)
                    denom = exp_avg_sq.sqrt().add_(group["eps"])

                    step_size = group["lr"]
                    if group["correct_bias"]:  # No bias correction for Bert
                        bias_correction1 = 1.0 - beta1 ** state["step"]
                        bias_correction2 = 1.0 - beta2 ** state["step"]
                        step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

                    p.data.addcdiv_(exp_avg, denom, value=-step_size)

                    # Just adding the square of the weights to the loss function is *not*
                    # the correct way of using L2 regularization/weight decay with Adam,
                    # since that will interact with the m and v parameters in strange ways.
                    #
                    # Instead we want to decay the weights in a manner that doesn't interact
                    # with the m/v parameters. This is equivalent to adding the square
                    # of the weights to the loss with plain (non-momentum) SGD.
                    # Add weight decay at the end (fixed version)
                    if group["weight_decay"] > 0.0:
                        p.data.add_(p.data, alpha=-group["lr"] * group["weight_decay"])

            return loss


    def fetch_optimizer(model, config):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
             "weight_decay": config.weight_decay,},
            {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
             "weight_decay": 0.0,},
        ]
        if config.adam8:
            optimizer = bnb.optim.Adam8bit(optimizer_parameters, lr=config.lr)
        elif config.optimizer == 'AdamW':
            #optimizer = torch.optim.AdamW(optimizer_parameters, lr=config.lr, eps=config.eps, betas=config.betas)
            optimizer = AdamW(optimizer_parameters, lr=config.lr, eps=config.eps, betas=config.betas)
        #elif config.optimizer == 'Adamw':
        #    optimizer = torch.optim.Adam(optimizer_parameters, lr=config.lr)
        elif config.optimizer == 'Adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
        elif config.optimizer == 'RAdam':
            optimizer = torch.optim.RAdam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay or 0)
        elif config.optimizer == 'Lion':
            optimizer = Lion(model.parameters(), lr=config.lr, weight_decay=config.weight_decay or 0)

        # restore state if applicable
        optimizer_loaded = False
        if config.saved_optimizer:
            filename = config.saved_optimizer
            if os.path.exists(filename):
                print(f"Loading optimizer '{filename}'")
                try:
                    checkpoint = torch.load(filename)
                except RuntimeError:
                    checkpoint = torch.load(filename, map_location=torch.device('cpu'))
                optimizer.load_state_dict(checkpoint['optimizer_states'][-1])
                optimizer_loaded = True
            else:
                # set initial learning rate if resuming training without a saved optimizer state.
                print('optimizer not found.')
        if config.previous_step is not None and config.previous_step != -1 and not optimizer_loaded:
            for group in optimizer.param_groups:
                group.setdefault('initial_lr', group['lr'])
        return optimizer


    def fetch_scheduler(config, optimizer):
        previous_step = config.previous_step or -1
        if config.schedule_type in ['linear', 'cos']:
            pct_start = config.num_warmup_steps / config.num_train_steps
            div_factor = config.lr / config.start_lr # default 1/25
            final_div_factor = config.start_lr / config.end_lr if config.end_lr is not None else 1e4
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer, anneal_strategy=config.schedule_type, last_epoch=previous_step,
                total_steps=config.num_train_steps, pct_start=pct_start,
                max_lr=config.lr, div_factor=div_factor, final_div_factor=final_div_factor)
        elif config.schedule_type == 'multi_cos':
            gamma = get_gamma(
                config.lr, config.end_lr, config.num_train_steps - config.num_warmup_steps)
            num_warmup_steps = config.num_warmup_steps
            start_factor = 1e-5 / config.lr if config.start_lr is None else config.start_lr / config.lr
            scheduler1 = torch.optim.lr_scheduler.LinearLR(
                optimizer, start_factor=start_factor, end_factor=1, total_iters=num_warmup_steps)
            cycle_steps = int(config.num_train_steps / config.num_cycles)
            scheduler2 = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, cycle_steps, T_mult=1, eta_min=config.end_lr)
            scheduler = torch.optim.lr_scheduler.SequentialLR(
                optimizer, schedulers=[scheduler1, scheduler2],
                milestones=[config.num_warmup_steps], last_epoch=-1)
        elif config.schedule_type == 'StepLR':
            gamma = config.gamma or 0.999
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma, last_epoch=previous_step)
        elif config.schedule_type == 'exponential':
            gamma = config.gamma or get_gamma(config.lr, config.end_lr, config.num_train_steps)
            print(gamma)
            scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma, last_epoch=previous_step)
        elif config.schedule_type == 'exponential_warmup':
            gamma = config.gamma or get_gamma(
                config.lr, config.end_lr, config.num_train_steps - config.num_warmup_steps)
            num_warmup_steps = config.num_warmup_steps
            start_factor = 1e-5 / config.lr if config.start_lr is None else config.start_lr / config.lr
            scheduler1 = torch.optim.lr_scheduler.LinearLR(
                optimizer, start_factor=start_factor, end_factor=1, total_iters=num_warmup_steps)
            scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma, last_epoch=previous_step)
            scheduler = torch.optim.lr_scheduler.SequentialLR(
                optimizer, schedulers=[scheduler1, scheduler2],
                milestones=[config.num_warmup_steps], last_epoch=previous_step)
            #scheduler = torch.optim.lr_scheduler.ChainedScheduler([scheduler1, scheduler2])
        elif config.schedule_type == 'rop':
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **config.rop)

        # restore state if applicable
        if config.saved_scheduler:
            filename = config.saved_scheduler
            if os.path.exists(filename):
                print(f"Loading scheduler '{filename}'")
                try:
                    checkpoint = torch.load(filename)
                except RuntimeError:
                    checkpoint = torch.load(filename, map_location=torch.device('cpu'))
                scheduler.load_state_dict(checkpoint['lr_schedulers'][-1])
                # if hasattr(scheduler, 'last_epoch'):
                #     config.previous_step = scheduler.last_epoch
                # else:
                #     config.previous_step = scheduler._schedulers[0].last_epoch
            else:
                print('scheduler not found.')
        return scheduler


    def get_gamma(start_lr, end_lr, steps):
        """ return 1 - gamma such that gamma gets us from start_lr to end_lr after 'steps' """
        diff = math.log(end_lr) - math.log(start_lr)
        rate = diff / steps
        return math.exp(rate)


    class CosineAnnealingWarmupRestarts(torch.optim.lr_scheduler._LRScheduler):
        """
            optimizer (Optimizer): Wrapped optimizer.
            first_cycle_steps (int): First cycle step size.
            cycle_mult(float): Cycle steps magnification. Default: -1.
            max_lr(float): First cycle's max learning rate. Default: 0.1.
            min_lr(float): Min learning rate. Default: 0.001.
            warmup_steps(int): Linear warmup step size. Default: 0.
            gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
            last_epoch (int): The index of last epoch. Default: -1.
        """

        def __init__(self,
                     optimizer: torch.optim.Optimizer,
                     first_cycle_steps: int,
                     cycle_mult: float = 1.,
                     max_lr: float = 0.1,
                     min_lr: float = 0.001,
                     warmup_steps: int = 0,
                     gamma: float = 1.,
                     last_epoch: int = -1
                     ):
            assert warmup_steps < first_cycle_steps

            self.first_cycle_steps = first_cycle_steps  # first cycle step size
            self.cycle_mult = cycle_mult  # cycle steps magnification
            self.base_max_lr = max_lr  # first max learning rate
            self.max_lr = max_lr  # max learning rate in the current cycle
            self.min_lr = min_lr  # min learning rate
            self.warmup_steps = warmup_steps  # warmup step size
            self.gamma = gamma  # decrease rate of max learning rate by cycle

            self.cur_cycle_steps = first_cycle_steps  # first cycle step size
            self.cycle = 0  # cycle count
            self.step_in_cycle = last_epoch  # step size of the current cycle

            super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)

            # set learning rate min_lr
            self.init_lr()

        def init_lr(self):
            self.base_lrs = []
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = self.min_lr
                self.base_lrs.append(self.min_lr)

        def get_lr(self):
            if self.step_in_cycle == -1:
                return self.base_lrs
            elif self.step_in_cycle < self.warmup_steps:
                return [(self.max_lr - base_lr) * self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
            else:
                return [base_lr + (self.max_lr - base_lr)
                        * (1 + math.cos(math.pi * (self.step_in_cycle - self.warmup_steps)
                                        / (self.cur_cycle_steps - self.warmup_steps))) / 2
                        for base_lr in self.base_lrs]

        def step(self, epoch=None):
            if epoch is None:
                epoch = self.last_epoch + 1
                self.step_in_cycle = self.step_in_cycle + 1
                if self.step_in_cycle >= self.cur_cycle_steps:
                    self.cycle += 1
                    self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                    self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
            else:
                if epoch >= self.first_cycle_steps:
                    if self.cycle_mult == 1.:
                        self.step_in_cycle = epoch % self.first_cycle_steps
                        self.cycle = epoch // self.first_cycle_steps
                    else:
                        n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                        self.cycle = n
                        self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                        self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
                else:
                    self.cur_cycle_steps = self.first_cycle_steps
                    self.step_in_cycle = epoch

            self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
            self.last_epoch = math.floor(epoch)
            for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
                param_group['lr'] = lr

    # ----------------------------------------------------------------------------------------------------
    # 0.8 Lightning training
    # ----------------------------------------------------------------------------------------------------

    def get_amp_context(device_type, datatype):
        """ get context for """
        ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[datatype]
        device_type = 'cuda' if 'cuda' in str(device_type) else str(device_type)
        #context = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
        context = nullcontext() if device_type == 'cpu' else torch.cuda.amp.autocast(dtype=ptdtype)  # for torch v1.11
        return context


    def pb_format_num(n):
        """ format numbers in progress bar """
        f = '{0:.4g}'.format(n).replace('+0', '+').replace('-0', '-')
        n = str(n)
        return f if len(f) < len(n) else n


    class My_progress_bar(TQDMProgressBar):
        """ customize TQDMProgressBar to add lr """
        def get_metrics(self, trainer, pl_module):
            items = super().get_metrics(trainer, pl_module)
            items.pop('v_num', None)  # added in base class; contains version number
            scheduler = trainer.model.lr_schedulers()
            lr = scheduler.get_last_lr()[0] if hasattr(scheduler, 'get_last_lr') else trainer.model.optimizers().param_groups[0]['lr']
            items['lr'] = lr[0] if isinstance(lr, list) else lr
            items = {k: pb_format_num(v) for k, v in items.items()}
            return items

        def init_validation_tqdm(self):
            """ disable validation progress bar """
            bar = tqdm(disable=True)
            return bar
            

    def fg_format_seconds(seconds):
        seconds = int(seconds)
        minutes = seconds // 60
        remaining_seconds = seconds % 60
        return f"{minutes:02}:{remaining_seconds:02}"


    class Exponential_average(object):
        def __init__(self, decay=.01):
            self.decay = decay
            self.val = None

        def reset(self):
            pass

        def update(self, pred=None):
            val = pr_auc_score(true, pred)
            if self.keep_pred:
                self.preds.append(pred)
            self.val = (self.val*(1-self.decay) + val*self.decay) if self.val is not None else val

        @property
        def avg(self):
            return self.val

        def get_pred(self):
            res = np.concatenate(self.preds, axis=0)
            self.preds = []
            return res


    class AverageMeter:
        def __init__(self, labels=None):
            self.reset()
            self.auc = None
            self.labels = labels if labels is not None else None
            
        def reset(self):
            self.loss_sum = 0
            self.acc_sum = 0
            self.count = 0
            self.preds = []
        
        def update(self, loss, acc):
            self.loss_sum += loss
            self.acc_sum += acc
            self.count += 1
            return loss
        
        def update_preds(self, preds):
            self.preds.append(preds.detach().cpu())
        
        @property
        def loss(self):
            return self.loss_sum / self.count if self.count > 0 else 0

        @property
        def acc(self):
            return self.acc_sum / self.count if self.count > 0 else 0
        
        def calculate_auc(self):
            self.preds = np.concatenate(self.preds, axis=0)
            len_preds = self.preds.shape[0]  # to support sanity check
            self.auc = auc_multi(self.preds, self.labels[:len_preds])
            

    class Pl_model(L.LightningModule):
        def __init__(self, model, cfg, val_labels=None, silent=None):
            super().__init__()
            self.model = model
            self.cfg = cfg
            self.silent = silent if silent is not None else False
            self.save_hyperparameters()

            self.automatic_optimization = False
            self.current_amp_dtype = None
            self.current_loss_name = None
            self.saved_inputs = [] if cfg.awp or cfg.fgm else None
            self._setup_amp()
            self._setup_loss()
            self.train_meter = AverageMeter()
            self.val_meter = AverageMeter(val_labels) if val_labels is not None else None
            self.t0 = time.time()

        def _setup_amp(self):
            """ adjust automatic mode precision dtype based on epoch """
            dtype = self.cfg.precision_schedule.get(self.current_epoch, None)
            if dtype is not None and self.current_amp_dtype != dtype:
                self.amp_context = get_amp_context(self.device, dtype)
                if not self.silent:
                    print(f"Switched amp context to {dtype}")
                self.scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'),
                                                        growth_interval=600)
            assert hasattr(self, 'amp_context'), "Amp context not set, check precision schedule"
            self.current_amp_dtype = dtype
            os.environ['EPOCH'] = str(self.current_epoch)

        def _setup_loss(self):
            """ adjust loss based on epoch """
            loss_name = self.cfg.loss_schedule.get(self.current_epoch, None)
            if loss_name is not None and self.current_loss_name != loss_name:
                self.loss_fn = get_loss_fn(loss_name)
                self.current_loss_name = loss_name

        def _compute_loss_metrics(self, pred, data, meter):
            y = data['y']
            mask = data.get('mask', None)
            loss = self.loss_fn(pred, y)
            if mask is not None:
                loss = (loss * mask).sum() / mask.sum()
            else:
                loss = loss.mean()
                
            # assert loss >= 0
            acc = rank_based_accuracy_multi(pred, y)
            meter.update(loss.item(), acc)
            return loss

        def on_train_start(self):
            pass
            # self._setup_amp()

        def on_train_epoch_start(self):
            self._setup_amp()
            self._setup_loss()
            self.train_meter.reset()
            if self.val_meter:
                self.val_meter.reset()
            self.t0 = time.time()                     
        
        def training_step(self, batch, batch_idx):
            cfg = self.cfg
            meter = self.train_meter
            if cfg.aug.mixup:
                mixup(batch)
            if self.saved_inputs is not None:
                self.saved_inputs.append(batch)
            with self.amp_context:
                preds = self(batch['x'])
                loss = self._compute_loss_metrics(preds, batch, meter)
            out = dict(loss=meter.loss, acc=meter.acc)
            if torch.isnan(loss).any():
                print('ERROR: loss is nan. Interrupting training.')
                self.trainer.should_stop = True
            self.log_dict(out, prog_bar=True)  # prog_bar shows most recent value, not mean, which is fine 
                                               # because out already contains the mean.
            # print(loss.item(), meter.acc, trainer.callback_metrics)
            self.manual_backward(self.scaler.scale(loss))

            if (batch_idx + 1) % cfg.accumulation_steps == 0:  # replace by GradientAccumulationScheduler callback?
                score = self.val_meter.auc if self.val_meter else self.train_meter.loss
                if cfg.awp and score is not None and score >= cfg.adv_th:
                    self.awp.attack_backward(self.saved_inputs)
                if cfg.fgm and score is not None and score >= cfg.adv_th:
                    self.fgm.attack_backward(self.saved_inputs)

                optimizer = self.optimizers()
                self.scaler.unscale_(optimizer)
                if cfg.max_grad_value is not None:
                    torch.nn.utils.clip_grad_value_(self.parameters(), cfg.max_grad_value)
                elif cfg.max_grad_norm is not None:
                    norm_grad = torch.nn.utils.clip_grad_norm_(parameters=self.parameters(),
                                                               max_norm=cfg.max_grad_norm)
                self.scaler.step(optimizer)
                self.scaler.update()
                optimizer.zero_grad(set_to_none=True)
                if cfg.step_scheduler_after == 'step':
                    self.lr_schedulers().step()
                if self.saved_inputs is not None:
                    self.saved_inputs.clear()
            # return loss_metrics  # don't return because we do the backpass here

        def on_train_epoch_end(self):
            """ invoked after validation to process outputs of training_step. """
            ds = self.trainer.train_dataloader.dataset
            if hasattr(ds, 'on_epoch_end'):
                ds.on_epoch_end()
            if self.cfg.step_scheduler_after == 'epoch':  # needed due to manual optimization  
                self.lr_schedulers().step()
            if self.val_meter is None:
                loss_metrics_str = f'loss: {self.train_meter.loss:.4f} - acc: {self.train_meter.acc:.4f}'
                scheduler = self.lr_schedulers()
                lr = scheduler.get_last_lr()[0] if hasattr(scheduler, 'get_last_lr') else self.optimizers().param_groups[0]['lr']
                lr = lr[0] if isinstance(lr, list) else lr
                lr = f'{lr:.3e}'.replace('+0', '+').replace('-0', '-')
                if not self.silent:                   
                    print(f'epoch:{self.current_epoch:>3} - {loss_metrics_str} - lr: {lr}')
            return super().on_train_epoch_end()

        def on_validation_start(self):
            self._setup_amp()

        def validation_step(self, batch, batch_idx):
            """ no logging per step """
            meter = self.val_meter
            with self.amp_context:
                preds = self(batch['x'])
                self._compute_loss_metrics(preds, batch, meter)
                meter.update_preds(preds)
            # out = dict(val_loss=meter.loss, val_kl=meter.kl)
            # self.log_dict(out, on_step=True, prog_bar=True)

        def on_validation_epoch_end(self):
            """ process outputs of validation_step """
            self.val_meter.calculate_auc()
            out = dict(val_loss=self.val_meter.loss, val_acc=self.val_meter.acc, val_auc=self.val_meter.auc)
            self.log_dict(out, prog_bar=False)  # disable prog_bar to simplify progress bar; we write it to the screen anyway
            loss_metrics_str = f'loss: {self.train_meter.loss:.4f} - acc: {self.train_meter.acc:.4f} - ' \
                               f'val_loss: {self.val_meter.loss:.4f} - val_acc: {self.val_meter.acc:.4f} - ' \
                               f'val_auc: {self.val_meter.auc:.4f}'
            scheduler = self.lr_schedulers()
            lr = scheduler.get_last_lr()[0] if hasattr(scheduler, 'get_last_lr') else self.optimizers().param_groups[0]['lr']
            lr = lr[0] if isinstance(lr, list) else lr
            lr = f'{lr:.3e}'.replace('+0', '+').replace('-0', '-')
            duration = time.time() - self.t0
            if not self.silent:                                
                print(f'epoch:{self.current_epoch:>3} - {loss_metrics_str} - lr: {lr} - {duration:,.0f}s', ' '*40)
            
            # print() # Keep the previous progbar
            # self.trainer.val_dataloaders[0].dataset.on_epoch_end()
            return super().on_validation_epoch_end()

        def forward(self, batch):
            return self.model(batch)

        def configure_optimizers(self):
            cfg = self.cfg
            if not cfg.warmup_share:
                cfg.start_lr = cfg.lr 
            elif cfg.start_lr is None:
                cfg.start_lr = cfg.lr / 25
            optimizer = fetch_optimizer(self.model, cfg)
            if self.cfg.awp:
                self.awp = AWP(self, optimizer, adv_lr=cfg.adv_lr, adv_eps=cfg.adv_eps, cfg=cfg)
            if self.cfg.fgm:
                self.fgm = FGM(self, optimizer, self.scaler, epsilon=cfg.fgm_epsilon, emb_name='word_embeddings')
            scheduler = fetch_scheduler(self.cfg, optimizer)
            lr_scheduler_config = {"scheduler": scheduler, "interval": self.cfg.step_scheduler_after}
            if cfg.monitor:
                lr_scheduler_config['monitor'] = cfg.monitor
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_config}

    
    def train(meta, files, no_try=False):
        global config
        
        my_log = Log(f'{config.name}.log', time_key=False)
        logger = get_logger(config)
        my_log(f'config: {config}\n')
        folds = [0]
        config.img_dim = SPEC_DIMS.get(config.spec, (128, 256))
        
        for fold in folds:
            print(f'{"-"*40} {config.name}_f{fold} {"-"*40}')
            if config.monitor == 'val_auc':
                train_idx, val_idx = meta.fold.values != fold, meta.fold.values == fold
                if config.filter[-1] > 0:
                    train_idx &= meta.rating >= config.filter[-1]
                if config.dataset_val:
                    val_labels = load_obj(f'/kaggle/input/b5-data-{config.dataset_val}/y_f{fold}')
                else:
                    val_labels = labels[val_idx]
            else:
                train_idx, val_idx = np.full(meta.shape[0], True), None
                val_labels = None
            
            # set ws for train sample
            ws_power = config.ws_power or 1/2
            bird_sum = meta[train_idx].groupby('primary_label')['i'].transform('size').values
            sampling_th = 6
            ws = (1 / np.where(bird_sum <= sampling_th, bird_sum, sampling_th + np.power(bird_sum - sampling_th, ws_power)))
            times_picked = ws * bird_sum
            print(f'raw weights: {times_picked.min():.2f} - {times_picked.max():.2f}')
            ws /= ws.sum()
            
            epochs = config.epochs
            config.num_train_steps = int((train_idx.sum()) / (config.train_batch_size * config.accumulation_steps)
                                         * epochs)
            config.num_warmup_steps = int(config.num_train_steps * config.warmup_share)
            print(f'train steps: {config.num_train_steps:,} - warmup steps: {config.num_warmup_steps:,}')
            custom_filter = build_custom_filter(meta[train_idx])
            data_module = Ba_data_module_montage_h5(
                files, labels, birds, train_idx, val_idx, config.dataset_val, fold, train_mode=config.train_mode,
                ws=ws, files_unlabeled=files_unlabeled if config.unlabeled_prob else None,
                custom_filter=custom_filter)

            if 'resume' in mode:
                print('Resume training')
                trainer = L.Trainer()
                pl_model = Pl_model.load_from_checkpoint(config.checkpoint)
                trainer.fit(pl_model, data_module, ckpt_path=config.checkpoint)
            else:
                model_class = globals()[config.model_name]
                pretrained = config.pretrained_encoder if config.pretrained_encoder is not None else True
                in_chans = config.in_chans or 3
                model = model_class(
                    config, resize=config.resize, add_position=config.add_position,
                    out_indices=config.out_indices, pretrained=pretrained, in_chans=in_chans)
                inp_dim = (1, config.img_dim[0], config.img_dim[1] * config.img_duration // 5)
                example_input_array = [torch.zeros(8, *inp_dim)]
                _ = model(example_input_array[0])  # sanity check
                # model = SpecNetCwt2D(config, resize=config.resize)
                # model = RawSignalNet(config, resize=config.resize)

                if config.checkpoint:
                    print('load existing model')
                    try:
                        chk = torch.load(config.checkpoint)
                    except:
                        chk = torch.load(config.checkpoint, map_location=torch.device('cpu'))
                    
                    # model_state = {k.replace('model.', ''): v for k, v in chk['state_dict'].items()}
                    model_state = {k[6:]: v for k, v in chk['state_dict'].items()}
                    model.load_state_dict(model_state)

                pl_model = Pl_model(model, config, val_labels=val_labels)
                pl_model.example_input_array = example_input_array
                print_model_size(pl_model)
                # display(summary(model, col_names=["num_params","trainable"]))  # show layer sizes
                # model  # show dimensions

                if val_idx is not None:
                    loss_file = '{epoch}-{val_loss:.4f}'
                    loss_monitor = 'val_loss'
                    metric_file = '{epoch}-{val_auc:.4f}'
                    metric_monitor = 'val_auc'
                else:
                    loss_file = '{epoch}-{loss:.4f}'
                    loss_monitor = 'loss'
                    metric_file = '{epoch}-{acc:.4f}'
                    metric_monitor = 'acc'
                chk_callback_metric = ModelCheckpoint(
                    dirpath='./', filename=f'{config.name}_f{fold}-{metric_file}',
                    save_top_k=3, monitor=metric_monitor, mode=config.mode, 
                    save_last=True, save_on_train_epoch_end=True, save_weights_only=True)
                chk_callback_loss = ModelCheckpoint(
                    dirpath='./', filename=f'{config.name}_f{fold}-{loss_file}',
                    save_top_k=3, monitor=loss_monitor, mode='min', 
                    save_last=False, save_on_train_epoch_end=True, save_weights_only=True)
                callbacks = [chk_callback_metric, chk_callback_loss, 
                             LearningRateMonitor(logging_interval='step'),
                             My_progress_bar(refresh_rate=config.log_every_n_steps),
                            ]
                if config.patience and val_idx is not None:
                    early_stop_callback = EarlyStopping(
                        monitor=config.monitor, min_delta=1e-5, patience=config.patience,
                        verbose=False, mode=config.mode)
                    callbacks.append(early_stop_callback)
                if config.swa is not None:
                    swa_lrs, swa_epoch_start, annealing_epochs = config.swa
                    swa_callback = StochasticWeightAveraging(
                        swa_lrs=swa_lrs, swa_epoch_start=swa_epoch_start, annealing_epochs=annealing_epochs,
                        annealing_strategy='cos')
                    callbacks.append(swa_callback)
                    print('***** swa *****')
                    
                max_epochs = config.max_epochs if config.max_epochs is not None else epochs
                num_sanity_val_steps = 1 if is_interactive() and config.monitor != 'acc' else 0
                trainer = L.Trainer(max_epochs=max_epochs, max_time=config.duration, logger=logger,
                                    log_every_n_steps=config.log_every_n_steps, #val_check_interval=config.val_check_interval,
                                    check_val_every_n_epoch=config.check_val_every_n_epoch,
                                    accelerator='auto', devices='auto',
                                    num_sanity_val_steps=num_sanity_val_steps,
                                    limit_train_batches=None, limit_val_batches=None,
                                    enable_checkpointing=True, callbacks=callbacks,
                                    # gradient_clip_val=1.0,
                                   )
                print('start training...')
                if no_try:
                    trainer.fit(pl_model, data_module)
                else:
                    try:
                        trainer.fit(pl_model, data_module)
                    except Exception as e:
                        print(f'\ntraining interrupted: {e}')

                trainer.save_checkpoint(f'last_train_{config.name}_f{fold}.ckpt', weights_only=True)  # needed if swa
                if pl_model.val_meter:
                    save_obj(pl_model.val_meter.preds, f'preds_{config.name}_f{fold}')

        print('Finished training...')
        del data_module
        gc.collect()
        if config.wandb_project:
            wandb.finish()
    
    # ----------------------------------------------------------------------------------------------------
    # 0.9 adversarial training classes
    # ----------------------------------------------------------------------------------------------------

    class FGM():
        """ FGM attack. Rather than updating the model using the gradient, it perturbs the weights of the embedding layer
            using the computed gradients, calculates a perturbed loss and gradients, restores the weights and uses the
            perturbed gradients to update the weights.

            The emb_name parameter should be replaced with the parameter name of the embedding in your model
        """
        def __init__(self, model, optimizer, scaler=None,
                     epsilon=1., emb_name='word_embeddings', do_add_gradients=True):
            self.model = model
            self.optimizer = optimizer
            self.scaler = scaler
            self.emb_name = emb_name
            self.epsilon = epsilon
            self.emb_name = emb_name
            self.do_add_gradients = do_add_gradients
            self.backup = {}

        def attack_backward(self, inputs):
            accumulation_steps = config.accumulation_steps
            self._attack_step()
            if not self.do_add_gradients:
                self.optimizer.zero_grad()  # usually we add the gradients
            for input in inputs:
                with torch.cuda.amp.autocast():
                    tr_logits = self.model(input)
                    adv_loss = Von_mises_fisher_3D_loss(tr_logits, input)
                if accumulation_steps > 1:
                    adv_loss /= accumulation_steps
                if self.scaler is not None:
                    self.scaler.scale(adv_loss).backward()
                else:
                    adv_loss.backward()
                torch.nn.utils.clip_grad_norm_(parameters=model.parameters(),
                                               max_norm=config.max_grad_norm)
            self._restore()
            #print(f'fgm {epoch} {adv_loss.item()}')


        def _attack_step(self):
            emb_name = self.emb_name
            epsilon = self.epsilon
            for name, param in self.model.named_parameters():
                if param.requires_grad and emb_name in name:
                    self.backup[name] = param.data.clone()
                    norm = torch.norm(param.grad)
                    if norm != 0 and not torch.isnan(norm):
                        r_at = epsilon * param.grad / norm
                        param.data.add_(r_at)

        def _restore(self):
            # The emb_name parameter should be replaced with the parameter name of the embedding in your model
            emb_name = self.emb_name
            for name, param in self.model.named_parameters():
                if param.requires_grad and emb_name in name:
                    assert name in self.backup
                    param.data = self.backup[name]
            self.backup = {}

        # example of usage
        # fgm = FGM(model)
        # for batch_input, batch_label in data:
        #       loss = model(batch_input, batch_label)
        #       loss.backward()
        #
        #       # adversarial training
        #       fgm.attack()
        #       loss_adv = model(batch_input, batch_label)
        #       loss_adv.backward()
        #       fgm.restore()
        #
        #       optimizer.step()
        #       model.zero_grad()

    # --------------------
    # 0.10 AWP
    # --------------------

    class AWP:
        """ adversarial weight perturbation. Rather than updating the model using the gradient, it perturbs the weights
            using the computed gradients, calculates a perturbed loss and gradients, restores the weights and uses the
            perturbed gradients to update the weights.
        """
        def __init__(self, pl_model, optimizer, adv_param="weight", adv_lr=1, 
                     adv_eps=0.01, adv_step=1, config=None):
            self.pl_model = pl_model
            self.optimizer = optimizer
            self.adv_param = adv_param
            self.adv_lr = adv_lr
            self.adv_eps = adv_eps
            self.adv_step = adv_step
            self.config = config
            self.backup = {}
            self.backup_eps = {}

        def attack_backward(self, inputs):
            if (self.adv_lr == 0):
                return None

            config = self.config
            pl_model = self.pl_model
            model = pl_model.model
            amp_context = pl_model.amp_context
            scaler = pl_model.scaler
            loss_fn = pl_model.loss_fn if pl_model.loss_fn is not None else model.compute_loss
            accumulation_steps = config.accumulation_steps

            self._save()
            for i in range(self.adv_step):
                self._attack_step()
                self.optimizer.zero_grad()
                for batch in inputs:   # to support accumulation_steps > 1
                    with amp_context:
                        preds = pl_model(batch)
                        adv_loss = loss_fn(preds, batch)
                        if isinstance(adv_loss, dict):
                            adv_loss = adv_loss['loss']
                    if accumulation_steps > 1:
                        adv_loss /= accumulation_steps
                    if scaler is not None:
                        scaler.scale(adv_loss).backward()
                    else:
                        adv_loss.backward()
                    if config.max_grad_value is not None:
                        torch.nn.utils.clip_grad_value_(model.parameters(), config.max_grad_value)
                    elif config.max_grad_norm is not None:
                        norm_grad = torch.nn.utils.clip_grad_norm_(parameters=model.parameters(),
                                                                   max_norm=config.max_grad_norm)
            self._restore()
            #print(f'awp {epoch} {adv_loss.item()}')

        def _attack_step(self):
            e = 1e-6
            for name, param in self.pl_model.model.named_parameters():
                if param.requires_grad and param.grad is not None and self.adv_param in name:
                    norm1 = torch.norm(param.grad)
                    norm2 = torch.norm(param.data.detach())
                    if norm1 != 0 and not torch.isnan(norm1):
                        r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                        param.data.add_(r_at)
                        param.data = torch.min(
                            torch.max(param.data, self.backup_eps[name][0]),
                            self.backup_eps[name][1]
                        )
                    # param.data.clamp_(*self.backup_eps[name])

        def _save(self):
            for name, param in self.pl_model.model.named_parameters():
                if param.requires_grad and param.grad is not None and self.adv_param in name:
                    if name not in self.backup:
                        self.backup[name] = param.data.clone()
                        grad_eps = self.adv_eps * param.abs().detach()
                        self.backup_eps[name] = (
                            self.backup[name] - grad_eps,
                            self.backup[name] + grad_eps,
                        )

        def _restore(self,):
            for name, param in self.pl_model.model.named_parameters():
                if name in self.backup:
                    param.data = self.backup[name]
            self.backup = {}
            self.backup_eps = {}

    # ----------------------------------------------------------------------------------------------------
    # 0.11 dummy lightning logger
    # ----------------------------------------------------------------------------------------------------

    from pytorch_lightning.loggers.logger import Logger, rank_zero_experiment
    from pytorch_lightning.utilities import rank_zero_only

    class Dummy_logger(Logger):
        """ Needed to bypass bug in default logger class.
            for pytorch_lightning==1.9.4. 
        """
        
        @property
        def name(self):
            return "MyLogger"

        @property
        def version(self):
            return "0.1"

        @rank_zero_only
        def log_hyperparams(self, params):
            # params is an argparse.Namespace
            pass

        @rank_zero_only
        def log_metrics(self, metrics, step):
            # metrics is a dictionary of metric names and values
            pass

        @rank_zero_only
        def save(self):
            # Optional. Any code necessary to save logger data
            pass

        @rank_zero_only
        def finalize(self, status):
            # Optional. Any code that needs to be run after training finishes.
            pass

    def get_logger(config, group='att', job_type='train'):
        if config.wandb_project:
            os.environ['WANDB_SILENT'] = 'true'
            key = UserSecretsClient().get_secret('wandb')
            wandb.login(key=key)
            logger = WandbLogger(name=config.name, project=config.wandb_project, config=config, group=group, job_type=job_type)
        else:
            logger = Dummy_logger()
        return logger

    # ----------------------------------------------------------------------------------------------------
    # 0.12 submit
    # ----------------------------------------------------------------------------------------------------

    def merge_stat_dict(m1, m2, layers=['_conv_layers', '_post_processing']):
        """ takes the state_dict of two models and replaces 'm1' parameters
            whose key matches 'layers' with the values from 'm2'
        """
        for k, v in m1.items():
            for match in layers:
                if match in k and k in m2:
                    m1[k] = m2[k]
                    continue
        return m1


    def merge_stat_dict_files(m1_path, m2_path, layers=['_conv_layers', '_post_processing']):
        """ reads the state_dict of two models given their paths and replaces 'm1' parameters
            whose key matches 'layers' with the values from 'm2'
        """
        m1 = torch.load(m1_path)
        m2 = torch.load(m2_path)
        return merge_stat_dict(m1, m2, layers)


    def merge_models(model, m2_path, layers):
        """ takes a model and merges it with the model whose stat_dict is saved in m2_path
            for the layers whose key matches 'layers'.
        """
        m1_path = 'temp.pth'
        model.save_state_dict(m1_path)
        m1 = merge_stat_dict_files(m1_path, m2_path, layers)
        model.load_state_dict(m1)


    def get_best_checkpoints(model_name):
        file = f'/kaggle/working/{model_name}.ckpt'
        if os.path.exists(file):
            return [file]
        
        files = glob.glob(f'/kaggle/working/{model_name}_epoch*.ckpt')
        sorted_files = []
        for file in files:
            metrics = file.split('_')[1][:-5]
            epoch, acci = metrics.split('-')
            epoch = int(epoch.split('=')[1])
            acci = float(acci.split('=')[1])
            sorted_files.append((acci, epoch, file))
        sorted_files = sorted(sorted_files)
        return [x[-1] for x in sorted_files]


    def get_checkpoint_path(name, filename, fold, folder, th_id=34):
        """ gets best checkpoint path but ignores differences <= `min_delta` and prefers
            models with more epochs if their model id >= `th_id` (to avoid changing
            past models).
        """
        path = f'{folder}/{filename}.ckpt'
        if not os.path.exists(path):
            files = glob.glob(f'{folder}/{filename}_f{fold}*.ckpt')
            scores = [float(x.split('=')[-1][:-5]) if 'val_auc' in x or 'acc' in x else
                      -np.inf for x in files]
            id_ = int(name.split('.')[-1])
            if id_ >= th_id:
                epochs = [float(x.split('=')[1].split('-')[0]) for x in files]
                scores = [score + epoch * 1e-7 for score, epoch in zip(scores, epochs)]
            if len(scores) > 0:
                idx = np.argsort(scores)[-1]
                path = files[idx]
            else:
                print(f"No file found for '{folder}/{filename}_f{fold}' -> {files}")
                path = None
        return path
        

    def load_model_state(model, chkp_path):
        """ load checkpoint into model """
        chkp = torch.load(chkp_path, map_location=torch.device('cpu'))
        model_state = {k[6:]: v for k, v in chkp['state_dict'].items()}  # to remove initial 'model.'
        model.load_state_dict(model_state)


    def average_model_state(chkp_paths, save=None):
        """ average various checkpoints """
        chkp = torch.load(chkp_paths[0], map_location=torch.device('cpu'))
        for chkp_path in chkp_paths[1:]:
            temp_chk = torch.load(chkp_path, map_location=torch.device('cpu'))
            for k in chkp['state_dict']:
                if isinstance(chkp['state_dict'][k], dict):
                    for k2 in chkp['state_dict'][k]:
                        chkp[k][k2] += temp_chk[k][k2]
                else:
                    chkp[k] += temp_chk[k]
        for k in chkp:
            chkp[k] /= len(chkp_paths)
        if save is not None:
            torch.save(chkp)
        return chkp
                

    def load_model_state_average(model, chkp_paths):
        """ load checkpoints, average their weights and load into model """
        chkp = average_model_state(chkp_paths)
        model_state = {k[6:]: v for k, v in chkp['state_dict'].items()}  # to remove initial 'model.'
        model.load_state_dict(model_state)
        

    def load_encoder_state(encoder, chkp_path):
        # chkp_path = f'{ROOT}/b5-train-cola-1-0-0/last_train_f0.ckpt'
        try:
            chkp = torch.load(chkp_path)
        except:
            chkp = torch.load(chkp_path, map_location=torch.device('cpu'))
        
        if 'state_dict' in chkp:
            encoder_state = {k[6:]: v for k, v in chkp['state_dict'].items()}
            encoder_state = {k[6:]: v for k, v in encoder_state.items() if k[:6] == 'model.'}
        else:
            encoder_state = chkp.copy()
            for k, v in encoder_state.items():
                if '.running_mean' in k and torch.isnan(v).any():
                    encoder_state[k] = torch.ones_like(v, dtype=torch.float32) * 0
                if '.running_var' in k and torch.isnan(v).any():
                    encoder_state[k] = torch.ones_like(v, dtype=torch.float32)

        encoder_state = {
             k.replace('backbone.0', 'stem').replace('backbone.1', 'stages')
              .replace('backbone.2', 'final_conv').replace('stem_', 'stem.')
              .replace('stages_', 'stages.').replace('global_pool.', ''): v 
                 for k, v in encoder_state.items()}
        encoder.load_state_dict(encoder_state, strict=STRICT)  # for Vladimir pretrained encoders
        print(f"loaded encoder '{chkp_path}'")
        return encoder_state
        

    def create_models(models_config, half=True, verbose=False):
        models = dict()
        for name, folds, filename, folder, config in models_config:
            if not os.path.exists(folder):
                print(f"folder '{folder}' not found")
                continue
            for fold in folds:
                chkp_path = get_checkpoint_path(name, filename, fold, folder)
                if chkp_path is None:
                    continue
                if verbose:
                    print(f"{name} f{fold} -> '{chkp_path}'")
                cls = globals()[config.model_name] if config.model_name else SpecNetImg
                in_chans = config.in_chans or 3
                out_indices = config.out_indices or None
                model = cls(config, resize=config.resize, add_position=config.add_position,
                            out_indices=out_indices, pretrained=False, in_chans=in_chans)
                load_model_state(model, chkp_path)
                if half:
                    model = model.half().float()
                model.eval()

                models[f'{name}_f{fold}'] = model
        print(f'models: {list(models.keys())}')
        return models


    def save_models_jit(models_config):
        models = create_models(models_config)
        for name, model in models.items():
            model_jit = torch.jit.script(model)
            torch.jit.save(model_jit, f'{name}.pt')
            

    def load_models_jit(models_config):
        models = dict()
        for name, folds, filename, folder, config in models_config:
            for fold in folds:
                name_fold = f'{name}_f{fold}'
                model = torch.jit.load(f'./{name_fold}.pt')
                model.eval()
                # model = torch.jit.optimize_for_inference(torch.jit.script(model.eval()))
                models[name_fold] = model
        return models


    def load_models_onnx(models_config):
        models = dict()
        for name, folds, filename, folder, config in models_config:
            for fold in folds:
                filename = f'{name}_f{fold}'
                onnx_model = onnx.load(f'{folder}/{name}/{filename}.onnx')
                # onnx_model_graph = onnx_model.graph
                onnx_session = ort.InferenceSession(onnx_model.SerializeToString())
                models[filename] = onnx_session
        print(f'models: {list(models.keys())}')
        return models


    def load_model_onnx(name, folder, fold):
        filename = f'{name}_f{fold}'
        onnx_model = onnx.load(f'{folder}/{name}/{filename}.onnx')
        # onnx_model_graph = onnx_model.graph
        onnx_session = ort.InferenceSession(onnx_model.SerializeToString())
        return onnx_session


    def load_models_vino(models_config, verbose=1):
        models = dict()
        for name, folds, filename, folder, config in models_config:
            for fold in folds:
                filename = f'{name}_f{fold}'
                ir_path = f'{ROOT}/b4-models-1-0/{name}/{filename}.xml'
                core = ovrt.Core()
                model = core.read_model(model=ir_path)
                compiled_model = core.compile_model(
                    model=model, device_name='CPU',
                    config={ovrt.properties.inference_num_threads(): 4,
                            ovrt.properties.hint.enable_hyper_threading(): True,
                            ovrt.properties.hint.enable_cpu_pinning(): False})
                # compiled_model = compiled_model.create_infer_request()
                models[filename] = compiled_model
        if verbose:
            print(f'models: {list(models.keys())}')
        return models


    def reshape_spec(spec):
        """ converts batch, chunk, heigh, width to BHW """
        B, C, H, W = spec.size()
        return spec.reshape((B * C, H, W))        


    def predict_fn_single(ds_test, models):
        """ run prediction for `models` using dataset `ds_test`. returns a list[models, batch]
        """
        dl_test = DataLoader(ds_test, batch_size=ARGS.loader_batch_size, num_workers=ARGS.num_workers,
                             # prefetch_factor=ARGS.prefetch_factor, 
                             drop_last=False,
                             pin_memory=True,
                             # multiprocessing_context=get_context('loky')
                            )
        autocast_ = ARGS.autocast
        range_ = tqdm(dl_test) if ARGS.verbose else dl_test
        batch_size = ARGS.batch_size
        preds = []
        for spec in range_:
            #if SPEC_128_512_5 in spec_types:
            B, C, H, W = spec.size()
            n_batch = B * C / batch_size
            spec = spec.reshape((int(n_batch), batch_size, 1, H, W))
            # assert n_batch == int(n_batch)

            # spec = specs[SPEC_224_224B_10]
            # B, C, H, W = spec.size()
            # n_batch = B * C / batch_size
            # spec_224_224b_10 = spec.reshape((int(n_batch), batch_size, 1, H, W))
            
            models_preds = []
            for name, model in models.items():
                batch_out = []
                for batch in spec:
                    with torch.no_grad():
                        out = model(batch)
                    batch_out.append(out)
                batch_out = torch.cat(batch_out, 0)
                #if n_repeat:
                #    batch_out = batch_out.repeat_interleave(n_repeat, dim=1)
                models_preds.append(batch_out)
            preds.append(torch.stack(models_preds))
        return preds


    def predict_fn_regular(ds_test, models):
        """ run prediction for `models` using dataset `ds_test`. returns a list[models, batch]
        """
        dl_test = DataLoader(ds_test, batch_size=ARGS.loader_batch_size, num_workers=ARGS.num_workers,
                             # prefetch_factor=ARGS.prefetch_factor, 
                             drop_last=False,
                             pin_memory=True,
                             # multiprocessing_context=get_context('loky')
                            )
        autocast_ = ARGS.autocast
        range_ = tqdm(dl_test) if ARGS.verbose else dl_test
        batch_size = ARGS.batch_size
        preds = []
        for spec in range_:
            #if SPEC_128_512_5 in spec_types:
            B, C, H, W = spec.size()
            n_batch = B * C / batch_size
            spec = spec.reshape((int(n_batch), batch_size, 1, H, W))
            # assert n_batch == int(n_batch)

            # spec = specs[SPEC_224_224B_10]
            # B, C, H, W = spec.size()
            # n_batch = B * C / batch_size
            # spec_224_224b_10 = spec.reshape((int(n_batch), batch_size, 1, H, W))
            
            models_preds = []
            for name, model in models.items():
                batch_out = []
                for batch in spec:
                    with torch.no_grad():
                        out = model(batch)
                    batch_out.append(out)
                batch_out = torch.cat(batch_out, 0)
                #if n_repeat:
                #    batch_out = batch_out.repeat_interleave(n_repeat, dim=1)
                models_preds.append(batch_out)
            preds.append(torch.stack(models_preds))
        return preds


    def predict_fn_opt(specs):
        """ run prediction for specs using opt models. """
        out = []
        specs = specs.numpy()
        n = math.ceil(specs.shape[0] / bs)
        for model in models.values():
            out_model = []
            for i in tqdm(range(n)):
                y = model.run(['y'], {'x': specs[i*bs : (i+1)*bs]})[0]
                out_model.append(y)
            out_model = np.concatenate(out_model, axis=0)
            out.append(out_model)
        out = np.stack(out, axis=0)
        return out


    def predict_fn_onnx(specs, verbose=False):
        """ run prediction for specs using onnx models. """
        batch_size = ARGS.batch_size
        out = []
        specs = specs.numpy()
        n = math.ceil(specs.shape[0] / batch_size)
        for model in models:
            out_model = []
            range_ = tqdm(range(n)) if verbose else range(n)
            for i in range_:
                y = model.run(['y'], {'x': specs[i*batch_size : (i+1)*batch_size]})[0]
                out_model.append(y)
            out_model = np.concatenate(out_model, axis=0)
            out.append(out_model)
        out = np.stack(out, axis=0)
        return out


    def predict_fn_onnx_timeout(specs):
        """ run prediction for specs using onnx models. """
        deadline = DEADLINE
        stop = False
        batch_size = ARGS.batch_size
        out = []
        specs = specs.numpy()
        n = math.ceil(specs.shape[0] / batch_size)
        
        for model in models:
            out_model = []
            for i in tqdm(range(n)):
                if time.time() >= deadline:
                    stop = True
                    print(f'STOP: deadline breached {time.time() - deadline:,.0f}')
                    break
                y = model.run(['y'], {'x': specs[i*batch_size : (i+1)*batch_size]})[0]
                out_model.append(y)
            out_model = np.concatenate(out_model, axis=0)
            out.append(out_model)
            if stop:
                gap = len(specs) - len(out[-1])
                if gap > 0:
                    out[-1] = np.pad(out[-1], pad_width=((0, gap), (0, 0)), mode='constant', constant_values=np.nan)
                break
        out = np.stack(out, axis=0)
        return out


    def predict_fn_vino(specs):
        """ run prediction for specs using vino models. """
        models = load_models_vino(models_config, verbose=0)
        batch_size = ARGS.batch_size
        out = []
        specs = specs.numpy()
        n = math.ceil(specs.shape[0] / batch_size)
        for model in models.values():
            out_model = []
            for i in tqdm(range(n)):
                y = model([specs[i*batch_size : (i+1)*batch_size]])['y']
                # y = model.infer(inputs=[specs[i*batch_size : (i+1)*batch_size]])['y']
                out_model.append(y)
            out_model = np.concatenate(out_model, axis=0)
            out.append(out_model)
        out = np.stack(out, axis=0)
        return out


    def prepare_predict():
        """ create models """
        if ARGS.engine == 'opt':
            models = load_models_jit(models_config)
            predict_fn = predict_fn_regular
        elif ARGS.engine == 'onnx':
            models = load_models_onnx(models_config)
            predict_fn = predict_fn_onnx
        elif ARGS.engine == 'vino':
            models = None # load_models_vino(models_config)
            predict_fn = predict_fn_vino
        return models, predict_fn


    def split_spec_configs(ws, models_config_dict):
        """ split models based on the melspec parameters """
        spec_configs = dict()
        for name in ws.keys():
            cfg = models_config_dict[name]
            spec = (cfg.spec, cfg.img_dim, cfg.img_duration, cfg.spec_norm, cfg.snipet)
            if spec not in spec_configs:
                spec_configs[spec] = [name]
            else:
                spec_configs[spec].append(name)
        return spec_configs

    # ----------------------------------------------------------------------------------------------------
    # 0.13 end
    # ----------------------------------------------------------------------------------------------------

print('done')

libs torch v4.2
done


In [2]:
if 'SKIP' not in globals() or not SKIP:     # features
    print('features v3.1')
    
    # 3.1: additional files
    # 3.0: h5 data + curated v2
    # 2.5: enhanced filter speech
    # 2.4: filter speech
    # 2.3: speech filter
    # 2.2: update ensemble
    # 2.1: update methods
    # 2.0: initial

    from typing import Any, Dict, List, Optional, Sequence, Tuple, Union, NamedTuple
    import copy
    from contextlib import contextmanager
    import warnings
    warnings.filterwarnings('ignore')
    
    import os
    import sys
    import gc
    import time
    import torch
    import joblib
    import pickle
    import psutil
    import ast
    import copy
    import glob
    import numpy as np
    import pandas as pd
    from tqdm.autonotebook import tqdm
    import multiprocessing as mp
    import concurrent.futures
    from itertools import groupby as grp_by
    from datetime import datetime, timedelta
    import h5py

    from itertools import permutations
    import math

    # import sklearn
    from sklearn.model_selection import KFold, GroupKFold, GroupShuffleSplit, StratifiedKFold, StratifiedGroupKFold
    from sklearn.model_selection._split import _BaseKFold
    from sklearn.utils.validation import _num_samples, check_array
    from sklearn.utils.multiclass import type_of_target
    from sklearn.utils import check_random_state
    # from sklearn.model_selection._split import _BaseKFold, indexable
    
    try:
        import polars as pl
    except:
        pass
        # !pip install polars
        # import polars

    try:
        import pyarrow.parquet as pq
    except:
        pass
        # !pip install pyarrow
        # import pyarrow.parquet as pq

    try:
        import optuna
    except:
        pass
    
    from numba import njit


        # -------------------
        # plot
        # -------------------

    import matplotlib
    import matplotlib.pyplot as plt
    from IPython.display import Audio
    try:
        import seaborn as sns
        import plotly
        from plotly.offline import iplot
        from plotly.subplots import make_subplots
        import plotly.express as px
        import plotly.graph_objects as go

        import ipywidgets as widgets
        from ipywidgets import Layout

        # the following 2 lines resolve issue with charts not showing
        import plotly.io as pio
        pio.renderers.default = "iframe"
    except:
        pass

    try:
        # pydot-ng is a fork of pydot that is better maintained.
        import pydot_ng as pydot
    except ImportError:
        # pydotplus is an improved version of pydot
        try:
            import pydotplus as pydot
        except ImportError:
            # Fall back on pydot if necessary.
            try:
                import pydot
            except ImportError:
                pydot = None

    try:
        import cupy, cudf, cuml
        from cuml.linear_model import LinearRegression
        from cuml.neighbors import NearestNeighbors, KNeighborsRegressor, KNeighborsClassifier
        CUDF = True
    except:
        try:
            from sklearn.linear_model import LinearRegression
            from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor, KNeighborsClassifier
        except:
            pass
        CUDF = False
    from sklearn.cluster import KMeans

    # scipy
    from scipy.stats import linregress
    from scipy.interpolate import interp1d
    from scipy import signal
    from scipy.signal import argrelmax
    from scipy.signal import spectrogram
    from scipy.signal import find_peaks
    from scipy.signal import butter, lfilter
    # import scipy.fft as fft
    from scipy import optimize
    
    # sklearn
    try:
        from sklearn import linear_model
        from sklearn.preprocessing import normalize, QuantileTransformer, MinMaxScaler, RobustScaler, normalize, minmax_scale
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import make_scorer, \
                                    mean_squared_error, mean_absolute_error, r2_score, \
                                    accuracy_score, classification_report, precision_score, recall_score, f1_score, \
                                    roc_auc_score, precision_recall_curve, roc_curve, auc, average_precision_score, confusion_matrix
        from sklearn.decomposition import PCA, TruncatedSVD
    except:
        pass

    # skspatial
    # from skspatial.objects import Line, Points

    import pickle
    import random
    import pdb
    import warnings
    import math
    warnings.filterwarnings('ignore')

    # competition
    import pywt
    import librosa
    
    COLAB = "google.colab" in sys.modules

    # ----------------------------------------------------------------------------------------------------
    # 0.1 utils
    # ----------------------------------------------------------------------------------------------------

    import pickle

    def save_obj(obj, name, protocol=4): # pickle.HIGHEST_PROTOCOL):
        name = name.replace('.pkl', '')
        with open('./'+ name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, protocol)

    def load_obj(name, folder=''):
        name = name.replace('.pkl', '')
        with open(folder + name + '.pkl', 'rb') as f:
            return pickle.load(f)

    def save_obj2(obj, name):
        joblib.dump(obj, './'+ name + '.jl')

    def load_obj2(name, folder=''):
        name = name.replace('.jl', '')
        return joblib.load(folder + name + '.jl')


    @contextmanager
    def timer(name=''):
        s = time.time()
        mem_before = psutil.virtual_memory().used/1024/1024/1024
        yield
        elapsed = time.time() - s
        mem_after = psutil.virtual_memory().used/1024/1024/1024
        print(f'[{name}] {elapsed: .3f} s   {mem_before: .2f} GB -> {mem_after: .2f} GB')


    def split_letters_numbers(text):
        if len(text) == 0:
            return text

        if text[0].isalpha():
            # e.g. 'eb13'
            for index, char in enumerate(text):
                if char.isdigit():
                    return text[:index], text[index:]
            return text, ''
        else:
            # e.g. '19a'
            for index, char in enumerate(text):
                if char.isalpha():
                    return text[:index], text[index:]
            return text, ''
        
    
    def sort_names(names):
        """ sort model names """
        sortable_names = []
        for name in names:
            type, id_ = name.split('.')
            # split letters from numbers in type
            letters, numbers = split_letters_numbers(type)
            type = f'{letters:<6}{numbers:>6}'
            numbers, letters = split_letters_numbers(id_)
            id_ = f'{numbers:>6}{letters:<6}'
            sortable_names.append(f'{type}.{id_}')
            
        sorted_pairs = sorted(zip(sortable_names, names))
        return [x[1] for x in sorted_pairs]


    def argsort_names(names):
        """ sort model names """
        sortable_names = []
        for name in names:
            type, id_ = name.split('.')
            # split letters from numbers in type
            letters, numbers = split_letters_numbers(type)
            type = f'{letters:<6}{numbers:>6}'
            numbers, letters = split_letters_numbers(id_)
            id_ = f'{numbers:>6}{letters:<6}'
            sortable_names.append(f'{type}.{id_}')
            
        idx = np.argsort(sortable_names)
        return idx


    def list_match(str_, list_):
        for x in list_:
            if str_ in x:
                return True
        return False


    def wait():
        for i in range(1000):
            print('.', end='')
            time.sleep(60)


    def is_interactive():
        if COLAB:
            return hasattr(sys, 'ps1')
        else:
            return os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
        

    def sort_external(data, order, reverse=False, return_order=False):
        """ sort list `data` based on the content of list `order` """
        merged = list(zip(data, order))
        sorted_merged = sorted(merged, key=lambda x: x[1], reverse=reverse)
        if return_order:
            return [x[0] for x in sorted_merged], [x[1] for x in sorted_merged]
        else:
            return [x[0] for x in sorted_merged]
        

    def filter_list(l, filter):
        """ filter list l using filter (either a list of bool or indices) """
        if isinstance(filter, bool):
            return [x for x, idx in zip(l, filter) if idx]
        return [x for i, x in enumerate(l) if i in filter]


    def apply_temperature(probs, temp):
        """ apply temperature to probs:
            temp = 2: smothes probs and tends to also pick less likely options (more exploratory)
            temp = 1: same
            temp = .5: sharpens probs and tends to pick most probable (more deterministic)
        """
        scaled_probs = np.exp(np.log(probs) / temp)
        return scaled_probs / np.sum(scaled_probs)


    def apply_temperature_log(log_probs, temp):
        """ apply temperature to log of probs:
            temp = 2: smothes probs and tends to also pick less likely options (more exploratory)
            temp = 1: same
            temp = .5: sharpens probs and tends to pick most probable (more deterministic)
        """
        if not isinstance(log_probs, np.ndarray):
            log_probs = np.array(log_probs)
        scaled_probs = np.exp(log_probs / temp)
        return scaled_probs / np.sum(scaled_probs)    


    def seed_python(seed=1234):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)


    def mySystem(cmd):
        """ run shell command from python """
        print(cmd)
        process = Popen(cmd, stdout=PIPE, stderr=STDOUT, shell=True)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end='')
        assert(process.wait() == 0)


    def fg_format_seconds(seconds):
        seconds = int(seconds)
        minutes = seconds // 60
        remaining_seconds = seconds % 60
        return f"{minutes:02}:{remaining_seconds:02}"


    def random_choice(range_, k):
        """ faster version of random choice without repetition """
        while True:
            choice = random.choices(range_, k=k)
            if len(set(choice)) == k:
                return choice


    def gaussian_kernel(l=5, sig=1.):
        """ create 1D gaussian kernel with side length `l` and a sigma of `sig` """
        ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
        gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
        return gauss / np.sum(gauss)


    def pad_tuples(tuples, pad_value=-1):
        """
        Pads a list of tuples to the size of the longest tuple and returns the padded list,
        with tuples converted to lists, along with the original sizes.

        Args:
            tuples (list of tuples): The list of tuples to pad.
            pad_value (any): The value used for padding.

        Returns:
            tuple: A tuple containing:
                - padded_list (list of lists): The padded tuples as lists.
                - original_sizes (list of int): The original sizes of the tuples.
        """
        # Get the size of the longest tuple
        max_size = max(len(t) for t in tuples)
        
        # Create the padded list and record original sizes
        padded_list = [list(t) + [pad_value] * (max_size - len(t)) for t in tuples]
        original_sizes = [len(t) for t in tuples]
        return padded_list, original_sizes


    def first_smaller_index(lst, th):
        return next((i for i, x in enumerate(lst) if x < th), None)


    def get_scaler(type='quantile'):
         if type == 'std':
            return StandardScaler()
         if type == 'minmax':
            return MinMaxScaler()
         if type == 'max':
            return MaxAbsScaler() 
         if type == 'robust':
            return RobustScaler()  # iqr
         if type == 'norm':
            return Normalizer()
         if type == 'quantile':
            return QuantileTransformer(n_quantiles=100, random_state=0, output_distribution='normal')
         if type == 'power':
            return PowerTransformer()


    def logit(p):
        return np.log(p) - np.log(1 - p)


    def select_uncorrelated_indices(corr_matrix, th_corr=.95):
        """
        Selects indices starting from 0 and adds indices one by one 
        if their correlation with all previously selected indices is below th_corr.

        :param corr_matrix: 2D numpy array (correlation matrix)
        :param th_corr: float (correlation threshold)
        :return: List of selected indices
        """
        n = corr_matrix.shape[0]
        selected = [0]  # Start with the first index

        for i in range(1, n):
            if all(abs(corr_matrix[i, j]) < th_corr for j in selected):
                selected.append(i)

        return selected
           
    # torch

    def print_num_parameters(model, note=None):
        n = sum(p.numel() for p in model.parameters())
        if note:
            print(f'{note:<30}: {n:,.0f}')
        return n

    
    def print_weights(model):
        """ prints model weights and returns them as a dataframe """
        items = []
        for i, (name, module) in enumerate(model.named_modules()):
            weight = module.weight.shape if hasattr(module, 'weight') else ''
            items.append((name, f'{module.__class__.__module__}.{module.__class__.__name__}', weight))

        res = pd.DataFrame(items, columns=['name', 'class', 'weight'])
        res.to_csv('x3d_l.csv')
        pd.set_option('display.max_rows', None)
        display(res)
        return res

    # ----------------------------------------------------------------------------------------------------
    # 0.2 target
    # ----------------------------------------------------------------------------------------------------

    def get_pseudo_labels(coef, folder='/kaggle/input/ba-models-2/pseudo'):
        """ load and average pseuso labels """
        if isinstance(coef, list):
            coef = {name: 1 for name in coef}
        names = list(coef.keys())
        oof = []
        for name in names:
            file = f'{folder}/{name}/oof {name}.pkl'
            if not os.path.exists(file):
                file = f'{folder}/oof {name}.pkl'
            oof.append(load_obj(file))
        oof = np.stack(oof, axis=0)
        pseudo_labels = ensemble_oof(coef, oof, names)
        return pseudo_labels


    def expand_labels(labels, sizes):
        """ expand labels per file such that each label is repeated `sizes` """
        expanded = []
        for l, n in zip(labels, sizes):
            expanded += [l] * n
        return np.stack(expanded, axis=0)

    # ----------------------------------------------------------------------------------------------------
    # 0.3 meta
    # ----------------------------------------------------------------------------------------------------

    def get_audio_dims(folder='train_audio'):
        def _process(file):
            audio, sr = torchaudio.load(f'/kaggle/input/birdclef-2025/{folder}/{file}')
            len_ = audio.shape[1] / SR
            return len_
        
        out = Parallel(n_jobs=-1)(delayed(_process)(file) for file in tqdm(meta.filename.values))
        audios = meta[['filename']]
        audios['dim'] = out
        audios.to_csv('len_train_audios.csv')
        return audios


    def print_dup():
        metak = meta.set_index('filename').to_dict(orient='index')
        dup = []
        for a, b in DUPLICATED_FILES:
            l1, l2 = a.split('/')[0], b.split('/')[0]
            is_same = l1 == l2
            dup.append((*metak[a].values(), *metak[b].values(), is_same))
        dup = pd.DataFrame(dup, columns=['lat_a', 'long_a', 'label_a', 'lat_b', 'long_b', 'label_b', 'same'])
        dup.to_csv('duplicates')


    def read_meta(split_author=True, pivot_folds=False):
        file = f'{ROOT}/birdclef-2025/train.csv'
        meta = pd.read_csv(file).sort_values(['filename'])
        meta['i'] = np.arange(meta.shape[0])
        authors = meta.author.unique()
        author_to_num = dict(zip(authors, range(len(authors))))
        meta['author'] = meta.author.map(author_to_num)
        
        # split author in case there is only one per primary_label
        if split_author:
            meta['count_unique_author'] = meta.groupby('primary_label')['author'].transform('nunique')
            meta['order'] = meta.groupby('primary_label').cumcount()
            meta['author'] = np.where(meta.count_unique_author == 1, meta.author + meta.order / 1000, meta.author)

        meta = meta[['i', 'primary_label', 'secondary_labels', 'author', 'latitude', 'longitude', 'rating', 'filename']]
        
        # merge labels
        labels = []
        for row in meta.to_dict(orient='records'):
            row = dotdict(row)
            secondary = row.secondary_labels[1:-1].replace("'", '')
            secondary = secondary.split(',') if len(secondary) > 0 else []
            secondary = [x.replace(' ', '') for x in secondary]
            labels.append([row.primary_label] + secondary)
        meta['labels'] = labels
        meta['n_labels'] = [len(x) for x in labels]
        meta = meta.drop(['secondary_labels'], axis='columns')
        
        # save pivot of folds
        if pivot_folds:
            meta.pivot_table(index='primary_label', columns='fold', values='filename',
                             aggfunc='count').to_csv('folds.csv')
        return meta
        

    def build_previus_meta():
        """ build meta for previous files. """
        previous_lens = pd.read_csv('/kaggle/input/b5-data-2021-to-2025/len_previous_audios')
        previous = pd.read_csv('/kaggle/input/b5-data-2021-to-2025/previous.csv')
        cols = ['primary_label', 'secondary_labels', 'record_name', 'author', 'latitude', 'longitude']
        meta = previous[cols]
        filenames = [f'{primary_label}/{record_name}' for primary_label, record_name in \
                         zip(meta.primary_label, meta.record_name)]
        meta['filename'] = filenames
        files_exist = [os.path.exists(f'/kaggle/input/b5-data-2021-to-2025/{x}') for x in tqdm(filenames)]
        meta = meta[files_exist]
        
        # merge labels
        labels = []
        for row in meta.to_dict(orient='records'):
            row = dotdict(row)
            secondary = row.secondary_labels[1:-1].replace("'", '')
            secondary = secondary.split(',') if len(secondary) > 0 else []
            secondary = [x.replace(' ', '') for x in secondary]
            labels.append([row.primary_label] + secondary)
        meta['labels'] = labels
        meta['n_labels'] = [len(x) for x in labels]
        
        if 'rating' not in meta.columns:
            meta['rating'] = 5
        meta['i'] = np.arange(50_000, 50_000 + len(meta))
        meta['fold'] = -2
        meta = meta.merge(previous_lens, on='filename', how='left')
        meta['filename'] = meta.filename.astype(str) + '.ogg'
        
        meta_cols = ['i', 'primary_label', 'author', 'latitude', 'longitude', 'rating',
               'filename', 'labels', 'n_labels', 'dim', 'fold']
        meta = meta[meta_cols]
        return meta
        

    def build_meta_add(folder_base, previous_filenames):
        meta_add = pd.read_csv(f'{folder_base}/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5.csv')
        filter_new = (meta_add.dataset != 'comp_2025') & ~meta_add.filename.isin(previous_filenames)
        meta_add = meta_add[filter_new]
        meta_add['fold'] = -2
        meta_add['dim'] = meta_add.duration_s
        
        # merge labels and build list of files
        labels, files = [], []
        for row in meta_add.to_dict(orient='records'):
            row = dotdict(row)
            secondary = row.secondary_labels[1:-1].replace("'", '')
            secondary = secondary.split(',') if len(secondary) > 0 else []
            secondary = [x.replace(' ', '') for x in secondary]
            secondary = [x for x in secondary if x in BIRDS]
            labels.append([row.primary_label] + secondary)
            files.append(f'{folder_base}/{row.data_root_id}/{row.data_root_id}/{row.filename}')
        
        meta_add['filename'] = [x.replace('.mp3', '.ogg') for x in meta_add.filename.values]
        meta_add['labels'] = labels
        meta_add['n_labels'] = [len(x) for x in labels]
        cols = ['primary_label', 'author', 'latitude', 'longitude', 'rating', 'filename', 'labels', 'n_labels', 'dim', 'fold']
        meta_add = meta_add[cols]
        return meta_add, files


    def get_series_mode(x):
        counts = x.value_counts()
        return counts.index[0] if len(counts) > 0 else None


    def add_geo_label(meta, n=60, verbose=False):
        """ generate `n` clusters of geo-coordinates and add them as a geo_label.
            Fill records with missing geo-coordinates using existing records in the following order:
            - same author and primary
            - same author
            - same primary
        """
        n = 60
        geo = meta.loc[~meta.latitude.isnull(), ['i', 'latitude', 'longitude']]
        model = KMeans(n_clusters=n, max_iter=300, random_state=42)
        model.fit(geo[['latitude', 'longitude']])
        centers = model.cluster_centers_
        i_to_geo_label = dict(zip(geo.i.values, model.labels_))
        meta['geo_label'] = meta.i.map(i_to_geo_label)
        meta['mode_primary'] = meta.groupby('primary_label')['geo_label'].transform(get_series_mode)
        meta['mode_author'] = meta.groupby('author')['geo_label'].transform(get_series_mode)
        meta['mode_author_primary'] = meta.groupby(['author', 'primary_label'])['geo_label'].transform(get_series_mode)
        f = meta.geo_label.isnull()
        if verbose: print('null coordinates', f.sum())
        meta.loc[f, 'geo_label'] = meta.loc[f, 'mode_author_primary']
        f = meta.geo_label.isnull()
        if verbose: print('null coordinates', f.sum())
        meta.loc[f, 'geo_label'] = meta.loc[f, 'mode_author']
        f = meta.geo_label.isnull()
        if verbose: print('null coordinates', f.sum())
        meta.loc[f, 'geo_label'] = meta.loc[f, 'mode_primary']
        f = meta.geo_label.isnull()
        if verbose: print('null coordinates', f.sum())
        meta['geo_label'] = meta.geo_label.astype('int16')
        meta = meta.drop(labels=['mode_author_primary', 'mode_author', 'mode_primary'], axis=1)
        return meta


    def get_meta(geo_label=0, split_author=False):
        meta = read_meta(split_author)
        audios = pd.read_csv(f'{ROOT}/b5-cache/len_train_audios.csv')
        audios.columns = ['filename', 'dim']
        meta = meta.merge(audios)
        if geo_label:
            meta = add_geo_label(meta, n=geo_label)
        return meta


    def filter_meta(meta, filters):
        filter_secondary, min_rating = filters
        if filter_secondary:
            meta = meta[meta.n_labels == 1]
        #if min_rating > 0:
        #    meta = meta[meta.rating >= min_rating]
        return meta

    
    def single_edge_files(files):
        """ takes a list of edge files and leaves only the end (if there is one) or teh start otherwise """
        if files[0][-1] not in ['s', 'e']:
            return files
        
        single = []
        for f in files:
            if f[-1] != 's' or f[:-1]+'e' in files:
                single.append(f)
        return np.array(single)
                

    def get_meta_previous(folder='raw-30s'):
        previous_files = glob.glob(f'{ROOT}/b5-data-previous-{folder}/*/*.ogg*')
        previous_files = single_edge_files(previous_files)
        primary_label = [x.split('/')[-2] for x in previous_files]
        meta_previous = pd.DataFrame(dict(primary_label=primary_label, filename=previous_files))
        meta_previous['labels'] = [[x] for x in primary_label]
        meta_previous['rating'] = 5
        meta_previous['n_labels'] = 1
        meta_previous['fold'] = -1 # only use for training
        return meta_previous


    def get_meta_previous_from_source():
        previous_files = glob.glob(f'{ROOT}/audio-bc-212223/AUDIO/*/*.ogg*')
        previous_label = [x.split('/')[-2] for x in previous_files]
        meta_previous = pd.DataFrame(dict(
            primary_label=previous_label, 
            filename=previous_files,
            dim=len_previous))
        meta_previous['labels'] = [[x] for x in previous_label]
        meta_previous['rating'] = 5
        meta_previous['n_labels'] = 1
        meta_previous['fold'] = -1 # only use for training
        return meta_previous


    def get_labels_primary(meta, num_classes=N_BIRDS):
        """ get one_hot encoded labels for primary class only """
        labels = meta.primary_label.map(label_to_num).values.astype(np.int64)
        labels = np.eye(num_classes)[labels].astype(np.float32)
        return labels


    def get_labels(meta, num_classes=N_BIRDS):
        """ get one_hot encoded labels for all classes """
        labels_str = meta.labels
        labels = np.zeros((meta.shape[0], num_classes), dtype=np.float32)
        for i, ls in enumerate(labels_str):
            for l in ls:
                idx = label_to_num.get(l, -1)
                if idx >= 0:
                    labels[i, idx] = 1.
        return labels
    

    def memory_usage_cap(meta, cap_seconds=30, dim=224):
        """ Memory needed to old spectrograms with size (dim,dim) if capped at `cap_seconds` """
        caped_seconds = np.where(meta.dim < sr*cap_seconds, meta.dim/sr, cap_seconds)
        size = caped_seconds.sum() / 5 * dim*dim * 4 / 1e9
        print(f'<={cap_seconds}s: {size:.1f}MB')


    def get_bird_groups():
        """ group birds that show together in an audio """
        bird_to_group = dict()
        next_id = 0
        groups = dict()
        for birds in meta.labels.values:
            g = set()

            # check for existing groups
            for bird in birds:
                if bird in bird_to_group:
                    g.add(bird_to_group[bird])

            if len(g) == 0:  # new group
                gid = next_id
                next_id += 1
                groups[gid] = set()
            elif len(g) == 1:  # existing group
                gid = next(iter(g))
            if len(g) > 1:  # merge groups
                gid = next_id
                next_id += 1
                new_group = set()
                for g_ in g:
                    new_group = new_group.union(groups[g_])
                    for b in groups[g_]:
                        bird_to_group[b] = gid
                    del groups[g_]
                groups[gid] = new_group

            # set group for each bird
            for bird in birds:
                bird_to_group[bird] = gid
                groups[gid].add(bird)
            # print(gid, birds, groups[gid])

        return groups

    # from https://www.kaggle.com/code/robbynevels/bc24-duplicate-audio-files/

    def set_dataset(files, config):
        new_files = []
        for x in files:
            folders = x.split('/')
            if 'unlabeled' not in folders[3] and 'previous' not in folders[3]:
                folders[3] = f'b5-data-{config.dataset}'
            new_files.append('/'.join(folders))
        return np.array(new_files)


    # ----------------------------------------------------------------------------------------------------
    # 0.5 L2
    # ----------------------------------------------------------------------------------------------------

    def build_bird_catalog(meta, min_duration):
        """ build dict of lists of audios per bird. Each entry of the list contains the
            filename, size of the audio and position in meta (this must be unique across all folds 
            as it's used to index all files and all labels):
            - bird_catalog[bird_idx] = [(name, size, i)*].
            audios muts have a minimum duration of `min_duration` seconds.
        """
        bird_catalog = {k: [] for k in RANGE_BIRDS}
        min_size = min_duration * 32_000
        for row in meta.to_dict(orient='records'):
            row = dotdict(row)
            if row.dim < min_size:
                continue
            bird_idx = label_to_num[row.primary_label]
            filename = row.filename
            bird_catalog[bird_idx].append((filename, row.dim, int(row.i)))
        return bird_catalog
        

    def get_usable_birds(bird_catalog, min_count=20, min_total_duration=4*4*60, verbose=True):
        """ takes a bird_catalog dict and returns two arrays per bird:
            birds_aux: birds with qualifying audio
            birds_main: birds with at least `min_count` audios and whose total duration of 
                all audios adds to `min_total_duration` seconds.
        """
        birds_aux, birds_main = set(), set()
        for bird, audios in bird_catalog.items():
            if len(audios) > 0:
                birds_aux.add(bird)
                total_duration = sum((x[1] for x in audios))
                if len(audios) > min_count and total_duration > min_total_duration:
                    birds_main.add(bird)
        birds_aux = list(birds_aux)
        birds_main = list(birds_main)
        if verbose:
            print(f'birds aux: {len(birds_aux)} - main: {len(birds_main)}')
        return birds_aux, birds_main
    

    # ----------------------------------------------------------------------------------------------------
    # 0.5 samples
    # ----------------------------------------------------------------------------------------------------

    class GroupKFoldShuffle(_BaseKFold):
        """ random shuffle in K folds with unique groups in each fold.
            Attempts to equalize the size of each fold as follows: groups are shuffled 
            and one at a time placed in the fold with the smallest size.
        """
       
        def __init__(self, n_splits=5, *, random_state=None):
            super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state)

        def split(self, x, groups=None):
            # assign each group to a fold
            unique_groups, counts = np.unique(groups, return_counts=True)
            group_counter = {g: c for g, c in zip(unique_groups, counts)}
            rng = np.random.default_rng(seed=self.random_state)
            unique_groups = rng.permutation(unique_groups)
            groups_per_fold = [[] for x in range(self.n_splits)]
            fold_size = [0] * self.n_splits
            for g in unique_groups:
                fold = np.argmin(fold_size)
                groups_per_fold[fold].append(g)
                fold_size[fold] += group_counter[g]
            
            # return fold indices
            for test_group_ids in groups_per_fold:
                test_mask = np.isin(groups, test_group_ids)
                # train_mask = ~test_mask
                # train_idx = np.where(train_mask)[0]
                train_idx = None
                test_idx = np.where(test_mask)[0]
                yield train_idx, test_idx

        def test(self, meta):
            """ """
            df = meta.groupby(['patient_id', 'eeg_id'])['label_id'].count().reset_index()
            x = df.index
            g = df.patient_id.values
            targets = []
            ref = []
            for i in range(20):
                kf = GroupKFoldShuffle(n_splits=5, random_state=i)
                l, idxs = [], []
                for j, (train_index, test_index) in enumerate(kf.split(x, groups=g)):
                    if len(set(g[train_index]).intersection(set(g[test_index]))) > 0:
                        print('error')
                    l.append(len(test_index))
                    idxs.append(test_index)
                diff = max(l) - min(l)
                if i == 0:
                    ref = idxs
                else:
                    for i1 in range(5):
                        print('\t', i1, end=':  ')
                        for i2 in range(5):
                            print(f'{len(set(idxs[i1]).intersection(set(ref[i2])))*100/len(idxs[i1]):.1f}', end='  ')
                        print()
                if diff < len(x)*.02:
                    targets.append((i, idxs))
                print(i, diff, f'{diff/len(x):.4f}')


    def add_folds(meta, random_state=42, n_folds=5, verbose=0):
        """ simple split. Return folds """
        if n_folds is None:
            n_folds = config.n_folds
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)
        index = meta.index
        folds = np.arange(len(index))
        for fold, (_, val_idx) in enumerate(kf.split(index)):
            folds[val_idx] = fold
        meta['fold'] = folds
    

    def add_group_folds(meta, group_col, random_state=42, n_folds=5, verbose=0):
        """ split per group return folds as a dict """
        kf = GroupKFoldShuffle(n_splits=n_folds, random_state=random_state)
        index = meta.index
        groups = meta[group_col].values
        folds = pd.Series(0, index=index)
        for fold, (_, val_idx) in enumerate(kf.split(index, groups=groups)):
            folds.iloc[val_idx] = fold
        meta['fold'] = folds
        # return folds.to_dict()


    def add_stratified_folds(meta, strat_col, random_state=42, n_folds=5, verbose=0):
        """ split per strat """
        skf = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=True)
        strats = meta[strat_col]
        index = np.arange(meta.shape[0])

        folds = pd.Series(0, index=index)
        for fold, (_, val_idx) in enumerate(skf.split(index, strats)):
            folds.iloc[val_idx] = fold
        meta['fold'] = folds
            
        if verbose:
            values_col = 'ID'  # column that has no null values
            labels_per_fold = meta.pivot_table(
                index=strat_col, columns='fold', values=values_col, aggfunc=['count'])
            print(labels_per_fold)
            # labels_per_fold.to_csv('labels_per_fold.csv')
            # labels_per_fold[labels_per_fold.isnull().any(axis=1)]

   
    def add_folds_strat(meta, strat_col='primary_label', group_col='author', seed=42, n_folds=5, verbose=0):
        """ add folds to meta using stratification (split equally between folds) and groups
            (each group in a single fold). Use add_mlstratified_folds because it's better.
        """

        def _get_folds_strat(index, groups, strats, random_state=42, n_folds=5):
            if n_folds is None:
                n_folds = config.n_folds
            sgkf = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
            folds = pd.Series(0, index=index)
            for fold, (_, val_idx) in enumerate(sgkf.split(index, y=strats, groups=groups)):
                folds.iloc[val_idx] = fold
            return folds

        groups = meta[group_col] if group_col is not None else None
        strats = meta[strat_col]
        index = np.arange(meta.shape[0])
        meta['fold'] = _get_folds_strat(index, groups, strats, random_state=seed, n_folds=n_folds)
        
        if verbose:
            print(meta.fold.value_counts().sort_index())

        # return folds


    def IterativeStratification(labels, r, random_state):
        """This function implements the Iterative Stratification algorithm described
        in the following paper:
        Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
        Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
        (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
        2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
        Heidelberg.
        """

        n_samples = labels.shape[0]
        test_folds = np.zeros(n_samples, dtype=int)

        # Calculate the desired number of examples at each subset
        c_folds = r * n_samples

        # Calculate the desired number of examples of each label at each subset
        c_folds_labels = np.outer(r, labels.sum(axis=0))

        labels_not_processed_mask = np.ones(n_samples, dtype=bool)

        while np.any(labels_not_processed_mask):
            # Find the label with the fewest (but at least one) remaining examples,
            # breaking ties randomly
            num_labels = labels[labels_not_processed_mask].sum(axis=0)

            # Handle case where only all-zero labels are left by distributing
            # across all folds as evenly as possible (not in original algorithm but
            # mentioned in the text). (By handling this case separately, some
            # code redundancy is introduced; however, this approach allows for
            # decreased execution time when there are a relatively large number
            # of all-zero labels.)
            if num_labels.sum() == 0:
                sample_idxs = np.where(labels_not_processed_mask)[0]

                for sample_idx in sample_idxs:
                    fold_idx = np.where(c_folds == c_folds.max())[0]

                    if fold_idx.shape[0] > 1:
                        fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                    test_folds[sample_idx] = fold_idx
                    c_folds[fold_idx] -= 1

                break

            label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
            if label_idx.shape[0] > 1:
                label_idx = label_idx[random_state.choice(label_idx.shape[0])]

            sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

            for sample_idx in sample_idxs:
                # Find the subset(s) with the largest number of desired examples
                # for this label, breaking ties by considering the largest number
                # of desired examples, breaking further ties randomly
                label_folds = c_folds_labels[:, label_idx]
                fold_idx = np.where(label_folds == label_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    temp_fold_idx = np.where(c_folds[fold_idx] ==
                                             c_folds[fold_idx].max())[0]
                    fold_idx = fold_idx[temp_fold_idx]

                    if temp_fold_idx.shape[0] > 1:
                        fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                labels_not_processed_mask[sample_idx] = False

                # Update desired number of examples
                c_folds_labels[fold_idx, labels[sample_idx]] -= 1
                c_folds[fold_idx] -= 1

        return test_folds
        
    
    class MultilabelStratifiedKFold(_BaseKFold):
        """Multilabel stratified K-Folds cross-validator
        Provides train/test indices to split multilabel data into train/test sets.
        This cross-validation object is a variation of KFold that returns
        stratified folds for multilabel data. The folds are made by preserving
        the percentage of samples for each label.
        Parameters
        ----------
        n_splits : int, default=3
            Number of folds. Must be at least 2.
        shuffle : boolean, optional
            Whether to shuffle each stratification of the data before splitting
            into batches.
        random_state : int, RandomState instance or None, optional, default=None
            If int, random_state is the seed used by the random number generator;
            If RandomState instance, random_state is the random number generator;
            If None, the random number generator is the RandomState instance used
            by `np.random`. Unlike StratifiedKFold that only uses random_state
            when ``shuffle`` == True, this multilabel implementation
            always uses the random_state since the iterative stratification
            algorithm breaks ties randomly.
        Examples
        --------
        >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
        >>> import numpy as np
        >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
        >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
        >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
        >>> mskf.get_n_splits(X, y)
        2
        >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
        MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
        >>> for train_index, test_index in mskf.split(X, y):
        ...    print("TRAIN:", train_index, "TEST:", test_index)
        ...    X_train, X_test = X[train_index], X[test_index]
        ...    y_train, y_test = y[train_index], y[test_index]
        TRAIN: [0 3 4 6] TEST: [1 2 5 7]
        TRAIN: [1 2 5 7] TEST: [0 3 4 6]
        Notes
        -----
        Train and test sizes may be slightly different in each fold.
        See also
        --------
        RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
        n times.
        """

        def __init__(self, n_splits=3, shuffle=False, random_state=None):
            super(MultilabelStratifiedKFold, self).__init__(n_splits, shuffle=shuffle, random_state=random_state)

        def _make_test_folds(self, X, y):
            y = np.asarray(y, dtype=bool)
            type_of_target_y = type_of_target(y)

            if type_of_target_y != 'multilabel-indicator':
                raise ValueError(
                    'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

            num_samples = y.shape[0]

            rng = check_random_state(self.random_state)
            indices = np.arange(num_samples)

            if self.shuffle:
                rng.shuffle(indices)
                y = y[indices]

            r = np.asarray([1 / self.n_splits] * self.n_splits)

            test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

            return test_folds[np.argsort(indices)]

        def _iter_test_masks(self, X=None, y=None, groups=None):
            test_folds = self._make_test_folds(X, y)
            for i in range(self.n_splits):
                yield test_folds == i

        def split(self, X, y, groups=None):
            """Generate indices to split data into training and test set.
            Parameters
            ----------
            X : array-like, shape (n_samples, n_features)
                Training data, where n_samples is the number of samples
                and n_features is the number of features.
                Note that providing ``y`` is sufficient to generate the splits and
                hence ``np.zeros(n_samples)`` may be used as a placeholder for
                ``X`` instead of actual training data.
            y : array-like, shape (n_samples, n_labels)
                The target variable for supervised learning problems.
                Multilabel stratification is done based on the y labels.
            groups : object
                Always ignored, exists for compatibility.
            Returns
            -------
            train : ndarray
                The training set indices for that split.
            test : ndarray
                The testing set indices for that split.
            Notes
            -----
            Randomized CV splitters may return different results for each call of
            split. You can make the results identical by setting ``random_state``
            to an integer.
            """
            y = check_array(y, ensure_2d=False, dtype=None)
            return super(MultilabelStratifiedKFold, self).split(X, y, groups)

        
    def add_mlstratified_folds(meta, n_folds=5, group='author', random_state=42, verbose=True):
        """ splits instances of score equally while keeping all instances of each anchor in the same fold """
        group_col = group
        strat_col = 'primary_label'
        dfx = pd.get_dummies(meta, columns=[strat_col]).groupby([group_col], as_index=False).sum()
        cols = [c for c in dfx.columns if c.startswith('primary_label_') or c == group_col]
        dfx = dfx[cols]

        mskf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
        labels = [c for c in dfx.columns if c != group_col]
        dfx_labels = dfx[labels]
        dfx['fold'] = -1

        for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
            dfx.loc[val_, 'fold'] = fold

        if 'fold' in meta:
            del meta['fold']
        meta = meta.merge(dfx[[group_col, 'fold']], on=group_col, how='left')
        if verbose:
            print(meta.fold.value_counts().sort_index())
            if verbose == 2:
                labels_per_fold = meta.pivot_table(index='primary_label', columns='fold', values='author', aggfunc=['count'])
                print(labels_per_fold)
                labels_per_fold.to_csv('labels_per_fold.csv')
                # labels_per_fold[labels_per_fold.isnull().any(axis=1)]
        return meta
        
    
    def adjust_fold_small_samples(meta, th=10, fold=0):
        """ moves small samples from fold `fold` to fold -1 """
        count_labels = meta.groupby('primary_label')['primary_label'].transform('count')
        f_small = count_labels < th
        meta.loc[f_small, 'fold'] = -1
    
    # ----------------------------------------------------------------------------------------------------
    # 0.4 audios
    # ----------------------------------------------------------------------------------------------------

    def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=5):
        nyquist = 0.5 * sampling_rate
        normal_cutoff = cutoff_freq / nyquist  # to express cutoff in range 0..1 = cutoff * 2 / sampling_rate
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        filtered_data = lfilter(b, a, data, axis=-1)
        return filtered_data


    def maddest(d, axis=None):
        return np.mean(np.absolute(d - np.mean(d, axis)), axis)


    def wavelet_denoise(x, wavelet='haar', level=1):
        """ set `wavelet` to the desired mode, e.g, db8, db2, dmey. """
        coeff = pywt.wavedec(x, wavelet, mode="per")
        sigma = (1/0.6745) * maddest(coeff[-level])

        uthresh = sigma * np.sqrt(2*np.log(len(x)))
        coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

        ret = pywt.waverec(coeff, wavelet, mode='per')
        return ret


    def butter_bandpass_filter(data, lowcut=.5, highcut=20, fs=200, order=5):
        b, a = butter(order, [lowcut, highcut], fs=fs, btype='band')
        y = lfilter(b, a, data, axis=-1)
        return y


    def read_audios(meta, parallel=True):
        folder = f'{ROOT}/birdclef-2025/train_audio'
        
        def _process(file):
            # audio_data, sr = librosa.load(f'{folder}/{file}', sr=None)
            audio_data, sr = torchaudio.load(file)
            f_nan = np.isnan(audio_data)
            if f_nan.mean() < 1:
                m = np.nanmean(val)
                audio_data[f_nan] = m
            else:
                audio_data[:] = 0
            return audio_data.astype(np.float32)
        
        files = meta.filename.values
        if parallel:
            data = Parallel(n_jobs=-1)(delayed(_process)(file) for file in tqdm(files))
        else:
            data = []
            for file in tqdm(files):
                data.append(_process(file))

        audios = dict()
        for file, audio in zip(files, data):
            audios[file] = audio
        return audios

    # ----------------------------------------------------------------------------------------------------
    # 0.4 raw audio
    # ----------------------------------------------------------------------------------------------------

    import soundfile as sf

    def build_raw(files, duration=5, filter=None, parallel=True):
        """ read raw audios, limits size and saves as file """

        _chunk_size = 5*32_000

        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            audio = audio[:1, :sr*duration]
            path = file.split('/')
            label, filename = path[-2:]
            if not os.path.exists(label):
                try:
                    os.makedirs(label)
                except:
                    pass
            if filter is not None:
                name = '/'.join(file.split('/')[-2:]).split('.')[0]
                start, end = filter(name, audio)
                if start is not None:
                    audio = audio[:, start : end]
            
            audio = audio[:1].numpy().T
            # following doesn't work for files > 60*32_000 and is slower otherwise
            # sf.write(f'{label}/{filename}', audio.numpy().T, sr, format='ogg', subtype='vorbis') # or
            # torchaudio.save('test.ogg', audios, format='ogg', sample_rate=sr, backend='soundfile')
            if filename[-4:] != '.ogg':
                filename += '.ogg'
            with sf.SoundFile(f'{label}/{filename}', mode='w', samplerate=sr,
                              channels=1, format='ogg', subtype='vorbis') as file:
                for start in range(0, audio.shape[0], _chunk_size):
                    end = start + _chunk_size
                    file.write(audio[start:end])

        if parallel:
            Parallel(n_jobs=-1)(delayed(_process)(file) for file in tqdm(files))
        else:
            for file in tqdm(files):
                _process(file)


    def build_raw_edges(files, duration=7, min_duration=0, filter=None, parallel=True):
        """ read raw audios, limits size and saves as file """
        
        _chunk_size = 5*32_000
        
        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            len_ = audio.shape[-1]
            if min_duration and len_ < min_duration * sr:
                return
            path = file.split('/')
            label, filename = path[-2:]
            if not os.path.exists(label):
                try:
                    os.makedirs(label)
                except:
                    pass
            if filter is not None:
                name = '/'.join(file.split('/')[-2:]).split('.')[0]
                start, end = filter(name, audio)
                if start is not None:
                    audio = audio[:, start : end]
                    # print(name, raw_start/SR, raw_end/SR, audio.shape)
            else:
                start, end = None, None

            audio = audio[:1].numpy().T
            audios = audio[:sr*duration]
            # following doesn't work for files > 60*32_000 and is slower otherwise
            # sf.write(f'{label}/{filename}_s', audios, sr, format='ogg', subtype='vorbis')
            if filename[-4:] != '.ogg':
                filename += '.ogg'
            with sf.SoundFile(f'{label}/{filename}_s', mode='w', samplerate=sr,
                              channels=1, format='ogg', subtype='vorbis') as file:
                for s in range(0, audios.shape[0], _chunk_size):
                    e = s + _chunk_size
                    file.write(audios[s : e])
            
            if audio.shape[0] > duration * sr:
                audioe = audio[-sr*duration:]
                # print('audioe', duration, audioe.shape)
                # sf.write(f'{label}/{filename}_e', audioe, sr, format='ogg', subtype='vorbis')
                with sf.SoundFile(f'{label}/{filename}_e', mode='w', samplerate=sr,
                                  channels=1, format='ogg', subtype='vorbis') as file:
                    for s in range(0, audioe.shape[0], _chunk_size):
                        e = s + _chunk_size
                        file.write(audioe[s : e])
            
            return start if start !=0 else None, end if end != len_ else None

        if parallel:
            edges = Parallel(n_jobs=-1)(delayed(_process)(file) for file in tqdm(files))
        else:
            edges = []
            for file in tqdm(files):
                edge = _process(file)
                edges.append(edge)
        starts, ends = zip(*edges)
        files = ['/'.join(file.split('/')[-2:]) for file in files]
        edges_df = pd.DataFrame(dict(start=starts, end=ends), index=files)
        edges_df.to_csv('edges.csv')

    
    def load_audios(files, duration=5, parallel=True):
        """ read raw audios, limits size and saves as file """

        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if duration is not None:
                audio[:, :sr*duration]
            return audio

        if parallel:
            audios = Parallel(n_jobs=-1)(delayed(_process)(file) for file in tqdm(files))
        else:
            for file in tqdm(files):
                _process(file)
        return audios


    def size_audios(files, parallel=True):
        """ read raw audios, limits size and saves as file """

        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            #if sr != 32_000:
            #    print(file)
            return audio.shape[-1]

        if parallel:
            sizes = Parallel(n_jobs=-1)(delayed(_process)(file) for file in tqdm(files))
        else:
            for file in tqdm(files):
                _process(file)
        return sizes


    def build_unlabeled(meta, duration=5, parallel=True):
        """ read raw audios, limits size and saves as file """

        _chunk_size = 5*32_000

        def _process(row):
            """ returns single img """
            row = dotdict(row)
            audio_file = f'/kaggle/input/birdclef-2025/train_soundscapes/{row.audio}.ogg'
            audio, sr = torchaudio.load(audio_file)
            len_ = audio.shape[1]
            if row.order == 0:  # first 5s
                start = 0
            elif row.order == 11:  # last 5s
                start = len_ - sr * duration
            else:
                band = (duration - 5) // 2
                start = (row.order - band) * sr
            audio = audio[:1, start : start + sr * duration]
            folder = row.primary_label
            if not os.path.exists(folder):
                try:
                    os.makedirs(folder)
                except:
                    pass
            
            audio = audio.numpy().T
            # following doesn't work for files > 60*32_000 and is slower otherwise
            # sf.write(f'{label}/{filename}', audio.numpy().T, sr, format='ogg', subtype='vorbis') # or
            # torchaudio.save('test.ogg', audios, format='ogg', sample_rate=sr, backend='soundfile')
            with sf.SoundFile(f'{row.filename}', mode='w', samplerate=sr,
                              channels=1, format='ogg', subtype='vorbis') as file:
                for start in range(0, audio.shape[0], _chunk_size):
                    end = start + _chunk_size
                    file.write(audio[start:end])

        rows = meta.to_dict(orient='records')
        if parallel:
            Parallel(n_jobs=-1)(delayed(_process)(row) for row in tqdm(rows))
        else:
            for row in tqdm(rows):
                _process(row)

    
    # ----------------------------------------------------------------------------------------------------
    # 0.5 h5 audio
    # ----------------------------------------------------------------------------------------------------
    
    def build_h5_files(files, folder, custom_filter={}, speech_filter=None):
        """ save audio in """
        
        files_edges = dict()
        for file in tqdm(files):
            """ returns single img """
            path = f'{folder}/{file}' if folder is not None else file
            audio, sr = torchaudio.load(path)
            if sr != SR:
                resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SR)
                audio = resampler(audio)
            audio = audio[0]
            id_ = '/'.join(file.split('/')[-2:]).split('.')[0]
            sections = custom_filter.get(id_, None)
            if sections is not None:
                start = int(sections[0][0] * SR)
                end = min(int(sections[-1][1] * SR), len(audio))
            elif speech_filter is not None:
                start, end = speech_filter(audio)
                end = min(end, len(audio))
                    
            if start is not None:
                audio = audio[start : end]
        
            # create folder if needed
            label, filename = file.split('/')[-2:]
            if not os.path.exists(label):
                try:
                    os.makedirs(label)
                except:
                    pass
            files_edges[id_ + '.ogg'] = (start, end)
        
            # save h5 file
            with h5py.File(f'{id_}.h5', 'w') as data_file:
                data_file.create_dataset('raw', data=audio.numpy())
        save_obj(files_edges, 'files_edges')

    # ----------------------------------------------------------------------------------------------------
    # 0.6 audio filters
    # ----------------------------------------------------------------------------------------------------

    def ts_to_s(speech_timestamps):
        """ convert audio index positions with sampling rate `SR` to seconds """
        out = []
        for dt in speech_timestamps:
            ds = dict(start=dt['start']/SR, end=dt['end']/SR)
            out.append(ds)
        return out


    def complete_audio_filters(raw_audio_filters):
        """ finalize a dict of audio dilters by converting audio hits into audio sections (start, end),
            merging them when applicable (if the distance to the previous hit <= 5s).
            section for a hit is defined a (hit - BAND, hit + BAND)
        """
        BAND = 4
        
        audio_filters = dict()
        for id_, raw_sections in raw_audio_filters.items():
            sections = []
            prior_hit = -100
            for sec in raw_sections:
                if isinstance(sec, tuple):
                    sections.append(sec)
                    prior_hit = sec[1] - 4
                else:
                    if prior_hit + 5 >= sec:  # merge with previous section
                        sections[-1] = (sections[-1][0], sec + BAND)
                    else:
                        start = max(sec - BAND, 0)
                        end = 5 if start == 0 else sec + BAND
                        sections.append((start, end))
                    prior_hit = sec
                        
            audio_filters[id_] = sections
        return audio_filters   
    

    def build_custom_filter(meta):
        """ build list `custom_filter` with audio sections, i.e., tuple(start, end), for 
            the files in meta in the same order.
            The original start and end values are adjusted to account for the fact that the
            audio file skips the begining of the recording up to the start of the first section.
        """
        # complete train filters and get file edges
        train_audio_filters = complete_audio_filters(TRAIN_AUDIO_FILTERS)
        additional_audio_filters = complete_audio_filters(ADDITIONAL_AUDIO_FILTERS)
        raw_audio_filters = train_audio_filters | additional_audio_filters
        files_edges = get_files_edges([config.dataset, config.previous_dataset])
        
        # prepare customm filter format
        custom_filter = []
        for row in meta.to_dict(orient='records'):
            row = dotdict(row)
            id_ = row.filename[:-4]
            sections = raw_audio_filters.get(id_, None)
            if sections is None:
                section_start, section_end = files_edges[row.filename]
                sections = [(section_start/SR, section_end/SR)]
            else:
                # make sure that end of last section is not larger than the length of the audio file
                sections[-1] = (sections[-1][0], min(sections[-1][1], files_edges[row.filename][1] / SR))

            # adjust section start and end based on audio start
            start = sections[0][0]
            sections = [(s - start, e - start) for s, e in sections]
            
            # defin section probability based on size
            sections_prob = [(end - start) for start, end in sections]
            custom_filter.append(dotdict(sections=sections, sections_prob=sections_prob))
        return custom_filter


    def get_lot_per_species(meta):
        dim_per_species = meta.groupby('primary_label')['dim'].sum()
        lot_per_species = dict()
        lot = 0
        lot_size = 0
        max_dim_per_lot = 19.5e9 / 32_000 / 4
        for species, dim in zip(dim_per_species.index, dim_per_species.values):
            if lot_size + dim > max_dim_per_lot:
                lot += 1
                lot_size = 0
            lot_size += dim
            lot_per_species[species] = lot
        return lot_per_species


    def get_files_for_lot(meta, lot, filename=None):
        """ split files in lots that fit in 19GB """
        lot_per_species = get_lot_per_species(meta)
        files = []
        for i, row in enumerate(meta.to_dict(orient='records')):
            row = dotdict(row)
            if lot_per_species[row.primary_label] != lot:
                continue
            if filename is not None:
                files.append(filename[i])
            else:
                files.append(row.filename)
        return files    
    

    def get_filename_to_lot(dataset, verbose=0):
        filename_to_lot = dict()
        for lot in range(7):
            lot_files = glob.glob(f'{ROOT}/b5-data-{dataset}-l-{lot}/*/*.h5')
            if len(lot_files) == 0:
                break
            for file in lot_files:
                filename = '/'.join(file.split('/')[-2:])[:-3] + '.ogg'
                filename_to_lot[filename] = file
            if verbose:
                print(lot, len(lot_files))
        return filename_to_lot
    
    
    def get_files_edges(datasets):
        """ gest the start and end indices of the audio (ignoring speech and non vocalization sections) """
        files_edges = dict()
        
        n_files = 0
        for dataset in datasets:
            files = glob.glob(f'{ROOT}/b5-data-{dataset}*/files_edges.pkl')
            for file in files:
                temp = load_obj(file)
                files_edges = files_edges | temp
                n_files += 1
        print(f'Loaded files_edges: {n_files}')
        return files_edges
        
    
    SPEECH_DB_TH = -50

    class Speech_filter:
        """ Class that takes an audio and returns the start and end indices of the species
            singing mostly by cutting off human speech.
        
            It identifies human speech using a combination of two mechanisms:
            1) We accumulate the audio power over chunks of .1s and identify points in which it intersects
                -50 db, which is considered silence. Usually a silent period occurs before the author adds
                speech comments to the audio.
            2) We use a model to identify the segments (start / end) of speech in the audio.
            
            We use the following parameters:
                - sing_min_duration: min duration of singing between two periods of silence (power=-50db).
                - speech_notes_time: if silence initiates (power=-50db) after this many seconds we cut the
                    audio (likely speech follows).
                - spech_merge_th: two consecutive speeches with an interval smaller than `spech_merge_th`
                    are merged into a single speech
                - speech_start_th: spech that occurs before `speech_start_th` seconds is considered a false
                    positive.
                - speech_min_duration: any speech that last less than this number of seconds is ignored
                    
            And implement the following rules (firt 1a/1b and then 2):
            1a) If power crosses -50 db up and then down, for a duration >= sing_min_duration then we mark
                those two points as the start and end of the audio.
            1b) otherwise, if power crosses -50 down after speech_notes_time seconds, we mark the audio as
                starting at 0 and ending at that point.
            
            2) We then use a model to identify periods of speech and traverse those periods:
                - if a speech period initiates within spech_merge_th seconds of the previous one, we merge
                    them.
                - if the duration of the current speech is >= speech_min_duration:
                    - if the speech start time <= speech_start_th, we assume it as false positive and ignore
                        further speech periods.
                    - otherwise we cut the audio at the start of the speech and return
                - if the duration of the current speech > .5 and speech start time > 30 seconds, we cut the 
                    audio at the start of the speech and return.
                - otherwise we move to the following speech period.
        """

        def __init__(self, sing_min_duration=2, speech_notes_time=7, spech_merge_th=.3, speech_min_duration=2, 
                     speech_start_th=8, th=.5, threads=1):
            self.sing_min_duration = sing_min_duration
            self.speech_notes_time = speech_notes_time
            self.spech_merge_th = spech_merge_th
            self.speech_min_duration = speech_min_duration
            self.speech_start_th = speech_start_th
            self.th = th
            
            # interval variables
            self.chunk_len = 0.1
            self.chunk = int(self.chunk_len * SR)
            torch.set_num_threads(threads)
            self.model, (self.get_speech_timestamps, _, read_audio, _, _) = \
                torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')

                
        def __call__(self, audio, th=None):
            if len(audio.shape) > 1:
                audio = audio[0]
            len_ = len(audio)
            
            # power based detection
            chunk = int(self.chunk_len * SR)
            power = audio ** 2
            pad = int(np.ceil(len(power) / chunk) * chunk - len(power))
            power = np.pad(power, (0, pad))
            power = power.reshape((-1, chunk)).sum(axis=1)
            power_dB = 10 * np.log10(power)
            x = power_dB - SPEECH_DB_TH
            start, end = 0, len_
            intersections = np.where(x[:-1] * x[1:] < 0)[0]
            for s, e in zip(intersections[:-1], intersections[1:]):
                if x[s] < x[s+1] and (e - s) * self.chunk_len >= self.sing_min_duration:
                    start, end = s * chunk, e * chunk
                    break
                elif x[s] > x[s+1] and s * self.chunk_len > self.speech_notes_time:
                    start, end = 0, s * chunk
                    break

            # model based detection
            threshold = th if th is not None else self.th
            speech_timestamps = self.get_speech_timestamps(
                audio[start : end], self.model, sampling_rate=SR, threshold=threshold)
            
            if len(speech_timestamps) > 0:
                s, e = -1e6, -1e6
                # print(ts_to_s(speech_timestamps))
                for ts in speech_timestamps:
                    if ts['start'] - e < self.spech_merge_th * SR:  # merge
                        e = ts['end']
                    else:
                        s, e = ts['start'], ts['end']
                    duration = (e - s) / SR
                    start_s = (start + s) / SR
                    if duration >= self.speech_min_duration or (duration > .5 and start_s >= 30):
                        if start_s <= self.speech_start_th:
                            break  # probably a false positive
                        start, end = start, start + s
                        break
            return start, end        

    # ----------------------------------------------------------------------------------------------------
    # 0.5 png specs
    # ----------------------------------------------------------------------------------------------------

    def build_specs_png(files, img_dim=(224,224), nfft=2048, low_cut=50, high_cut=16_000, parallel=True,
                        bits=8, max_duration=None, norm=True, snipet=False):
        """ read audios, convert to spectrogram, convert it to png and save as file """

        hop_length = SR * 5 // (img_dim[1] - 1)
        n_mels = img_dim[0]

        melspec_transform_ = T.MelSpectrogram(
            n_fft=nfft, hop_length=hop_length, f_min=low_cut, f_max=high_cut, sample_rate=SR,
            n_mels=n_mels, norm='slaney', mel_scale='slaney', pad_mode='constant')
        db_transform_ = db_transform_snipet if snipet else db_transform
        tensor_to_pil = v2.ToPILImage()
        max_val = 2**bits - 1
        dtype = torch.uint8 if bits == 8 else torch.int
        process_fn = _process_norm if norm else _process

        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if max_duration is not None:
                audio = audio[:max_duration * sr]
            spec = melspec_transform_(audio)
            spec = db_transform_(spec)
            min_, max_ = torch.min(spec), torch.max(spec)
            norm_spec = (spec - min_) / (max_ - min_)
            img = (norm_spec * max_val).to(dtype)
            img = tensor_to_pil(img)
            path = file.split('/')
            label, filename = path[-2:]
            if not os.path.exists(label):
                try:
                    os.makedirs(label)
                except:
                    pass
            output_filename = filename.replace('.ogg', '_3.png')
            img.save(f'{label}/{output_filename}')

        def _process_norm(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if max_duration is not None:
                audio = audio[:max_duration * sr]
            spec = melspec_transform_(audio)
            spec = db_transform_(spec)
            # norm
            mean_, std_ = torch.mean(spec), torch.std(spec)
            spec = (spec - mean_) / (std_ + eps)
            # minmax
            min_, max_ = torch.min(spec), torch.max(spec)
            norm_spec = (spec - min_) / (max_ - min_)
            img = (norm_spec * max_val).to(dtype)
            img = tensor_to_pil(img)
            path = file.split('/')
            label, filename = path[-2:]
            if not os.path.exists(label):
                try:
                    os.makedirs(label)
                except:
                    pass
            output_filename = filename.replace('.ogg', '_3.png')
            img.save(f'{label}/{output_filename}')


        if parallel:
            Parallel(n_jobs=-1)(delayed(process_fn)(file) for file in tqdm(files))
        else:
            for file in tqdm(files):
                process_fn(file)        


    def build_specs_uint(files, img_dim=(128,256), nfft=2048, low_cut=50, high_cut=16_000, parallel=True,
                        bits=8, max_duration=4*60, norm=True, snipet=False, min_duration=15, duration=5):
        """ read audios, convert to spectrogram, convert it to uint with `bits` and save as file """

        hop_length = SR * duration // (img_dim[1] - 1)
        n_mels = img_dim[0]

        melspec_transform_ = T.MelSpectrogram(
            n_fft=nfft, hop_length=hop_length, f_min=low_cut, f_max=high_cut, sample_rate=SR,
            n_mels=n_mels, norm='slaney', mel_scale='slaney', pad_mode='constant')
        db_transform_ = db_transform_snipet if snipet else db_transform
        max_val = 2**bits - 1
        dtype = torch.uint8 if bits == 8 else torch.int
        eps = 1e-6

        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if audio.shape[-1] < min_duration*sr:
                return
            if max_duration is not None:
                audio = audio[:max_duration * sr]
            spec = melspec_transform_(audio)
            spec = db_transform_(spec)
            min_, max_ = torch.min(spec), torch.max(spec)
            norm_spec = (spec - min_) / (max_ - min_)
            img = (norm_spec * max_val).to(dtype)
            filename = file.split('/')[-1][:-4]
            save_obj(img, filename)

        def _process_norm(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if audio.shape[-1] < min_duration*sr:
                return
            if max_duration is not None:
                audio = audio[:max_duration * sr]
            spec = melspec_transform_(audio)
            spec = db_transform_(spec)
            # norm
            mean_, std_ = torch.mean(spec), torch.std(spec)
            spec = (spec - mean_) / (std_ + eps)
            # minmax
            min_, max_ = torch.min(spec), torch.max(spec)
            norm_spec = (spec - min_) / (max_ - min_)
            img = (norm_spec * max_val).to(dtype)
            filename = file.split('/')[-1][:-4]
            save_obj(img, filename)

        process_fn = _process_norm if norm else _process

        if parallel:
            Parallel(n_jobs=-1)(delayed(process_fn)(file) for file in tqdm(files))
        else:
            for file in tqdm(files):
                process_fn(file)  


    def build_specs_uint_segments(
            files, img_dim=(128,256), nfft=2048, low_cut=50, high_cut=16_000, parallel=True,
            bits=8, max_duration=4*60, norm=True, snipet=False, min_duration=15, duration=5):
        """ read audios, convert to spectrogram, convert it to uint with `bits` and save as file """

        hop_length = SR * duration // (img_dim[1] - 1)
        n_mels = img_dim[0]

        melspec_transform_ = T.MelSpectrogram(
            n_fft=nfft, hop_length=hop_length, f_min=low_cut, f_max=high_cut, sample_rate=SR,
            n_mels=n_mels, norm='slaney', mel_scale='slaney', pad_mode='constant')
        db_transform_ = db_transform_snipet if snipet else db_transform
        max_val = 2**bits - 1
        dtype = torch.uint8 if bits == 8 else torch.int
        eps = 1e-6

        def _process(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if audio.shape[-1] < min_duration*sr:
                return
            if max_duration is not None:
                audio = audio[:, :max_duration * sr]
            n_segments = audio.shape[-1] // (sr * duration)
            audio = audio[:, : n_segments * duration * sr]
            audio = audio.view(-1, duration * sr)
            spec = melspec_transform_(audio)
            spec = db_transform_(spec)
            min_, max_ = torch.min(spec), torch.max(spec)
            norm_spec = (spec - min_) / (max_ - min_)
            img = (norm_spec * max_val).to(dtype)
            filename = file.split('/')[-1][:-4]
            save_obj(img, filename)

        def _process_norm(file):
            """ returns single img """
            audio, sr = torchaudio.load(file)
            if audio.shape[-1] < min_duration*sr:
                return
            if max_duration is not None:
                audio = audio[:, :max_duration * sr]
            n_segments = audio.shape[-1] // (sr * duration)
            audio = audio[:, : n_segments * duration * sr]
            audio = audio.view(-1, duration * sr)
            spec = melspec_transform_(audio)
            spec = db_transform_(spec)
            # norm
            mean_, std_ = torch.mean(spec), torch.std(spec)
            spec = (spec - mean_) / (std_ + eps)
            # minmax
            min_, max_ = torch.min(spec), torch.max(spec)
            norm_spec = (spec - min_) / (max_ - min_)
            img = (norm_spec * max_val).to(dtype)
            filename = file.split('/')[-1][:-4]
            save_obj(img, filename)

        process_fn = _process_norm if norm else _process

        if parallel:
            Parallel(n_jobs=-1)(delayed(process_fn)(file) for file in tqdm(files))
        else:
            for file in tqdm(files):
                process_fn(file)  

    # ----------------------------------------------------------------------------------------------------
    # 0.6 png specs val
    # ----------------------------------------------------------------------------------------------------

    def build_data_montage_val_fold(
            files, labels, montages, norm=False, dim=(128,256),
            duration=5, bits=8, snipet=False, spec=None,
            unlabeled_selected=None, unlabeled_reference=None, offset=None, norm_audio=False,
            parallel=True):
        transform_to_spec_ = transform_to_spec(
            dim=dim, norm=norm, duration=duration, snipet=snipet, spec=spec, bits=bits)

        def _process(idx):
            """ creates single img and return label """
            file, sample, y = files[idx], montages[idx], labels[idx]
            audio, sr = torchaudio.load(file)
            audio = audio[:, :duration * sr]
            gap = len_audio_5s - audio.shape[-1]

            if gap > 0:
                pad_before = gap // 2
                pad_after = gap - pad_before
                audio = F.pad(audio, pad=(pad_before, pad_after), mode='constant', value=0)
            
            if len(sample) > 0:
                for idx_m in sample:
                    secondary_audio, sr = torchaudio.load(files[idx_m])
                    secondary_audio = secondary_audio[:, :duration * sr]
                    gap = len_audio_5s - secondary_audio.shape[-1]
                    if gap > 0:
                        pad_before = gap // 2
                        pad_after = gap - pad_before
                        secondary_audio = F.pad(secondary_audio,
                            pad=(pad_before, pad_after), mode='constant', value=0)
                    audio = audio + secondary_audio
                    y = y + labels[idx_m]

            if unlabeled_selected is not None and unlabeled_selected[idx] is not None:
                audio = add_unlabeled(
                    audio, unlabeled_selected[idx], offset[idx], unlabeled_reference)
                
            if norm_audio:
                audio /= torch.abs(audio).max()
            spec = transform_to_spec_(audio)
            return spec, y

        if parallel:
            out = Parallel(n_jobs=-1)(delayed(_process)(idx) for idx in tqdm(range(len(files))))
        else:
            out = []
            for idx in tqdm(range(len(files))):
                out.append(_process(idx))
        x, y = zip(*out)
        x, y = np.stack(x), np.stack(y)
        return x, y


    def add_unlabeled(x, unlabeled_file, offset, unlabeled_reference):
        """ add unlabeled segment from file `unlabeled_selected` to x """
        duration = 5
        
        unlabeled_audio, sr = torchaudio.load(unlabeled_file)
        gap = duration * sr - unlabeled_audio.shape[-1]
        if gap > 0:
            pad_before = random.randint(0, gap - 1)
            pad_after = gap - pad_before
            unlabeled_audio = F.pad(unlabeled_audio, pad=(pad_before, pad_after), mode='constant', value=0)
        elif gap < 0:
            start = int(-gap * offset)
            unlabeled_audio = unlabeled_audio[:, start : start + duration * sr]

        if unlabeled_reference:
            max_x = np.abs(x).max()
            max_unlabeled = np.abs(unlabeled_audio).max()
            x = x / max_x * max_unlabeled + unlabeled_audio
        else:
            x = x + unlabeled_audio

        return x


    def build_catalog_birds(meta_val):
        """ adds to catalog bird instances with no secondary labels """
        catalog = {i: [] for i in RANGE_BIRDS}
        
        for i, row in enumerate(meta_val.to_dict(orient='records')):
            row = dotdict(row)
            if row.n_labels == 1:
                bird = label_to_num[row.primary_label]
                catalog[bird].append(i)
        return catalog
        

    def get_montages(catalog, birds, apply_montage, seed=10):
        """ gets montages """
        random.seed(seed)
        montages = []
        for i, apply in enumerate(apply_montage):
            if not apply:
                sample_birds = []
            else:
                sample_birds = [x for x in random.sample(RANGE_BIRDS, random.randint(*config.montage)) if x != birds[i]]
            sample = []
            for bird in sample_birds:
                options = catalog[bird]
                if len(options) > 0:
                    idx = random.randint(0, len(options) - 1)
                    sample.append(options[idx])
            montages.append(sample)
        random.seed()
        return montages


    def get_unlabeled(birds, files_unlabeled, unlabeled_prob, seed=10):
        """ gets list of unlabeled audios and random relative offset """
        random.seed(seed)
        n_birds, n_files = len(birds), len(files_unlabeled)
        unlabeled_idx = np.random.randint(0, n_files, n_birds)
        probs = np.random.rand(n_birds)
        offset = np.random.rand(n_birds)
        unlabeled_selected = [files_unlabeled[unlabeled_idx[i]] if probs[i] < unlabeled_prob
                              else None for i in range(n_birds)]
        random.seed()
        return unlabeled_selected, offset


    def build_data_val(meta, labels, norm=False, dim=(128,256), duration=5, bits=8,
                       folds=None, seed=10, folder=None, snipet=False, spec=None, 
                       unlabeled_prob=.9, files_unlabeled=None, unlabeled_reference=None,
                       norm_audio=False, parallel=True):
        """ build validation montage """
        if unlabeled_reference is None:
            unlabeled_reference = True if unlabeled_prob else False
        for fold in folds:
            val_idx = meta.fold.values == fold
            meta_val = meta[val_idx]
            catalog_birds = build_catalog_birds(meta_val)
            files = [f'{folder}/{x}' for x in meta_val.filename]
            montages = get_montages(catalog_birds, birds[val_idx], meta_val.n_labels == 1, seed=10)
            if unlabeled_prob:
                unlabeled_selected, offset = \
                    get_unlabeled(birds, files_unlabeled, unlabeled_prob, seed=10)
            else:
                unlabeled_selected, offset = None, None
            x, y = build_data_montage_val_fold(files, labels[val_idx], montages, norm=norm, dim=dim,
                    duration=duration, bits=bits, snipet=snipet, spec=spec,
                    unlabeled_selected=unlabeled_selected, unlabeled_reference=unlabeled_reference,
                    offset=offset, norm_audio=norm_audio, parallel=parallel)
            
            assert y.max() == 1
            save_obj(x, f'x_f{fold}')
            save_obj(y, f'y_f{fold}')


    def build_data_train(meta, labels, norm=False, dim=(128,256), duration=5, bits=8,
                       folds=None, seed=10, folder=None, snipet=False, spec=None, 
                       unlabeled_prob=.9, files_unlabeled=None, unlabeled_reference=None,
                       norm_audio=False, parallel=True):
        """ build validation montage """
        if unlabeled_reference is None:
            unlabeled_reference = True if unlabeled_prob else False
        for fold in folds:
            idx = meta.fold.values != fold
            meta_val = meta[idx]
            catalog_birds = build_catalog_birds(meta_val)
            files = [f'{folder}/{x}' for x in meta_val.filename]
            montages = get_montages(catalog_birds, birds[idx], meta_val.n_labels == 1, seed=10)
            if unlabeled_prob:
                unlabeled_selected, offset = \
                    get_unlabeled(birds, files_unlabeled, unlabeled_prob, seed=10)
            else:
                unlabeled_selected, offset = None, None
            x, y = build_data_montage_val_fold(files, labels[idx], montages, norm=norm, dim=dim,
                    duration=duration, bits=bits, snipet=snipet, spec=spec,
                    unlabeled_selected=unlabeled_selected, unlabeled_reference=unlabeled_reference,
                    offset=offset, norm_audio=norm_audio, parallel=parallel)
            
            assert y.max() == 1
            save_obj(x, f'x_f{fold}')
            save_obj(y, f'y_f{fold}')

    # ----------------------------------------------------------------------------------------------------
    # 0.11 optuna
    # ----------------------------------------------------------------------------------------------------

    try:
        from optuna.exceptions import ExperimentalWarning
        warnings.filterwarnings("ignore", category=ExperimentalWarning)
    except:
        pass

    class Objective_optuna():
        """ wrapper for function to be optimized by Optuna, whose purpose is to
            customize the parameters generated by Trial, including handling exceptions to
            parameters (fixed parameters for a trial).
        """
        def __init__(self, study, hp_fixed, hp_optimize, hp_exceptions, optimize_fn):
            self.study = study
            self.hp_fixed = hp_fixed
            self.hp_optimize = hp_optimize
            self.hp_exceptions = hp_exceptions
            self.optimize_fn = optimize_fn

        def __call__(self, trial):
            """ build hp and invoke training optimize_fn """
            study = self.study
            hp_fixed = self.hp_fixed
            hp_optimize = self.hp_optimize
            hp_exceptions = self.hp_exceptions
            optimize_fn = self.optimize_fn

            # placeholder for code that implements specific rules, e.g., if encoder!='n/a' set norm=True

            study.sampler = optuna.samplers.TPESampler()
            if len(hp_exceptions) > 0:
                fixed_params = self.hp_exceptions
                partial_sampler = optuna.samplers.PartialFixedSampler(fixed_params, study.sampler)
                study.sampler = partial_sampler

            hp = {**hp_fixed, **hp_optimize(trial)}

            return optimize_fn(hp)


    def train_optuna(scenario, hp_fixed, hp_optimize, coach, previous_scenario='',
                     duration=8, n_trials=100, trial_lot=10, hp_exceptions={}):
        """ run various optuna trials for a model
            Example:
                >>> train(coach, previous_scenario='previous_scenario',
                >>>       HP_FIXED_LINEAR, HP_OPTIMIZED_LINEAR,
                >>>       duration=2, n_trials=100, trial_lot=10)
        """
        SLEEP = 0
        timer_ = Timer()
        timer_.start(3600 * duration)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        if hasattr(coach, 'study'):
            study = coach.study
        else:
            study = optuna.create_study(study_name=scenario, direction='maximize')
            iteration = 0
            if previous_scenario != '':
                try:
                    study = joblib.load(previous_scenario)
                    iteration = len(study.trials)
                except:
                    print(f'Unable to load scenario {previous_scenario}')

            coach.study = study
            coach.iteration += iteration
        objective = Objective_optuna(study, hp_fixed, hp_optimize, hp_exceptions, coach)
        for i in range(n_trials // trial_lot):
            study.optimize(objective, n_trials=trial_lot, timeout=3600, gc_after_trial=True)
            gc.collect()

            study.trials_dataframe().to_csv(f'./trials {scenario}.csv')
            joblib.dump(study, f'./study {scenario}.dump'.format(scenario))
            coach.save(scenario)
            if not timer_.lap():
                break
            time.sleep(SLEEP)

        try:
            optuna.visualization.plot_optimization_history(study).show()
            optuna.visualization.plot_param_importances(study).show()
            optuna.visualization.plot_slice(study).show()
        except Exception:
            pass


    # ----------------------------------------------------------------------------------------------------
    # 0.12 feature importance
    # ----------------------------------------------------------------------------------------------------

    def predict_folds_(models, x, folds):
        """ predict multiple folds and consolidate """
        y_pred = np.empty(folds.shape[0])
        for fold in range(len(models)):
            filter_fold = folds == fold
            y_pred_fold = models[fold](x[fold]).squeeze()
            y_pred[filter_fold] = y_pred_fold
        return y_pred


    def feature_importance_permutation(model_spec, x, y_true, folds, scorer_fn=None, n=1):  # FG: pending
        """ supports only one model or ensemble at a time """
        if not isinstance(model_spec, list):
            model_spec = [model_spec]
        if not isinstance(model_spec[0][SPEC_MODEL_FOLDER], str):
            model_spec = [(x[SPEC_MODEL_NAME], get_folder(x[SPEC_MODEL_NAME], x[1]), *x[SPEC_MODEL_NAME + 1:]) for x in model_spec]
        results = []
        print('Calculate feature importance...')
        models = load_models(model_spec)
        y_pred = predict_folds_(models, x, folds)
        baseline = scorer_fn(y_true, y_pred)
        print('baseline:', baseline)

        for feature in tqdm(range(len(x_columns))):
            saved_feature = x[feature].copy()
            for i in range(n):
                np.random.shuffle(x[feature])
                y_pred = predict_folds_(models, x, folds)
                score = scorer_fn(y_true, y_pred)
                results.append({'feature': x_columns[feature], 'iteration': i, 'diff': score - baseline})
            x[feature] = saved_feature
            gc.collect()

        result = pd.DataFrame(results).set_index(['feature', 'iteration']).unstack('iteration')
        columns = [f'mae_{i}' for i in range(n)]
        result.columns = columns
        result['mean'] = result[columns].mean(axis=1)
        result['share'] = result['mean'] / result['mean'].sum() * 100
        result['std'] = result[columns].std(axis=1)
        result['var 95%'] = result['std'] * 1.96 / result['mean'] * 100
        result = result.sort_values(by='mean', ascending=False)
        return result

    # ----------------------------------------------------------------------------------------------------
    # 0.13 analysis
    # ----------------------------------------------------------------------------------------------------

    def print_scores(scores, name='', precision=4):
        target_scores = [x for x in scores]
        m = np.mean(target_scores)
        detail = ', '.join([f'{x:.{precision}f}' for x in target_scores])
        print(f'{name:<7}:  {m:.{precision}f} [{detail}]')


    def copy_oofs(names, in_folder, val_samples):
        """ copy oofs from one predict folder to another, inlcuidng only some models """
        for val_sample in val_samples:
            for name in names:
                path = f'{in_folder}/{val_sample}/oof_{name}_f0.pkl'
                if os.path.exists(path):
                    if not os.path.exists(f'{val_sample}'):
                        os.makedirs(f'{val_sample}')
                    shutil.copy(path, f'{val_sample}/oof_{name}_f0.pkl')

    # ----------------------------------------------------------------------------------------------------
    # 0.14 ensembles
    # ----------------------------------------------------------------------------------------------------

    t_cols = [f't{i}' for i in RANGE_BIRDS]
    p_cols = [f'p{i}' for i in RANGE_BIRDS]

    def names_to_idxs(selected, names):
        """ return idx for selected in the order present in selected """
        name_to_idx = dict(zip(names, range(len(names))))
        return np.array([name_to_idx[x] for x in selected if x in name_to_idx])

    def get_idx_selected(selected, all):
        """ returns the indices of `all` that are in selected """
        return np.array([i for i, item in enumerate(all) if item in selected])


    def ensemble_oof(coef, oof, names):
        w, idxs = [], []
        if isinstance(coef, list):
            coef = {x:1 for x in coef}
        for name, factor in coef.items():
            w.append(factor)
            idxs.append(names.index(name))
        w = np.array(w).reshape((-1,1,1))
        e = (oof[idxs] * w).sum(axis=0) / w.sum(axis=0)
        # e = e / e.sum(axis=-1, keepdims=True)
        return e


    def scores_ensemble(coef, oof, names, target):
        e_best = ensemble_oof(coef, oof, names)
        score_best = auc_multi(e_best, target)
        idxs = names_to_idxs(coef.keys(), names)
        e_mean = oof[idxs].mean(axis=0)
        score_mean = auc_multi(e_mean, target)
        if 'scores' in globals():
            score_base = np.mean([scores[x] for x in coef.keys()])
        else:
            score_base = 0
        return score_best, score_mean, score_base


    class Ensemble_params:
        def __init__(self, df, n=0, kappa=True):
            self.results = []
            self.count = n
            self.best = np.inf
            self.reference = 1.006589

            # prepare cartesians
            self.az = df.az.values
            self.ze = df.ze.values

            models = [x[4:] for x in df.columns if 'paz_' in x]
            xyz, wk = [], []
            for m in models:
                xyz_model = polars_to_cartesians(df[[f'paz_{m}', f'pze_{m}']].values)
                if kappa:
                    w_model = df[f'kappa_{m}'].values.reshape((-1, 1))
                    xyz_model = xyz_model * w_model
                    wk.append(w_model)
                #cols += [f'x_{m}', f'y_{m}', f'z_{m}']
                xyz.append(xyz_model)
            self.xyz = np.stack(xyz, axis=1)  # n, models, 3
            self.wk = np.concatenate(wk, axis=1)    # n, models, 1


        def __call__(self, hp):
            self.count += 1

            # ensemble pred
            w = np.array(list(hp.values())).reshape((1, -1))
            w = self.wk * w
            w_sum = np.sum(w, axis=1)
            w = w / w_sum.reshape((-1,1))
            w_3d = np.repeat(np.expand_dims(w, 2), 3, axis=2)
            xyz = (self.xyz * w_3d).sum(axis=1)
            paz, pze = cartesians_to_polars(xyz)

            # calculate score
            score = ad_polar(self.az, self.ze, paz, pze)
            delta = score - self.reference
            self.results.append(dict(**hp, ad=score, delta=delta))
            if score < self.best:
                str_hp = [k for k, v in hp.items() if v != 0]
                print(f'{self.count:>3}: {delta:8>.6f} -> {score:.6f} \t {str_hp}')
                self.best = score
            return score

        @property
        def result(self):
            return pd.DataFrame(self.results).sort_values('ad', ascending=True)


    def gaussian_kernel(l=5, sig=1.):
        """ https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy
            creates 1D gaussian kernel with side length `l` and a sigma of `sig`
        """
        ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
        gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
        return gauss / np.sum(gauss)

    # -------------------
    ## means
    # -------------------

    def ensemble_mean(selected, names, oof, target, ws=None, temperature=None):
        """ Simple mean of various models. Also generates mean per fold """
        idxs = names_to_idxs(selected, names)
        if not temperature:
            oof_ensemble = oof[idxs].mean(axis=0)
        else:
            if temperature > 1 and oof[idxs].min() < 0:
                oof = sigmoid(oof)
            oof_ensemble = np.power(np.power(oof[idxs], temperature).mean(axis=0), 1/temperature)
        score = auc_multi(oof_ensemble, y_true)
        print(f'{"mean":<6}: {score:.4f}')
        return score

    # -------------------
    ## best
    # -------------------

    from itertools import combinations

    def ensemble_best(names, scores_detail, n=5):  # PENDING
        """ ensembles up to `n` models by picking the best score for each row.
            Use to pick models for L2 model.
        """
        all_options, best = dict(), dict()
        best_option = None
        best_score = np.inf
        scores_detail_a = scores_detail.values
        names_a = np.array(names)
        for i in range(2, n+1):
            options = list(combinations(range(len(names)), i))
            for option in tqdm(options):
                cols = list(option)
                score = auc_multi(oof, y_true)
                score = scores_detail_a[:, cols].min(axis=1).mean()
                option_names = names_a[cols]
                all_options[tuple(option_names)] = score
                if score < best_score:
                    best_score = score
                    best_option_i = option_names
                    print(f'{score:.4f} - {list(option_names)}')
            best[i] = (best_score, best_option_i)
        return best, all_options

    # -------------------
    ## logistic regression
    # -------------------

    # -------------------
    ## logistic regression
    # -------------------

    def ensemble_regression(names, oof, y_true, fit_intercept=True, n=5):
        """ ensemble using regression. Reports 3 scores: 
            - best based on correlation
            - mean of all models used in correlation
            - mean of models indicated by correlation
        """
        LIMIT_ENSEMBLE = 2
        COEF_THRESHOLD_MAX = 1e7
        selected_models = names
       
        start_time = time.time()
        coefs, results = [], []
        best_base_score = baseline_score

        # run regression
        names_ = selected_models.copy()
        print(f'{names_}: x{len(names_)}')
        best_score = np.inf
        names_idx = None
        start_names = names_
        M, B, C = oof.shape
        yt = y_true.reshape(-1)
        yp = oof.reshape((M, -1)).T
        for step in range(len(start_names)):
            reg = LinearRegression(fit_intercept=fit_intercept, positive=True)
            if names_idx is not None:
                yp = yp[:, names_idx]
            reg.fit(yp, yt)
            gc.collect()
            coef, coef_all, skip = dict(), [], []
            # print(reg.coef_)
            for i, name in enumerate(names_):
                if reg.coef_[i] > COEF_THRESHOLD_MAX:
                    skip.append(name)
                if reg.coef_[i] != 0:
                    coef[name] = reg.coef_[i]
                coef_all.append(reg.coef_[i])
            count = len(coef)
            coef_s = pd.Series(coef)
            coef_s = coef_s.sort_index(key=lambda x: x.str[:4] + x.str[4:].str.zfill(4))
            coef_all = np.array(coef_all).reshape((-1, 1))
            coef['intercept'] = reg.intercept_
            #print(f'--- {coef}')

            pred_b = (yp @ coef_all).squeeze() + coef['intercept']
            pred_ma = np.mean(yp, axis=1)
            idxs = get_idx_selected(coef, names_)
            pred_m = np.mean(yp[:, idxs], axis=1)
            
            ensemble_best = auc_multi(pred_b.reshape((-1, B, C)), y_true)
            ensemble_mean = auc_multi(pred_m.reshape((-1, B, C)), y_true)
            ensemble_meana = auc_multi(pred_ma.reshape((-1, B, C)), y_true)
            results.append([step, count, ensemble_best, ensemble_mean, ensemble_meana,
                            ensemble_best - best_base_score, ensemble_mean - best_base_score, 
                            ensemble_meana - best_base_score, 
                            coef, reg.intercept_, None, list(coef_s.index)])
            coefs.append({'best': ensemble_best, 'mean': ensemble_mean, **coef})

            if ensemble_best <= best_score:
                best_score = ensemble_best
                best_idx = len(results) - 1

            if len(skip) > 0:
                to_remove = skip
            else:
                # remove model with smallest absolute weight
                to_remove = coef_s.sort_values().index[0]
            print(f'{step:>2} ({count:>2})  {ensemble_best:.6f}, {ensemble_mean:>9.6f}, {ensemble_meana:>9.6f} -> {to_remove}')
            names_idx = [names_.index(x) for x in coef_s.index if x != to_remove]
            names_ = [x for x in coef_s.index if x != to_remove]
            if count <= LIMIT_ENSEMBLE:
                break

        column_names = ['step', 'count', 'best', 'mean', 'meana', 'delta_best', 
                        'delta_mean', 'delta_meana', 'coefs', 'intercept', 'selected', 'models']
        results_df = pd.DataFrame(results, columns=column_names)
        coef_df = pd.DataFrame(coefs).T
        coef_df.to_csv(f'coef_df reg.csv')
        results_df.to_csv(f'results_df reg.csv')

        # summarize results
        best_idxs = results_df.best.argsort().values
        for idx in best_idxs[:n]:
            show_ensemble_result(results_df, idx, len(start_names), best_base_score)
        
        return results_df, coef_df


    def show_ensemble_result(results_df, idx, initial_len='', best_base_score=None):
        if best_base_score is None:
            best_base_score = baseline_score if 'baseline_score' in globals() else 0.30
        results_df.iloc[idx][-2] = 1
        coef = results_df.iloc[idx][-4]
        coef_s = pd.Series(coef)
        final_names = [x for x in coef.keys() if x != 'intercept']

        ensemble_best, ensemble_mean, ensemble_meana = results_df.iloc[idx][2:5]
        end_time = time.time()
        print(f"\n{'-'*20} x{len(final_names)} / {initial_len} = "
              f"{ensemble_best:.6f} {ensemble_mean:>.6f} {ensemble_meana:>.6f}  |  "
              f"{ensemble_best - best_base_score:.6f} {ensemble_mean - best_base_score:>.6f}"
              f"{ensemble_mean - best_base_score:>.6f}")
        coef_s = coef_s.sort_values(ascending=False)
        print(coef_s)
        print()
        print(sort_names(final_names))
        print()
        print(coef_s.to_dict())

    # -------------------
    ## Optimize
    # -------------------

    def auc_multi(preds, labels):
        """ Version of macro-averaged ROC-AUC score that ignores all classes that have no true positive labels. """
        scored_cols = labels.sum(axis=0) > 0
        return sklearn.metrics.roc_auc_score(
            labels[:, scored_cols], preds[:, scored_cols], average='macro')


    def opt_ensemble_models_(w_in):
        """ optimization function. In addition to weights, which are provided as input,
            uses global variables to compute metric.
        """
        wt = w_in.sum()
        w = w_in.reshape((-1, 1, 1))
        pred = (oof_ensemble * w).sum(axis=0) / wt
        score = auc_multi(pred, _opt_target)
        # print(score, oof_ensemble.shape, w.shape)
        return -score

    def ensemble_optimize(selected, names, oof, target, n=5, ws=None, method=None):
        """ find best combination of models using scipy optimize function. 
            Once a solution is found, removes the model with the lowest coeficient and
            tries to find a better solution without it.
            Tends to be better than optuna and worse than regression. It can be used in
            cases in which regression can't be used (the scoring metric desn't align with
            regression).
        """
        global oof_ensemble, _opt_target
        
        best_base_score, best, best_score =  baseline_score, [], -np.inf
        step, coefs, results = 0, [], []
        _opt_target = target
        _opt_weights = ws
        
        initial_names = selected.copy()
        print(f'{initial_names}: x{len(initial_names)}')
        while len(selected) > 0:
            t1 = time.time()
            # prepare data
            idxs = names_to_idxs(selected, names)
            oof_ensemble = oof[idxs]

            # run optimization
            dim = oof_ensemble.shape[0]
            guess = np.ones(dim)
            bounds = [(0., 1.)] * dim
            opt = optimize.minimize(opt_ensemble_models_, guess, bounds=bounds, method=method)
            score = -opt.fun
            # print results
            coef, coef_all = dict(), []
            for name, coef_ in zip(selected, opt.x):
                if coef_ > 0:
                    coef[name] = coef_
                coef_all.append(coef_)
            count = len(coef)
            coef_all = np.array(coef_all).reshape((-1, 1, 1))
            coef_s = pd.Series(coef)
            if score > best_score:
                best_score = score
                best = coef
            
            # recalculate scores
            #print(oof_ensemble.shape, coef_all.shape)
            pred_b = (oof_ensemble * coef_all.reshape((-1, 1, 1))).sum(axis=0)
            pred_ma = np.mean(oof_ensemble, axis=0)
            idxs = get_idx_selected(coef, selected)
            pred_m = np.mean(oof_ensemble[idxs], axis=0)
            
            # score = auc_multi(oof, y_true)
            ensemble_best = auc_multi(pred_b, _opt_target)
            if np.abs(score - ensemble_best) > 1e-6:
                print('Score in opt != auc:', score, ensemble_best)
            ensemble_mean = auc_multi(pred_m, _opt_target)
            ensemble_meana = auc_multi(pred_ma, _opt_target)
            results.append([step, count, ensemble_best, ensemble_mean, ensemble_meana,
                            ensemble_best - best_base_score, ensemble_mean - best_base_score, 
                            ensemble_meana - best_base_score, 
                            coef, None, None, list(coef.keys())])
            coefs.append({'best': ensemble_best, 'mean': ensemble_mean, **coef})
            
            # reduce number of selected models and try again. Remove models with insignificant coeficients.
            # if none exists, remove the model with the lowest coeficient.
            size = len(coef)
            coef_s = coef_s.sort_values(ascending=False)
            coef_s = coef_s[coef_s > 1e-4]
            selected = list(coef_s.keys())
            if len(selected) == size:
                selected = selected[:-1]
            duration = time.time() - t1
            print(f'{step:>2} ({count:>2})  {ensemble_best:.6f}, {ensemble_mean:>9.6f}, {ensemble_meana:>9.6f}   {duration:,.0f} s')
            step += 1

        column_names = ['step', 'count', 'best', 'mean', 'meana', 'delta_best', 
                        'delta_mean', 'delta_meana', 'coefs', 'intercept', 'selected', 'models']
        results_df = pd.DataFrame(results, columns=column_names)
        coef_df = pd.DataFrame(coefs).T
        coef_df.to_csv(f'coef_df opt.csv')
        results_df.to_csv(f'results_df opt.csv')

        # summarize results
        best_idxs = results_df.best.argsort().values[::-1]
        for idx in best_idxs[:n]:
            show_ensemble_result(results_df, idx, len(names), best_base_score)

        # print()
        # print('-'*40, 'best', len(best), '-'*40)
        # print(f'{len(best):>2}: {best_score:.6f}  {best_score - baseline_score:.6f}')
        # print(f'{sorted(list(best.keys()))} \n{best}')
        return results_df, coef_df
    # -------------------
    ## Hill climb
    # -------------------

    def is_large_model(name):
        if 'eca' in name:
            return True
        return False
        

    def count_large_models(names):
        return sum(1 for x in names if is_large_model(x))


    def ensemble_hill_climb(candidates, names, oof, target, temperature=None, fixed=None, n=12):
        """ start by best model and add a model at a time (the one with the best improvement)
            until we stop improving.
        """
        
        # get candidates
        scores_ = {name: score_ for name, score_ in scores.items() if name in candidates}
        temp = pd.Series(scores_, name='score').to_frame().reset_index().sort_values('score', ascending=False)
        temp.rename(columns={'index': 'name'}, inplace=True)
        best = temp.name.values[0]
        if fixed is None:
            selected = [best]
            best_score = temp.score.values[0]
        else:
            selected = fixed
            best_score = np.max([scores_[x] for x in fixed])

        if temperature > 1 and oof.min() < 0:
            oof = sigmoid(oof)
        # select
        candidates = [x for x in candidates if x not in selected]
        while len(selected) < n:
            has_improved = False
            for candidate in candidates:
                idxs = names_to_idxs(selected + [candidate], names)
                if not temperature:
                    oof_ensemble = oof[idxs].mean(axis=0)
                else:
                    oof_ensemble = np.power(np.power(oof[idxs], temperature).mean(axis=0), 1/temperature)
                score = auc_multi(oof_ensemble, target)
                if score > best_score:
                    best_candidate = candidate
                    best_score = score
                    has_improved = True
            if not has_improved:
                print('finished improving')
                break
            selected.append(best_candidate)
            sorted_selected = sort_names(selected)
            print(f'{len(selected):>2}: {best_score:.6f}  {best_score - baseline_score:.6f}  - {sorted_selected}')
            
        # print('-' * 40)
        # print(f'{len(selected):>2}: {best_score:.6f}  {best_score - baseline_score:.6f}  - {sorted_selected}')
        return selected


    # -------------------
    ## Optuna
    # -------------------

    class Objective_weights:
        def __init__(self, target, oof, hp, reference, n=0, short_progress=0):
            self.results = []
            self.count = n
            self.best_ = -np.inf
            self.target = target
            self.oof = oof
            self.hp = hp
            self.reference = reference
            self.short_progress = short_progress

        def __call__(self, trial):
            self.count += 1

            # ensemble pred
            params = self.hp(trial)
            w = np.array(list(params.values())).reshape((-1, 1, 1))
            pred = (self.oof * w).sum(axis=0)
            score = auc_multi(pred, self.target)
            delta = score - self.reference
            self.results.append(dict(**params, c=score, delta=delta))
            if score > self.best_:
                str_hp = [k for k, v in params.items() if v != 0]
                if self.short_progress and len(str_hp) > self.short_progress:
                    str_hp = len(str_hp)
                sys.stdout.write(f'{" "*80}\r')
                sys.stdout.flush()
                print(f'{self.count:>3}: {delta:8>.6f} -> {score:.6f} \t {str_hp}')
                self.best_ = score
            else:
                text = f'\r{self.count:>3}: {delta:8>.6f} -> {score:.6f}        '
                sys.stdout.write(text)
            return score

        @property
        def result(self):
            return pd.DataFrame(self.results).sort_values('c', ascending=False)
            
        def best(self):
            coef = self.result.sort_values('c', ascending=False).head(1).to_dict(orient='records')[0]
            coef = {k: v for k, v in coef.items() if v != 0}
            score = coef.pop('c')
            delta = coef.pop('delta')
            return coef, score


    def ensemble_optuna(candidates, names, target, oof, hp, reference, initial=None, sampler=None,
                        scenario='optuna', n_results=5, study=None, previous_study=None, 
                        n_trials=2_000, duration=3600, short_progress=0):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        # create or load study
        if not study:
            if previous_study is not None:
                try:
                    study = joblib.load(previous_study)
                except:
                    print(f"Unable to load scenario '{previous_study}'")
                    study = optuna.create_study(study_name=scenario, direction='maximize')
            else:
                study = optuna.create_study(study_name=scenario, direction='maximize')

        # handle fixed values
        study.sampler = sampler if sampler is not None else optuna.samplers.TPESampler()
        if initial is not None:
            study.enqueue_trial(initial)   

        # optimize
        idx_candidates = names_to_idxs(candidates, names)
        oof_candidates = oof[idx_candidates]
        objective = Objective_weights(
            target, oof_candidates, hp, reference, n=len(study.trials), short_progress=short_progress)
        try:
            study.optimize(objective, n_trials=n_trials, timeout=duration, gc_after_trial=False)
        except KeyboardInterrupt:
            print(f'\nInterrupted by request')

        best_coef, best_score = objective.best()
        
        # output results
        objective.result.to_csv(f'./results {scenario}.csv')
        study.trials_dataframe().to_csv(f'./trials {scenario}.csv')
        joblib.dump(study, f'study {scenario}.dump')
        
        score_best, score_mean, score_base = scores_ensemble(best_coef, oof, names, target)
        delta = best_score - reference
        print(f'\n*** best: {delta:8>.6f} -> {score_best:.6f}, {score_mean:.6f}, {score_base:.6f}')
        print(f'\n{best_coef}')

        if n_results > 0:
            # visualize and store results
            print('best:\n')
            display(objective.result.head(n_results))
        
        return study, objective, best_coef, best_score

    # ----------------------------------------------------------------------------------------------------
    # 0.20 Visualize
    # ----------------------------------------------------------------------------------------------------

    def plot_audio_speech(file, play_audio=True, th=.5):
        global speech_filter, model, get_speech_timestamps
        
        if 'get_speech_timestamps' not in globals():
            torch.set_num_threads(1)
            model, (get_speech_timestamps, _, read_audio, _, _) = \
                torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
            speech_filter = Speech_filter()        
        
        audio, sr = torchaudio.load(file)
        name = '/'.join(file.split('/')[-2:]).split('.')[0]
        start, end = speech_filter(audio, th=th)
        if start is not None:
            print(file, start/SR, end/SR)
        else:
            print(file)
        #if mode != 2: continue
        audio = audio[0]
        chunk_len = 0.1
        power = audio ** 2
        chunk = int(chunk_len * sr)
        pad = int(np.ceil(len(power) / chunk) * chunk - len(power))
        power = np.pad(power, (0, pad))
        power = power.reshape((-1, chunk)).sum(axis=1)
        
        speech_timestamps = get_speech_timestamps(audio, model, sampling_rate=SR, threshold=th)
        segmentation = np.zeros_like(audio)
        for st in speech_timestamps:
            segmentation[st['start']: st['end']] = 20
        
        fig = plt.figure(figsize=(24, 3))
        fig.suptitle(f'{file}')
        
        t = np.arange(len(power)) * chunk_len
        plt.plot(t, 10 * np.log10(power), 'b')

        if start is not None:
            plt.axvline(x=start/SR, color='g')
        if end is not None:
            plt.axvline(x=end/SR, color='g')

        # plot time lines
        ax1 = plt.gca()
        duration = int(t.shape[0]*.1)
        interval = 5 if duration <=  5 * 50 else 20
        xi = [x for x in range(0, duration, interval)]
        ax1.set_xticks(xi)
        ax1.set_xticklabels(xi, rotation=90)
        for x in xi:
            ax1.axvline(x=x, color='black', linestyle='--', linewidth=1)

        t = np.arange(len(segmentation)) / sr
        plt.plot(t, segmentation, 'r')        
        plt.show()

        if play_audio:
            display(Audio(audio, rate=sr, autoplay=True))



features v3.1


In [3]:
if 'SKIP' not in globals() or not SKIP:     # audio filters
    print('audios filters v2.0')

    TRAIN_AUDIO_FILTERS = {  # list of tuple (start, end) and/or float (time of vocalization)
        'colcha1/XC337020': [(50, 200)],
        'colcha1/XC532406': [(0, 8)],
        'chbant1/XC315058': [(0, 19)],
        '52884/CSA15755': [(9.0, 22.57), (27, 49)],
        'gybmar/XC9608': [(0, 5), (25, 30), (35, 39)],
        '1194042/CSA18783': [(0.9, 18.5), (20, 26)],
        '1194042/CSA18802': [(0.9, 14.2), (25, 30)],
        '1346504/CSA18792': [(0, 21)],
        '134933/iNat1108984': [(1, 6), (11, 16), (21, 27)],
        '134933/iNat1160199': [(0, 20), (29, 47)],
        '21038/iNat297879.ogg': [(0, 12)],
        '21038/iNat65519': [(13, 120), (160, 300)],
        '21116/iNat65520': [(0, 6)],
        '24272/XC882885': [(5, 33), (40, 47), (49, 56)],
        '41778/XC959831': [(20, 35), (50, 75), (80, 123), (145, 170)],
        '42087/iNat155127': [(5, 12)],
        '47067/iNat68676': [(6, 43)],
        '548639/CSA34187': [(0, 8), (5, 10)],
        '555142/iNat31004': [(0, 8)],
        '64862/CSA18218': [(4.5, 22), (98, 135), (154, 161), (211, 235), (270, 290)],
        '64862/CSA18222': [(4.1, 30), (70, 95)],
        '65547/iNat1103224': [(0, 12), (11, 16.8)],
        '714022/CSA34203': [(0, 5.5), (2.5, 11), (8, 17), (17, 25)],
        '714022/CSA34204': [(0,6), (4, 12), (12, 20), (19, 27), (25, 34), (30, 37)],
        '714022/CSA34205': [(0, 5.2), (5, 14), (15.5, 24), (25, 34), (33, 40)],
        '714022/CSA34206': [(0, 7), (6, 15), (15, 23), (22,28)],
        '714022/CSA34207': [(0, 5.5), (7, 16), (18, 26), (29, 36), (35.8, 42)],
        '135045/iNat1122209': [(0, 10), (12, 22), (24, 32), (36, 46)],
        '135045/iNat1207345': [(9, 19), (34, 44), (64, 73), (69, 78), (78, 108), (105, 120), (120, 128)],
        '135045/iNat1207347': [(4, 14), (17, 42), (42, 50), (49.5, 58), (57, 66), (67, 76), (77, 87), (90, 100), (104, 111.4)],
        '135045/iNat1208549': [(9, 19), (27, 36), (51, 60.5), (64, 73), (78, 86.5), (93, 100), (104, 112), (120, 129), (145, 154), (169, 176.8)],
        '135045/iNat1208550': [(0, 7.5), (11, 20), (21, 30), (33, 41), (44.5, 53), (58, 66), (71, 81), (84, 94), (94, 104), (107, 116.5), (120, 130)],
        '135045/iNat1208551': [(6, 16), (15, 52), (55, 65), (67, 97), (101, 111), (116, 126), (128, 149), (147, 161), (160, 169), (170, 183.5), (185, 200)],
        '135045/iNat1208552': [(0,13), (15, 74), (84, 95), (95, 116), (123, 138), (136.5, 148), (149, 159), (158, 169), (171, 183), (189, 203)],
        '135045/iNat1208572': [(0, 30), (30, 41), (39, 51), (57, 71), (74, 84.5), (86.5, 96.5), (97, 107), (106, 117.5), (118, 136), (138, 152), (149, 198), (197, 212.2)],
        '135045/iNat327127': [(0, 9)],
        '135045/iNat48803': [(0, 8), (20.5, 31), (44, 51.3)],
        'norscr1/XC146508': [0, 6, 13, 19, 26, 30, 35, 42, 48, 53, 61, 63, 64, 69, 80, 87, 99, 107],
        'norscr1/XC148047': [2, 6, 20, 24, 28, 41, 46, 57, 65, 69, 76, 103, 108, 112, 116, (118, 136)],
        'norscr1/XC178590': [1, 5, 9, 12, 17, 21, 28, 35, 39, 43, 48, 58, 62, 66, 70, 75, 80, 86, 91],
        'norscr1/XC178594': [2, 5, 10, 17, 25, 31, 40, 44, 50, 55, 63, 68, 74, 80, 91, 98, 103, 108, 111],
        'norscr1/XC178596': [(1, 51.15)],
        'norscr1/iNat31894': [(5, 17.71)],
        
        # 31-Mar
        '52884/CSA18797': [(16, 28), (26, 84), (92, 125), (125, 135), (141, 168), (560, 585)],
        '52884/CSA18801': [(0, 250)],
        '52884/CSA18804': [(0, 140), (560, 700), (740, 870)],
        
        # 2-Apr (large audios)
        'compau/XC837459': [(0, 100), (620, 720), (1120, 1220)],   # not heard; there are pauses
        'greegr/XC558126': [(0, 250)],   # not heard
        'grekis/XC936081': [(0, 250)],   # not heard
        'grekis/XC936811': [(0, 250)],   # not heard
        'saffin/XC879442': [(0, 250)],   # not heard; there are pauses
        'speowl1/XC525219': [(0, 100), (300,400), (600, 700)],   # not heard
        'stbwoo2/XC709416': [(0, 100), (240, 340), (660, 760)],   # not heard
        'yercac1/XC245490': [(0, 255)],   # not heard
    }

    TEMP_TRAIN_AUDIO_FILTERS = {}
    
    ADDITIONAL_AUDIO_FILTERS = {}

    DUPLICATED_AUDIOS = [
        '65547/iNat1103224', '66893/iNat1109827',
    ]

audios filters v2.0


# 2. Config

In [4]:
LOT = 1
FOLDER = '/kaggle/input/b5-data-2021-to-2025/'
FOLDER_ADDITIONAL = '/kaggle/input/bird-clef-2025-add-data'
ROOT = '/kaggle/input'

# 3. Audio builders

In [5]:
if 'previous_filenames' not in globals():
    meta_previous = build_previus_meta()
    meta_previous = meta_previous[meta_previous.primary_label.isin(set(BIRDS))]
    files_previous = [f'/kaggle/input/b5-data-2021-to-2025/{x.replace(".ogg", "")}' \
                          for x in meta_previous.filename.values]
    previous_filenames = set(meta_previous.filename.values)
meta_add, files_add = build_meta_add(FOLDER_ADDITIONAL, previous_filenames)
meta = pd.concat([meta_previous, meta_add])
files = files_previous + files_add

save_obj(meta, 'meta')
meta.to_csv('meta.csv')
print(f'meta len: {len(meta):,} \n')

print(f'files: {len(files):,}  |  {files[0]} - {files[-1]} \n')
audio_filters = complete_audio_filters(ADDITIONAL_AUDIO_FILTERS)
speech_filter = Speech_filter(threads=4)
files_lot = get_files_for_lot(meta, LOT, filename=files)
build_h5_files(files_lot, None, audio_filters, speech_filter)

  0%|          | 0/24352 [00:00<?, ?it/s]

meta len: 7,550 

files: 7,550  |  /kaggle/input/b5-data-2021-to-2025/strher/XC255375 - /kaggle/input/bird-clef-2025-add-data/add_train_audio_from_xeno_canto_28032025/add_train_audio_from_xeno_canto_28032025/greani1/2612.mp3 



Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


  0%|          | 0/3022 [00:00<?, ?it/s]

In [6]:
if 1: # test load
    folder = '.'
    lens = []
    with timer():
        for file in files_lot[:32]:
            id_ = '/'.join(file.split('/')[-2:]).split('.')[0]
            start = 32_000 * 0
            with h5py.File(f'{folder}/{id_}.h5', "r") as f:
                audio = f['raw'][start : start + 5*32_000]
            lens.append(len(audio))
    print(f'mean audio duration: {np.mean(lens) / SR:.2f}')

[]  0.280 s    1.75 GB ->  1.74 GB
mean audio duration: 4.60


____